# 0_数据库连接

In [179]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

from collections import Counter
from numpy import nan as NA
from dateutil.parser import parse
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Data_Clean, Risk_Data
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
clean = Data_Clean()
risk = Risk_Data()

In [180]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="huangxq",
          password="f_epz6hDmZ0Hzi0OzI0N",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 1_数据准备

## 1_11.获取_租后数据

In [181]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name, cc.channel_type_id
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_digua_business.t_order_risk tor on tor.order_id=om.id
;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape

(35514, 66)

## 1_12. 剔除商家数据

In [182]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
   

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(33328, 66)

## 1_13.历史订单渠道名称补充

In [183]:
# 1148 条历史订单渠道名称补充
qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

# qudao_name_df.shape
# qudao_name_df[:2]
# qudao_name_df['渠道名称'].value_counts()

# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
dfzh1["channel_name"].isnull().sum()

# dfzh1[['order_number','channel_name','渠道名称']][:10]
# df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]

np.int64(0)

## 1_14.渠道归属

In [184]:
# df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x:clean.qudao_type(x["channel_name"],x["activity_source"],x["order_method"],x['channel_type_id']),axis=1)

# pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

## 1_15.判断颜色 内存

In [185]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
# dfzh1["颜色"].value_counts()[1:10]

## 1_16.判断二手、带锁

In [186]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
# note 20241124 修改口径，用product_name判断是否二手
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    32450
二手      878
Name: count, dtype: int64

In [187]:
#读取导入最新出库台账数据,2023.9及以后的数据
# （1）512 带锁
# （2）非512的多头带锁
# （3）抖音带锁
# note 20241204
# （1）512 带锁，但每天豁免一台 512P 不带锁，目前还没量
# （2）非512的命中多头提示带锁，但出库是否带锁由人工控制
# （3）抖音带锁（苹果11-15）
# 维客壹佰2023&2024年台账1204.xlsz 为截止到20241203的数据
path = 'F:/myfile/p站数据/台账数据/'
# f_path_ck = "维客壹佰2023&2024年台账1204.xlsx"
f_path_ck = "维客壹佰2025年台账.xlsx"
dfck = pd.read_excel(path + f_path_ck,sheet_name="2025")   #,skiprows=0 header 
# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存']=='512G')|(dfck['内存']=='512GB'), 1, 0)
dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存'].str.contains(r'512')), '带锁', '未带锁')
dfck = dfck[[u'订单号',u'是否带锁']]
dfck.shape
dfck['是否带锁'].value_counts()

dfck.drop_duplicates(subset=["订单号"],inplace=True)

# 合并到主表中去
# dfck 的带锁不一定出库，dfzh1 是出库的
dfck = dfck.rename(columns= {'订单号': 'order_number'})
dfzh1 = dfzh1.merge(dfck, on = 'order_number', how = 'left')
dfzh1['是否带锁'] = dfzh1['是否带锁'].fillna('未带锁') 
dfzh1.shape

pd.pivot_table(dfzh1,index='是否二手',columns='是否带锁',values='order_id',aggfunc='count')

(3388, 2)

是否带锁
未带锁    3233
带锁      155
Name: count, dtype: int64

(33328, 73)

是否带锁,带锁,未带锁
是否二手,,
二手,84,794
全新,67,32383


## 1_17.日期处理

In [188]:
dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

0   2022-07-05
1   2022-07-13
2   2022-07-15
3   2022-07-16
4   2022-07-17
Name: 下单日期, dtype: datetime64[ns]

In [189]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
4季度    11213
3季度     9679
1季度     6272
2季度     6164
Name: count, dtype: int64

## 1_21.获取_买断数据

In [190]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
# note 20241126 增加distinct，买断可能分多次付款
# 181950     2
# 338547     2
# 1083822    2
# 303756     2
SELECT distinct t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(3291, 7)

## 1_22.租后数据 关联 买断数据

In [191]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.shape
dfzh1.shape

(33328, 81)

(33328, 77)

In [192]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

np.int64(4251)

In [193]:
dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()

if_outpay
未知      30154
已买断      3104
部分买断       70
Name: count, dtype: int64

## 1_31.获取_分期取消数据

In [194]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
# 订单取消
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(1924, 7)

## 1_32.租后数据关联分期取消数据

In [195]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.shape
dfzh2.shape

# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

(33328, 85)

(33328, 82)

## 1_41.按风控策略调整时间节点划分时间段

In [196]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

## 1_51.获取_账期数据（含续租）

In [197]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(441734, 23)

In [198]:
# note 20241227 号卡 应付日期更新
# 分期表
sql = '''-- 租后应收监控  
SELECT distinct hk.order_number, tos.order_id, tos.refund_date as refund_date_1
from db_digua_business.t_order_stages tos
inner join
    (select distinct om.id as order_id, om.order_number
    from db_digua_business.t_order om
    left join db_digua_business.t_platform_activity tpa 
    on tpa.id=om.activity_id
    where tpa.type = 4) hk
on tos.order_id = hk.order_id
'''
tmp_hk = query(sql)
tmp_hk['refund_date_ym'] = tmp_hk['refund_date_1'].dt.strftime('%Y-%m')
tmp_hk.shape
tmp_hk[:2]

df_xzfq['refund_date_ym'] = df_xzfq['refund_date'].dt.strftime('%Y-%m')
df_xzfq = df_xzfq.merge(tmp_hk[['order_id', 'refund_date_1', 'refund_date_ym']], on = ['order_id', 'refund_date_ym'], how = 'left')
df_xzfq['refund_date_2'] = np.where(df_xzfq.refund_date_1.notnull(), df_xzfq.refund_date_1, df_xzfq.refund_date)
df_xzfq.shape

df_xzfq.drop(columns=['refund_date_1','refund_date_ym','refund_date'],inplace = True)
df_xzfq = df_xzfq.rename(columns = {'refund_date_2':'refund_date'})
df_xzfq.shape

(4500, 4)

,order_number,order_id,refund_date_1,refund_date_ym
0,A20240927154904302,1533488,2024-09-30,2024-09
1,A20240927154904302,1533488,2024-10-30,2024-10


(441734, 26)

(441734, 23)

## 1_52.重命名

In [199]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期','reality_refund_date':'实付日期','sort':'当前期数'})

## 1_53.实还金额核查

In [200]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

## 1_61.数据合并

In [201]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')
# note 20241126 增加
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape
# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228
df_xz_concat.实付金额 = df_xz_concat.实付金额 + df_xz_concat.sesame_promo_money_pay + df_xz_concat.promo_money
df_xz_concat = df_xz_concat[df_xz_concat.当前应付租金.notnull()]

# order_id == 1681669 的当前期数为空
# 1681669	A20241104144812228
# 1669581	A2024110109350797
df_xz_concat['当前期数'].notnull().sum()
df_xz_concat.shape
df_xzfq.shape
dfzh3.shape

# df_xz_concat.drop_duplicates(subset=["order_id"]).shape
# dfzh3.drop_duplicates(subset=["order_id"]).shape

# df_xz_concat['order_id'].nunique()
# dfzh3['order_id'].nunique()
# df_xz_concat[["order_id","当前期数"]].info()

np.int64(441667)

(441667, 107)

(441734, 23)

(33328, 85)

## 1_62.去重

In [202]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
# print(all_duplicates.sort_values(by=['order_id','当前期数']))

In [203]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)
df_xz_concat.shape

(441487, 107)

In [204]:
# 119321
# 1083822 303756
# all_duplicates[all_duplicates['order_id'] == 181950].shape
# df_xz_concat[df_xz_concat['order_id'] == 181950].shape

# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

# 2_计算逾期天数

## 2_11.异常查询

In [205]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# 实付金额>0 实付日期为空 finish_date_new非空 :实付日期=订单完成时间
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# part_payment==0 reality_refund_date.notnull() 还款状态== '续租中' part_payment=money 还款状态== '已逾期' reality_refund_date=pd.na
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# 实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消
# dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 部分还款，实付日期非空，完成时间为空 用户： 实付日期清空
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1

# 部分还款，实付日期为空，完成时间非空 用户： 实付日期=完成时间
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1.订单状态值.value_counts()

# 已买断 但无实付日期及金额：实付日回写
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# # dd_test1.订单状态值.value_counts()
# dd_test1

## 2_12.实还日期修正

In [206]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [207]:
# （1）部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)

df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()     

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
331555,A2024110123471583,1672926,499.85,499.85,2025-02-04,2025-02-01,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
335178,A20241106135426228,1689980,499.85,499.85,2024-12-09,2024-12-08,3.0,2.0,已逾期,NaT,4,NaT,NaN,NaT
335179,A20241106135426228,1689980,499.85,499.85,2025-01-09,2025-01-05,3.0,3.0,已逾期,NaT,4,NaT,NaN,NaT
335743,A2024110708472957,1692436,457.98,457.98,2025-01-10,2025-01-08,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
335744,A2024110708472957,1692436,457.98,457.98,2025-02-10,2025-02-08,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT


In [208]:
# （2）已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()       

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
335179,A20241106135426228,1689980,499.85,499.85,2025-01-09,2025-01-05,3.0,3.0,已逾期,NaT,4,NaT,NaN,NaT
335743,A2024110708472957,1692436,457.98,457.98,2025-01-10,2025-01-08,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
335744,A2024110708472957,1692436,457.98,457.98,2025-02-10,2025-02-08,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
372825,A20241210113506313,1839860,695.21,665.65,2025-01-13,NaT,1.0,2.0,还款中,NaT,4,NaT,NaN,NaT
372826,A20241210113506313,1839860,695.21,665.65,2025-02-13,NaT,1.0,3.0,还款中,NaT,4,NaT,NaN,NaT


In [209]:
# （3）租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60964,2024-07-22


In [210]:
# （4）未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
8862,A2023021015233353,125911,633.21,0.0,2023-03-13,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9558,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9636,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
10650,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
12255,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17,1.0,11.0,已买断,NaT,8,NaT,NaN,NaT


In [211]:
# （5）实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] .tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
182427,A2024041316125534,995895,1061.16,0.0,2024-12-16,NaT,1.0,9.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
182428,A2024041316125534,995895,1061.16,0.0,2025-01-16,NaT,1.0,10.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
182429,A2024041316125534,995895,1061.16,0.0,2025-02-16,NaT,1.0,11.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
182430,A2024041316125534,995895,1063.94,0.0,2025-03-16,NaT,1.0,12.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
212210,A2024060716074819,1111255,1085.68,0.0,2025-05-10,NaT,1.0,12.0,已买断,2024-12-14 14:59:21,8,2024-12-14,7.0,2024-12-14


In [212]:
# （6）部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [213]:
# （7）未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [214]:
# （8）实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [215]:
# （9）属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [216]:
# （10）df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [217]:
# （11）A202207050851373、A20241104144812228 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number.isin(['A202207050851373','A20241104144812228'])),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [218]:
# （12）实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)

In [219]:
# （13）df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 2_13.获取逾期截止时间

In [220]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date'].head()

0   2025-02-25
1   2025-02-25
2   2025-02-25
3   2025-02-25
4   2025-02-25
Name: now_date, dtype: datetime64[s]

## 2_14.重新定义状态

In [221]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

rembursement_status
还款中                    88305
已逾期                    44391
续租中                    16830
未到首期还款日                15835
检测中                       48
Name: count, dtype: int64

订单状态
租赁中     156186
待归还       6182
已退款       4584
待收货       2024
待发货        878
已完成        220
检测中         79
订单取消        60
Name: count, dtype: int64

In [222]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     209620
未到还款日    166858
已逾期支付     31417
已逾期       28788
已取消        4804
Name: count, dtype: int64

In [223]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 2_15.重新定义逾期天数

In [224]:
df_xz_concat['实付日期new'].isnull().sum()
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

np.int64(170213)

Series([], Name: order_number, dtype: object)

In [225]:
# 异常值删除，应付时间为空
df_xz_concat.shape
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(441487, 111)

(441487, 111)

In [226]:
#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 
#note 20241127 逾期天数 截止目前的逾期天数 ；逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][['order_number','逾期天数','应付日期','']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092919033313'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

# df_xz_concat['逾期天数'].value_counts()
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        42248
还款中         6505
续租中         5293
已买断         5106
已完成          865
检测中          148
未到首期还款日       27
Name: count, dtype: int64

In [227]:
# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

## 2_16.逾期天数处理

In [228]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

## 2_17.稽查

In [229]:
# df_ddfq.columns
# df_xz_concat.shape
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape
# check_fq_df.columns
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')


## 2_18.确认是否到表现期

In [230]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [231]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [232]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 

In [233]:
# from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','agr_15d','agr_7d','agr_4d','agr_1d','应付日期']].head()

,agr_30d,agr_15d,agr_7d,agr_4d,agr_1d,应付日期
0,2022-08-04,2022-07-20,2022-07-12,2022-07-09,2022-07-06,2022-07-05
1,2022-09-04,2022-08-20,2022-08-12,2022-08-09,2022-08-06,2022-08-05
2,2022-10-05,2022-09-20,2022-09-12,2022-09-09,2022-09-06,2022-09-05
3,2022-11-04,2022-10-20,2022-10-12,2022-10-09,2022-10-06,2022-10-05
4,2022-12-05,2022-11-20,2022-11-12,2022-11-09,2022-11-06,2022-11-05


In [234]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(5)
# df_xz_concat[(df_xz_concat.agr_7d_cum == 0) & (df_xz_concat.agr_4d_cum == 1)].head()
# df_xz_concat[df_xz_concat.order_id == 254588]

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2025-02-25,1
1,2022-09-04,2022-08-05,2025-02-25,1
2,2022-10-05,2022-09-05,2025-02-25,1
3,2022-11-04,2022-10-05,2025-02-25,1
4,2022-12-05,2022-11-05,2025-02-25,1


In [235]:
# note 20241211 数据检查
# df_xz_concat[(df_xz_concat.应付日期>='2024-12-01')&(df_xz_concat.应付日期<='2024-12-31')&(df_xz_concat.order_number.isin(
# ['A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# ]))][['order_number','agr_1d_cum','agr_4d_cum','agr_7d_cum','agr_4d','应付日期','date_str','agr_1d','下单日期']]

## 2_19.观察日定义 每月月底时间

In [236]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 月末的的跳级了，所以不要加1，mob0对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat = mob_date(df_xz_concat)


# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

# A202301011402371 2023-01-01
# A202301290815565 2023-01-29
# A2023013100341134 2023-01-31
# A2023022801142011 2023-02-28
# A2023022709283120  2023-02-27
# A202302050022319  2023-02-05
# A2023040503314720 2023-04-05
# A2023042803470516  2023-04-28
# A2023043000524068 2023-04-30
# A2023063003392242 2023-06-30
# A202306280105556 2023-06-28
# A202306050324407  2023-06-05
# A202310050100222 2023-10-05
# A2023102901224646 2023-10-29
# A2023103101255825 2023-10-31

In [237]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

## 2_20.更新逾期

In [238]:
# （1）逾期天数校正
# note 20241213
# 首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0（各期更新为0）

# 把符合条件的订单拿出来，逾期天数全部更新为0
sql_xz = ''' 
SELECT distinct tprm.order_number,1 as label
# ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
where ymos.sort=1 and ymos.reality_refund_date is null
;
'''
tmp = query(sql_xz)
tmp.shape
# tmp[:2]

df_xz_concat = df_xz_concat.merge(tmp, on = 'order_number', how = 'left')
df_xz_concat['label'] = df_xz_concat['label'].fillna(0)
df_xz_concat['label'].value_counts()

df_xz_concat[df_xz_concat['label'] == 1].drop_duplicates(subset = 'order_number').shape

df_xz_concat['逾期天数'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat['逾期天数_new'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].shape
# df_xz_concat[(df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']) & (df_xz_concat.逾期天数 > 30)].shape
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']]
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.max()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.min()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']].to_excel('tmp.xlsx', index = False)

# （2）2025.2.21
df_xz_concat['逾期天数'] = np.where(df_xz_concat.当前期数 == 1, 0, df_xz_concat.逾期天数)

(727, 2)

label
0.0    432907
1.0      8580
Name: count, dtype: int64

(706, 163)

# 3_逾期加工

In [239]:
# （1）历史最大逾期
df_maxoverdue = df_xz_concat.groupby(['order_number','order_id','下单日期','search_time','归属渠道','是否二手','是否带锁']).agg({'逾期天数':'max'}).reset_index()
df_maxoverdue.shape
df_maxoverdue[:1]

(33314, 8)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数
0,A202207050851373,8924,2022-07-05,2022-07,未知渠道,二手,未带锁,4


In [240]:
# （2）FPD SPD TPD
df_xz_concat['当前期数'] = df_xz_concat['当前期数'].astype(int)

df_fstq = df_xz_concat[["order_id","当前期数","逾期天数"]].pivot(index="order_id", columns="当前期数", values="逾期天数")
df_fstq = df_fstq.reset_index()
df_fstq.shape

# df_fstq = df_fstq.rename(columns = {2: 'FPD'})
df_fstq['FPD'] = df_fstq[2]
df_fstq.loc[:,"SPD"] = np.where( df_fstq['FPD'] == 0, df_fstq[3], 0)
df_fstq.loc[:,"TPD"] = np.where((df_fstq['FPD'] == 0) & (df_fstq[3] == 0), df_fstq[4], 0)

df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])][['order_id','FPD','SPD','TPD']]

(33314, 37)

当前期数,order_id,FPD,SPD,TPD
28,12449,0.0,11.0,0.0
3719,508404,0.0,0.0,0.0
24300,1678492,81.0,0.0,0.0
24564,1692916,8.0,0.0,0.0


In [241]:
# （3）第2-4期逾期天数
df_fstq = df_fstq.rename(columns = {2: '第二期逾期天数', 3: '第三期逾期天数', 4: '第四期逾期天数'})
df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数']]
tmp = df_xz_concat[df_xz_concat['当前期数'] == 1][['order_id','应付日期']]
df_fstq = df_fstq.merge(tmp, on = 'order_id', how = 'left')
df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])]
# df_xz_concat[df_xz_concat.order_number == 'A2023073114301076'][['order_id','下单日期','当前期数','应付日期']]
# A2023073114301076 508404
# df_fstq.FPD.max()

,order_id,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,应付日期
28,12449,0.0,11.0,0.0,0.0,11.0,9.0,2022-08-05
3719,508404,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-03
24300,1678492,81.0,0.0,0.0,81.0,50.0,19.0,2024-11-06
24564,1692916,8.0,0.0,0.0,8.0,0.0,15.0,2024-11-10


In [242]:
# （4）表关联 
df_analy_data = df_maxoverdue.merge(df_fstq, on = 'order_id', how = 'left')
df_analy_data = df_analy_data.rename(columns = {'逾期天数':'历史最大逾期天数','应付日期': '首期应还日期'})
df_analy_data = df_analy_data.merge(df_zhys[['order_number','overdue_type']], on = 'order_number', how = 'left')

df_analy_data.shape
df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

(33319, 16)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,首期应还日期,overdue_type
28,A202208052103231,12449,2022-08-05,2022-08,顶部搜索框的搜索结果页,全新,未带锁,19,0.0,11.0,0.0,0.0,11.0,9.0,2022-08-05,M6
3719,A2023073114301076,508404,2023-07-31,2023-07,芝麻租物,全新,未带锁,6,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-03,
24305,A20241103155713186,1678492,2024-11-03,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,81,81.0,0.0,0.0,81.0,50.0,19.0,2024-11-06,M2
24569,A20241107103123201,1692916,2024-11-07,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,15,8.0,0.0,0.0,8.0,0.0,15.0,2024-11-10,M0


In [243]:
df_analy_data[df_analy_data.历史最大逾期天数 >30].shape
df_analy_data[df_analy_data.历史最大逾期天数 >90].shape
df_analy_data[df_analy_data.历史最大逾期天数 >180].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 <= '2023-12-31')].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 >= '2024-01-01')&(df_analy_data.下单日期 <= '2024-12-31')].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 >= '2025-01-01')].shape

(6420, 16)

(4602, 16)

(3152, 16)

(2524, 16)

(628, 16)

(0, 16)

# 4_线下门店数据分析

In [244]:
# （1）20241211 
# 力哥 新引进一个线下渠道，对比历史线下门店的租后表现 
# activityIds: 287,151,150,149,148,147,146,138,81
# t_order里面 activity_id = 上面这些的

# M0：1-30
# M1:31-60
# M2：61-90
# M3:91-180
# M6：>180

sql = '''-- 租后应收监控  
SELECT  tprm.order_id,tprm.max_overdue_days,tprm.order_time,tprm.unrecovered_cost,tprm.purchase_amount,tprm.overdue_type
,om.activity_id
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
;
'''

df_xxmd = query(sql)
df_xxmd.overdue_type = df_xxmd.overdue_type.fillna('未逾期')
df_xxmd.shape

df_xxmd[df_xxmd.activity_id.isin([287,151,150,149,148,147,146,138,81])].activity_id.value_counts()
df_xxmd[df_xxmd.activity_id.isin([287,151,150,149,148,147,146,138,81])].sort_values(by = 'order_time')
conditions = (df_xxmd.activity_id.isin([287,151,150,149,148,147,146,138,81]))&(df_xxmd.unrecovered_cost > 0)
tmp1 = df_xxmd[df_xxmd.activity_id.isin([287,151,150,149,148,147,146,138,81])].groupby(['order_time']).purchase_amount.sum()
tmp2 = pd.pivot_table(df_xxmd[conditions],index='order_time',columns='overdue_type',values='unrecovered_cost',aggfunc='sum').reset_index()
tmp1
tmp2

(35514, 7)

activity_id
146    10
138     4
151     4
287     1
Name: count, dtype: int64

,order_id,max_overdue_days,order_time,unrecovered_cost,purchase_amount,overdue_type,activity_id
3357,470053,0,2023年07月,-2400.00,7999.0,,138
4051,521241,8,2023年08月,-1290.00,4299.0,,138
4284,538423,140,2023年08月,-2967.70,9899.0,,146
5175,603100,35,2023年09月,-1945.04,6299.0,,146
5203,605480,438,2023年09月,1699.06,2999.0,M6,151
5281,611751,496,2023年09月,5349.35,5999.0,M6,146
5536,631974,10,2023年09月,-2715.45,8999.0,,146
5686,643889,333,2023年09月,2936.45,8999.0,M6,146
6648,715490,367,2023年10月,1024.52,3299.0,M6,151
6425,701038,75,2023年10月,-1001.69,3299.0,,151


order_time
2023年07月     7999.0
2023年08月    14198.0
2023年09月    33295.0
2023年10月    32295.0
2023年11月     9999.0
2023年12月    17597.0
2024年01月     9999.0
2024年08月     8108.0
Name: purchase_amount, dtype: float64

overdue_type,order_time,,M6
0,2023年09月,NaN,9984.86
1,2023年10月,NaN,6632.74
2,2023年11月,NaN,6376.38
3,2024年08月,528.12,NaN


# 5_模型需求

In [245]:
# # （1）20241211 更新租后表现
# # 更新历史最大逾期、第2-4期的逾期天数
# path = 'C:/Users/Administrator/Desktop/mine/3_3临时分析需求/1.何老师/'
# data_h = pd.read_excel(path + '出库单三方数据表现-1114(1).xlsx')
# data_h.shape
# data_h = data_h[['order_number']]
# data_h.drop_duplicates(subset = 'order_number').shape
# # 订单号有重复
# # A20240923145253318

# data_h = data_h.merge(df_analy_data[['order_number','首期应还日期','历史最大逾期天数','第二期逾期天数','第三期逾期天数','第四期逾期天数']], on = 'order_number', how = 'left')
# data_h.shape
# data_h.tail()

# # （2）号卡剔除 --手动改一下
# # sql_xz = ''' 
# # select 
# # om.create_time 下单时间, om.order_number, om.id as order_id
# # from db_digua_business.t_order om
# # left join db_digua_business.t_platform_activity tpa on tpa.id=om.activity_id
# # where tpa.type=4
# # '''
# # query(sql_xz)

# # （3）首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0
# sql_xz = ''' 
# SELECT distinct tprm.order_number,1 as label
# # ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
# from db_rent.ya_merchant_order_stages ymos 
# inner join db_digua_business.t_postlease_receivables_monitoring tprm 
# on ymos.order_id = tprm.order_id 
# where ymos.sort=1 and ymos.reality_refund_date is null
# ;
# '''
# tmp = query(sql_xz)
# tmp.shape
# tmp[:2]

# data_h = data_h.merge(tmp, on = 'order_number', how = 'left')
# data_h['label'] = data_h['label'].fillna(0)
# data_h['label'].value_counts()

# data_h['历史最大逾期天数'] = np.where(data_h['label'] == 1, 0 , data_h['历史最大逾期天数'])
# data_h['第二期逾期天数'] = np.where(data_h['历史最大逾期天数'] == 0, 0, data_h['第二期逾期天数'])
# data_h['第三期逾期天数'] = np.where(data_h['历史最大逾期天数'] == 0, 0, data_h['第三期逾期天数'])
# data_h['第四期逾期天数'] = np.where(data_h['历史最大逾期天数'] == 0, 0, data_h['第四期逾期天数'])

# data_h['首期应还日期'] = data_h['首期应还日期'].astype('str')
# data_h.to_excel(path + '出库订单租后表现更新20241212v2.xlsx', index = False)

In [246]:
# # （2）20250120 模型验证 有y
# # 建模时点：2024.11.3 下单，2024.12.13 是 FPD7+，2025.1.13是F2PD7+
# # 首期应还款日：2024.9.3-2024.10.2 的样本用来验证 f3pd7、f2pd7、fpd7 的
# # 首期应还款日：2024.10.3-2024.11.2 的样本用来验证 f2pd7、fpd7 的
# # 首期应还款日：2024.11.3-2024.12.14 的样本用来验证 fpd7 的 -- 20250121 分析

# # （2-1）导入模型评分

# # df_analy_data.shape
# # df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

# path = 'C:/Users/Administrator/Desktop/mine/11_模型/2025.1/2.模型结果/'
# # 2024.9.3-2024.11.2
# df_model_y = pd.read_excel(path + 'oot.xlsx')
# # df_model_y_no = pd.read_excel(path + '无y样本.xlsx')
# df_model_y.shape
# # df_model_y_no.shape
# # df_model_y = pd.concat([df_model_y, df_model_y_no])
# # df_model_y.shape

# df_model_y = df_model_y.merge(df_analy_data, on = 'order_number', how = 'left')
# df_model_y['ascore2501_base_Bin'] = df_model_y.apply(lambda x: '1.(17.691, 100.0]' if x.ascore2501_base > 17.691 and x.ascore2501_base <= 100
#                                                      else '2.(15.259, 17.691]' if x.ascore2501_base > 15.259 and x.ascore2501_base <= 17.691
#                                                      else '3.(13.79, 15.259]' if x.ascore2501_base > 13.79 and x.ascore2501_base <= 15.259
#                                                      else '4.(12.622, 13.79]' if x.ascore2501_base > 12.622 and x.ascore2501_base <= 13.79
#                                                      else '5.(11.635, 12.622]' if x.ascore2501_base > 11.635  and x.ascore2501_base <= 12.622
#                                                      else '6.(10.803, 11.635]' if x.ascore2501_base > 10.803  and x.ascore2501_base <= 11.635
#                                                      else '7.(10.047, 10.803]' if x.ascore2501_base > 10.047  and x.ascore2501_base <= 10.803
#                                                      else '8.(9.17, 10.047]' if x.ascore2501_base > 9.17  and x.ascore2501_base <= 10.047
#                                                      else '9.(8.137, 9.17]' if x.ascore2501_base > 8.137  and x.ascore2501_base <= 9.17
#                                                      else '10.(0.0, 8.137]' if x.ascore2501_base > 0  and x.ascore2501_base <= 8.137
#                                                      else 'other'
#                                                      , axis = 1
#                                                      )
# df_model_y['ascore2501_pro_Bin'] = df_model_y.apply(lambda x: '1.(17.028, 100.0]' if x.ascore2501_pro > 17.028 and x.ascore2501_pro <= 100
#                                                      else '2.(15.24, 17.028]' if x.ascore2501_pro > 15.24 and x.ascore2501_pro <= 17.028
#                                                      else '3.(14.059, 15.24]' if x.ascore2501_pro > 14.059 and x.ascore2501_pro <= 15.24
#                                                      else '4.(13.182, 14.059]' if x.ascore2501_pro > 13.182 and x.ascore2501_pro <= 14.059
#                                                      else '5.(12.394, 13.182]' if x.ascore2501_pro > 12.394  and x.ascore2501_pro <= 13.182
#                                                      else '6.(11.702, 12.394]' if x.ascore2501_pro > 11.702  and x.ascore2501_pro <= 12.394
#                                                      else '7.(11.13, 11.702]' if x.ascore2501_pro > 11.13 and x.ascore2501_pro <= 11.702
#                                                      else '8.(10.531, 11.13]' if x.ascore2501_pro > 10.531  and x.ascore2501_pro <= 11.13
#                                                      else '9.(9.835, 10.531]' if x.ascore2501_pro > 9.835  and x.ascore2501_pro <= 10.531
#                                                      else '10.(0.0, 9.835]' if x.ascore2501_pro > 0  and x.ascore2501_pro <= 9.835
#                                                      else 'other'
#                                                      , axis = 1
#                                                      )

# df_model_y['f2pd7'] = np.where(df_model_y.第二期逾期天数 >= df_model_y.第三期逾期天数, df_model_y.第二期逾期天数, df_model_y.第三期逾期天数)
# df_model_y['f3pd7'] = np.where(df_model_y.f2pd7 >= df_model_y.第四期逾期天数, df_model_y.f2pd7, df_model_y.第四期逾期天数)
# df_model_y['y_fraud_f2pd7'] = df_model_y.apply(lambda x: 1 if x.f2pd7 >= 7 
#                                                else 0 if x.f2pd7 >= 0 and x.f2pd7 <= 6
#                                                else 999, axis = 1)
# df_model_y['y_fraud_f3pd7'] = df_model_y.apply(lambda x: 1 if x.f3pd7 >= 7 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 6
#                                                else 999, axis = 1)
# df_model_y['y_fraud_fpd7'] = df_model_y.apply(lambda x: 1 if x.FPD >= 7 
#                                                else 0 if x.FPD >= 0 and x.FPD <= 6
#                                                else 999, axis = 1)

# df_model_y.shape
# df_model_y[:1]

# # （2-2）统计
# conditions_1 = (df_model_y.首期应还日期 >= '2024-09-03') & (df_model_y.首期应还日期 <= '2024-10-02')
# conditions_2 = (df_model_y.首期应还日期 >= '2024-10-03') & (df_model_y.首期应还日期 <= '2024-11-02')
# conditions_3 = (df_model_y.ascore2501_base <= 17.691)

# # df_model_y.search_time.value_counts()
# # df_model_y.下单日期.min()
# # df_model_y.下单日期.max()

# # （2-2-1）ascore2501_base_Bin
# pd.pivot_table(df_model_y[conditions_1],index='ascore2501_base_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_1],index='ascore2501_base_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_1],index='ascore2501_base_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# pd.pivot_table(df_model_y[conditions_2],index='ascore2501_base_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_2],index='ascore2501_base_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # （2-2-2）ascore2501_pro_Bin
# pd.pivot_table(df_model_y[conditions_1],index='ascore2501_pro_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_1],index='ascore2501_pro_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_1],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# pd.pivot_table(df_model_y[conditions_2],index='ascore2501_pro_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_2],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # （2-2-3）去掉 ascore2501_base 后的 ascore2501_pro
# pd.pivot_table(df_model_y[conditions_1 & conditions_3],index='ascore2501_pro_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_1 & conditions_3],index='ascore2501_pro_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_1 & conditions_3],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# pd.pivot_table(df_model_y[conditions_2 & conditions_3],index='ascore2501_pro_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_2 & conditions_3],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

In [247]:
# # （3）20250120 模型验证 无y
# # （3-1）导入模型评分
# path = 'C:/Users/Administrator/Desktop/mine/11_模型/2025.1/2.模型结果/'
# df_model_y_no = pd.read_excel(path + '无y样本.xlsx')
# df_model_y_no = df_model_y_no.merge(df_analy_data[['order_number','首期应还日期','FPD','第二期逾期天数','第三期逾期天数','第四期逾期天数']], on = 'order_number', how = 'left')
# df_model_y_no.shape

# # （3-2）下单日期
# sql = '''-- 租后应收监控  
# SELECT  om.order_number, om.create_time
# from db_digua_business.t_order om
# ;
# '''
# df_info = query(sql)
# df_info["下单日期"] = df_info['create_time'].dt.date
# df_info["下单日期"] = pd.to_datetime(df_info["下单日期"],errors="coerce")
# df_info.shape

# # （3-3）宽表合并
# df_model_y_no = df_model_y_no.merge(df_info, on = 'order_number', how = 'left')
# df_model_y_no['ascore2501_base_Bin'] = df_model_y_no.apply(lambda x: '1.(17.691, 100.0]' if x.ascore2501_base > 17.691 and x.ascore2501_base <= 100
#                                                      else '2.(15.259, 17.691]' if x.ascore2501_base > 15.259 and x.ascore2501_base <= 17.691
#                                                      else '3.(13.79, 15.259]' if x.ascore2501_base > 13.79 and x.ascore2501_base <= 15.259
#                                                      else '4.(12.622, 13.79]' if x.ascore2501_base > 12.622 and x.ascore2501_base <= 13.79
#                                                      else '5.(11.635, 12.622]' if x.ascore2501_base > 11.635  and x.ascore2501_base <= 12.622
#                                                      else '6.(10.803, 11.635]' if x.ascore2501_base > 10.803  and x.ascore2501_base <= 11.635
#                                                      else '7.(10.047, 10.803]' if x.ascore2501_base > 10.047  and x.ascore2501_base <= 10.803
#                                                      else '8.(9.17, 10.047]' if x.ascore2501_base > 9.17  and x.ascore2501_base <= 10.047
#                                                      else '9.(8.137, 9.17]' if x.ascore2501_base > 8.137  and x.ascore2501_base <= 9.17
#                                                      else '10.(0.0, 8.137]' if x.ascore2501_base > 0  and x.ascore2501_base <= 8.137
#                                                      else 'other'
#                                                      , axis = 1
#                                                      )
# df_model_y_no['ascore2501_pro_Bin'] = df_model_y_no.apply(lambda x: '1.(17.028, 100.0]' if x.ascore2501_pro > 17.028 and x.ascore2501_pro <= 100
#                                                      else '2.(15.24, 17.028]' if x.ascore2501_pro > 15.24 and x.ascore2501_pro <= 17.028
#                                                      else '3.(14.059, 15.24]' if x.ascore2501_pro > 14.059 and x.ascore2501_pro <= 15.24
#                                                      else '4.(13.182, 14.059]' if x.ascore2501_pro > 13.182 and x.ascore2501_pro <= 14.059
#                                                      else '5.(12.394, 13.182]' if x.ascore2501_pro > 12.394  and x.ascore2501_pro <= 13.182
#                                                      else '6.(11.702, 12.394]' if x.ascore2501_pro > 11.702  and x.ascore2501_pro <= 12.394
#                                                      else '7.(11.13, 11.702]' if x.ascore2501_pro > 11.13 and x.ascore2501_pro <= 11.702
#                                                      else '8.(10.531, 11.13]' if x.ascore2501_pro > 10.531  and x.ascore2501_pro <= 11.13
#                                                      else '9.(9.835, 10.531]' if x.ascore2501_pro > 9.835  and x.ascore2501_pro <= 10.531
#                                                      else '10.(0.0, 9.835]' if x.ascore2501_pro > 0  and x.ascore2501_pro <= 9.835
#                                                      else 'other'
#                                                      , axis = 1
#                                                      )

# df_model_y_no['f2pd7'] = np.where(df_model_y_no.第二期逾期天数 >= df_model_y_no.第三期逾期天数, df_model_y_no.第二期逾期天数, df_model_y_no.第三期逾期天数)
# df_model_y_no['f3pd7'] = np.where(df_model_y_no.f2pd7 >= df_model_y_no.第四期逾期天数, df_model_y_no.f2pd7, df_model_y_no.第四期逾期天数)
# df_model_y_no['y_fraud_f2pd7'] = df_model_y_no.apply(lambda x: 1 if x.f2pd7 >= 7 
#                                                else 0 if x.f2pd7 >= 0 and x.f2pd7 <= 6
#                                                else 999, axis = 1)
# df_model_y_no['y_fraud_f3pd7'] = df_model_y_no.apply(lambda x: 1 if x.f3pd7 >= 7 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 6
#                                                else 999, axis = 1)
# df_model_y_no['y_fraud_fpd7'] = df_model_y_no.apply(lambda x: 1 if x.FPD >= 7 
#                                                else 0 if x.FPD >= 0 and x.FPD <= 6
#                                                else 999, axis = 1)
# df_model_y_no.shape
# df_model_y_no[:1]

# # （3-4）统计
# # （3-4-1）有标签 -- 首期应还日期为 2024.11.3-2024.12.14
# conditions_4 = (df_model_y_no.首期应还日期 >= '2024-11-03') & (df_model_y_no.首期应还日期 <= '2024-12-14')
# conditions_5 = (df_model_y_no.ascore2501_base <= 17.691)
# conditions_6 = (df_model_y_no.下单日期 >= '2024-12-15')

# pd.pivot_table(df_model_y_no[conditions_4],index='ascore2501_base_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y_no[conditions_4],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y_no[conditions_4 & conditions_5],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # （3-4-2）无标签 -- 进件日期为 2024.12.15-2025.1.14
# pd.pivot_table(df_model_y_no[conditions_6],index='下单日期',columns='ascore2501_base_Bin',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y_no[conditions_6],index='下单日期',columns='ascore2501_pro_Bin',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y_no[conditions_6 & conditions_5],index='下单日期',columns='ascore2501_pro_Bin',values='order_number',aggfunc='count')

In [248]:
# # （4）20250205 模型验证-细分渠道 有y
# # 建模时点：2024.11.3 下单，2024.12.13 是 FPD7+，2025.1.13是F2PD7+
# # 首期应还款日：2024.9.3-2024.10.2 的样本用来验证 f3pd7、f2pd7、fpd7 的
# # 首期应还款日：2024.10.3-2024.11.2 的样本用来验证 f2pd7、fpd7 的
# # 首期应还款日：2024.11.3-2024.12.14 的样本用来验证 fpd7 的 -- 20250205 分析,2024.11.2的fpd3还未完全表现出来，故不修改时间范围

# # （2-1）导入模型评分

# # df_analy_data.shape
# # df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

# path = 'C:/Users/Administrator/Desktop/mine/11_模型/2025.1/2.模型结果/'
# # 2024.9.3-2024.11.2
# df_model_y = pd.read_excel(path + 'oot.xlsx')
# # df_model_y_no = pd.read_excel(path + '无y样本.xlsx')
# df_model_y.shape
# # df_model_y_no.shape
# # df_model_y = pd.concat([df_model_y, df_model_y_no])
# # df_model_y.shape

# df_model_y = df_model_y.merge(df_analy_data, on = 'order_number', how = 'left')
# # 租物渠道 17.8051-13.73 19.955-14.104
# df_model_y['ascore2501_base_Bin'] = df_model_y.apply(lambda x: '1.(18.1312, 100.0]' if x.ascore2501_base > 18.1312 and x.ascore2501_base <= 100
#                                                      else '2.(15.259, 18.1312]' if x.ascore2501_base > 15.259 and x.ascore2501_base <= 18.1312
#                                                      else '3.(14.104, 15.259]' if x.ascore2501_base > 14.104 and x.ascore2501_base <= 15.259
#                                                      else '4.(12.622, 14.104]' if x.ascore2501_base > 12.622 and x.ascore2501_base <= 14.104
#                                                      else '5.(11.635, 12.622]' if x.ascore2501_base > 11.635  and x.ascore2501_base <= 12.622
#                                                      else '6.(10.803, 11.635]' if x.ascore2501_base > 10.803  and x.ascore2501_base <= 11.635
#                                                      else '7.(10.047, 10.803]' if x.ascore2501_base > 10.047  and x.ascore2501_base <= 10.803
#                                                      else '8.(9.17, 10.047]' if x.ascore2501_base > 9.17  and x.ascore2501_base <= 10.047
#                                                      else '9.(8.137, 9.17]' if x.ascore2501_base > 8.137  and x.ascore2501_base <= 9.17
#                                                      else '91.(0.0, 8.137]' if x.ascore2501_base > 0  and x.ascore2501_base <= 8.137
#                                                      else 'other'
#                                                      , axis = 1
#                                                      )
# # 非租物渠道 17.691 17.8016 18.098
# # df_model_y['ascore2501_base_Bin'] = df_model_y.apply(lambda x: '1.(18.098, 100.0]' if x.ascore2501_base > 18.098 and x.ascore2501_base <= 100
# #                                                      else '2.(15.259, 18.098]' if x.ascore2501_base > 15.259 and x.ascore2501_base <= 18.098
# #                                                      else '3.(13.79, 15.259]' if x.ascore2501_base > 13.79 and x.ascore2501_base <= 15.259
# #                                                      else '4.(12.622, 13.79]' if x.ascore2501_base > 12.622 and x.ascore2501_base <= 13.79
# #                                                      else '5.(11.635, 12.622]' if x.ascore2501_base > 11.635  and x.ascore2501_base <= 12.622
# #                                                      else '6.(10.803, 11.635]' if x.ascore2501_base > 10.803  and x.ascore2501_base <= 11.635
# #                                                      else '7.(10.047, 10.803]' if x.ascore2501_base > 10.047  and x.ascore2501_base <= 10.803
# #                                                      else '8.(9.17, 10.047]' if x.ascore2501_base > 9.17  and x.ascore2501_base <= 10.047
# #                                                      else '9.(8.137, 9.17]' if x.ascore2501_base > 8.137  and x.ascore2501_base <= 9.17
# #                                                      else '91.(0.0, 8.137]' if x.ascore2501_base > 0  and x.ascore2501_base <= 8.137
# #                                                      else 'other'
# #                                                      , axis = 1
# #                                                      )
# # df_model_y['ascore2501_pro_Bin'] = df_model_y.apply(lambda x: '1.(17.028, 100.0]' if x.ascore2501_pro > 17.028 and x.ascore2501_pro <= 100
# #                                                      else '2.(15.24, 17.028]' if x.ascore2501_pro > 15.24 and x.ascore2501_pro <= 17.028
# #                                                      else '3.(14.059, 15.24]' if x.ascore2501_pro > 14.059 and x.ascore2501_pro <= 15.24
# #                                                      else '4.(13.182, 14.059]' if x.ascore2501_pro > 13.182 and x.ascore2501_pro <= 14.059
# #                                                      else '5.(12.394, 13.182]' if x.ascore2501_pro > 12.394  and x.ascore2501_pro <= 13.182
# #                                                      else '6.(11.702, 12.394]' if x.ascore2501_pro > 11.702  and x.ascore2501_pro <= 12.394
# #                                                      else '7.(11.13, 11.702]' if x.ascore2501_pro > 11.13 and x.ascore2501_pro <= 11.702
# #                                                      else '8.(10.531, 11.13]' if x.ascore2501_pro > 10.531  and x.ascore2501_pro <= 11.13
# #                                                      else '9.(9.835, 10.531]' if x.ascore2501_pro > 9.835  and x.ascore2501_pro <= 10.531
# #                                                      else '91.(0.0, 9.835]' if x.ascore2501_pro > 0  and x.ascore2501_pro <= 9.835
# #                                                      else 'other'
# #                                                      , axis = 1
# #                                                      )

# df_model_y['f2pd7'] = np.where(df_model_y.第二期逾期天数 >= df_model_y.第三期逾期天数, df_model_y.第二期逾期天数, df_model_y.第三期逾期天数)
# df_model_y['f3pd7'] = np.where(df_model_y.f2pd7 >= df_model_y.第四期逾期天数, df_model_y.f2pd7, df_model_y.第四期逾期天数)
# df_model_y['y_fraud_f2pd7'] = df_model_y.apply(lambda x: 1 if x.f2pd7 >= 7 
#                                                else 0 if x.f2pd7 >= 0 and x.f2pd7 <= 6
#                                                else 999, axis = 1)
# df_model_y['y_fraud_f3pd7'] = df_model_y.apply(lambda x: 1 if x.f3pd7 >= 7 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 6
#                                                else 999, axis = 1)
# df_model_y['y_fraud_fpd7'] = df_model_y.apply(lambda x: 1 if x.FPD >= 7 
#                                                else 0 if x.FPD >= 0 and x.FPD <= 6
#                                                else 999, axis = 1)

# df_model_y.shape
# df_model_y[:1]

# # （2-2）统计
# conditions_1 = (df_model_y.首期应还日期 >= '2024-09-03') & (df_model_y.首期应还日期 <= '2024-10-02')
# conditions_2 = (df_model_y.首期应还日期 >= '2024-10-03') & (df_model_y.首期应还日期 <= '2024-11-02')
# # conditions_3 = (df_model_y.ascore2501_base <= 17.691)
# conditions_3 = (df_model_y.ascore2501_base <= 17.8051)

# condition_8 = (df_model_y.归属渠道 == '芝麻租物')
# # condition_9 = (df_model_y.归属渠道 == '搜索渠道')
# condition_9 = (df_model_y.归属渠道 != '芝麻租物')
# condition_10 = (~df_model_y.归属渠道.isin(['芝麻租物','搜索渠道']))

# # df_model_y.search_time.value_counts()
# # df_model_y.下单日期.min()
# # df_model_y.下单日期.max()

# # （2-2-1）ascore2501_base_Bin
# # 芝麻租物
# pd.pivot_table(df_model_y[conditions_1 & condition_8],index='ascore2501_base_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_1 & condition_8],index='ascore2501_base_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_1 & condition_8],index='ascore2501_base_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# pd.pivot_table(df_model_y[conditions_2 & condition_8],index='ascore2501_base_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y[conditions_2 & condition_8],index='ascore2501_base_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # 搜索渠道
# # pd.pivot_table(df_model_y[conditions_1 & condition_9],index='ascore2501_base_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y[conditions_1 & condition_9],index='ascore2501_base_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y[conditions_1 & condition_9],index='ascore2501_base_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # pd.pivot_table(df_model_y[conditions_2 & condition_9],index='ascore2501_base_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y[conditions_2 & condition_9],index='ascore2501_base_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # （2-2-2）ascore2501_pro_Bin
# # pd.pivot_table(df_model_y[conditions_1],index='ascore2501_pro_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y[conditions_1],index='ascore2501_pro_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y[conditions_1],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # pd.pivot_table(df_model_y[conditions_2],index='ascore2501_pro_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y[conditions_2],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # （2-2-3）去掉 ascore2501_base 后的 ascore2501_pro
# # pd.pivot_table(df_model_y[conditions_1 & conditions_3 & condition_8],index='ascore2501_pro_Bin',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y[conditions_1 & conditions_3 & condition_8],index='ascore2501_pro_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y[conditions_1 & conditions_3 & condition_8],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # pd.pivot_table(df_model_y[conditions_2 & conditions_3 & condition_8],index='ascore2501_pro_Bin',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y[conditions_2 & conditions_3 & condition_8],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # df_model_y.to_excel('df_model_y.xlsx', index = False)

In [249]:
# # （5）20250205 模型验证-细分渠道 无y
# # （3-1）导入模型评分
# path = 'C:/Users/Administrator/Desktop/mine/11_模型/2025.1/2.模型结果/'
# df_model_y_no = pd.read_excel(path + '无y样本.xlsx')
# df_model_y_no = df_model_y_no.merge(df_analy_data[['order_number','首期应还日期','FPD','第二期逾期天数','第三期逾期天数','第四期逾期天数']], on = 'order_number', how = 'left')
# df_model_y_no.shape

# # （3-2）下单日期
# sql = '''-- 租后应收监控  
# SELECT  om.order_number, om.create_time
# from db_digua_business.t_order om
# ;
# '''
# df_info = query(sql)
# df_info["下单日期"] = df_info['create_time'].dt.date
# df_info["下单日期"] = pd.to_datetime(df_info["下单日期"],errors="coerce")
# df_info.shape

# # （3-3）宽表合并
# df_model_y_no = df_model_y_no.merge(df_info, on = 'order_number', how = 'left')
# path = 'C:/Users/Administrator/Desktop/mine/11_模型/'
# tmp = pd.read_excel(path + '免押用户_20250116.xlsx')
# tmp.shape
# df_model_y_no = df_model_y_no.merge(tmp[['order_number', '归属渠道']], on = 'order_number', how = 'inner')
# del tmp

# # 租物渠道
# # df_model_y_no['ascore2501_base_Bin'] = df_model_y_no.apply(lambda x: '1.(17.8051, 100.0]' if x.ascore2501_base > 17.8051 and x.ascore2501_base <= 100
# #                                                      else '2.(15.259, 17.8051]' if x.ascore2501_base > 15.259 and x.ascore2501_base <= 17.8051
# #                                                      else '3.(13.73, 15.259]' if x.ascore2501_base > 13.73 and x.ascore2501_base <= 15.259
# #                                                      else '4.(12.622, 13.73]' if x.ascore2501_base > 12.622 and x.ascore2501_base <= 13.73
# #                                                      else '5.(11.635, 12.622]' if x.ascore2501_base > 11.635  and x.ascore2501_base <= 12.622
# #                                                      else '6.(10.803, 11.635]' if x.ascore2501_base > 10.803  and x.ascore2501_base <= 11.635
# #                                                      else '7.(10.047, 10.803]' if x.ascore2501_base > 10.047  and x.ascore2501_base <= 10.803
# #                                                      else '8.(9.17, 10.047]' if x.ascore2501_base > 9.17  and x.ascore2501_base <= 10.047
# #                                                      else '9.(8.137, 9.17]' if x.ascore2501_base > 8.137  and x.ascore2501_base <= 9.17
# #                                                      else '91.(0.0, 8.137]' if x.ascore2501_base > 0  and x.ascore2501_base <= 8.137
# #                                                      else 'other'
# #                                                      , axis = 1
# #                                                      )
# # df_model_y_no['ascore2501_base_Bin'] = df_model_y_no.apply(lambda x: '1.(19.955, 100]' if x.ascore2501_base > 19.955 and x.ascore2501_base <= 100
# #                                                      else '2.(15.259, 19.955]' if x.ascore2501_base > 15.259 and x.ascore2501_base <= 19.955
# #                                                      else '3.(14.104, 15.259]' if x.ascore2501_base > 14.104 and x.ascore2501_base <= 15.259
# #                                                      else '4.(12.622, 14.104]' if x.ascore2501_base > 12.622 and x.ascore2501_base <= 14.104
# #                                                      else '5.(11.635, 12.622]' if x.ascore2501_base > 11.635  and x.ascore2501_base <= 12.622
# #                                                      else '6.(10.803, 11.635]' if x.ascore2501_base > 10.803  and x.ascore2501_base <= 11.635
# #                                                      else '7.(10.047, 10.803]' if x.ascore2501_base > 10.047  and x.ascore2501_base <= 10.803
# #                                                      else '8.(9.17, 10.047]' if x.ascore2501_base > 9.17  and x.ascore2501_base <= 10.047
# #                                                      else '9.(8.137, 9.17]' if x.ascore2501_base > 8.137  and x.ascore2501_base <= 9.17
# #                                                      else '91.(0.0, 8.137]' if x.ascore2501_base > 0  and x.ascore2501_base <= 8.137
# #                                                      else 'other'
# #                                                      , axis = 1
# #                                                      )
# df_model_y_no['ascore2501_base_Bin'] = df_model_y_no.apply(lambda x: '1.(18.1312, 100.0]' if x.ascore2501_base > 18.1312 and x.ascore2501_base <= 100
#                                                      else '2.(15.259, 18.1312]' if x.ascore2501_base > 15.259 and x.ascore2501_base <= 18.1312
#                                                      else '3.(14.104, 15.259]' if x.ascore2501_base > 14.104 and x.ascore2501_base <= 15.259
#                                                      else '4.(12.622, 14.104]' if x.ascore2501_base > 12.622 and x.ascore2501_base <= 14.104
#                                                      else '5.(11.635, 12.622]' if x.ascore2501_base > 11.635  and x.ascore2501_base <= 12.622
#                                                      else '6.(10.803, 11.635]' if x.ascore2501_base > 10.803  and x.ascore2501_base <= 11.635
#                                                      else '7.(10.047, 10.803]' if x.ascore2501_base > 10.047  and x.ascore2501_base <= 10.803
#                                                      else '8.(9.17, 10.047]' if x.ascore2501_base > 9.17  and x.ascore2501_base <= 10.047
#                                                      else '9.(8.137, 9.17]' if x.ascore2501_base > 8.137  and x.ascore2501_base <= 9.17
#                                                      else '91.(0.0, 8.137]' if x.ascore2501_base > 0  and x.ascore2501_base <= 8.137
#                                                      else 'other'
#                                                      , axis = 1
#                                                      )
# # 非租物渠道 17.691 17.8016 18.098
# # df_model_y_no['ascore2501_base_Bin'] = df_model_y_no.apply(lambda x: '1.(17.691, 100.0]' if x.ascore2501_base > 17.691 and x.ascore2501_base <= 100
# #                                                      else '2.(15.259, 17.691]' if x.ascore2501_base > 15.259 and x.ascore2501_base <= 17.691
# #                                                      else '3.(13.79, 15.259]' if x.ascore2501_base > 13.79 and x.ascore2501_base <= 15.259
# #                                                      else '4.(12.622, 13.79]' if x.ascore2501_base > 12.622 and x.ascore2501_base <= 13.79
# #                                                      else '5.(11.635, 12.622]' if x.ascore2501_base > 11.635  and x.ascore2501_base <= 12.622
# #                                                      else '6.(10.803, 11.635]' if x.ascore2501_base > 10.803  and x.ascore2501_base <= 11.635
# #                                                      else '7.(10.047, 10.803]' if x.ascore2501_base > 10.047  and x.ascore2501_base <= 10.803
# #                                                      else '8.(9.17, 10.047]' if x.ascore2501_base > 9.17  and x.ascore2501_base <= 10.047
# #                                                      else '9.(8.137, 9.17]' if x.ascore2501_base > 8.137  and x.ascore2501_base <= 9.17
# #                                                      else '91.(0.0, 8.137]' if x.ascore2501_base > 0  and x.ascore2501_base <= 8.137
# #                                                      else 'other'
# #                                                      , axis = 1
# #                                                      )

# # df_model_y_no['ascore2501_base_Bin'] = df_model_y_no.apply(lambda x: '1.(18.098, 100.0]' if x.ascore2501_base > 18.098 and x.ascore2501_base <= 100
# #                                                      else '2.(15.259, 18.098]' if x.ascore2501_base > 15.259 and x.ascore2501_base <= 18.098
# #                                                      else '3.(13.79, 15.259]' if x.ascore2501_base > 13.79 and x.ascore2501_base <= 15.259
# #                                                      else '4.(12.622, 13.79]' if x.ascore2501_base > 12.622 and x.ascore2501_base <= 13.79
# #                                                      else '5.(11.635, 12.622]' if x.ascore2501_base > 11.635  and x.ascore2501_base <= 12.622
# #                                                      else '6.(10.803, 11.635]' if x.ascore2501_base > 10.803  and x.ascore2501_base <= 11.635
# #                                                      else '7.(10.047, 10.803]' if x.ascore2501_base > 10.047  and x.ascore2501_base <= 10.803
# #                                                      else '8.(9.17, 10.047]' if x.ascore2501_base > 9.17  and x.ascore2501_base <= 10.047
# #                                                      else '9.(8.137, 9.17]' if x.ascore2501_base > 8.137  and x.ascore2501_base <= 9.17
# #                                                      else '91.(0.0, 8.137]' if x.ascore2501_base > 0  and x.ascore2501_base <= 8.137
# #                                                      else 'other'
# #                                                      , axis = 1
# #                                                      )

# # df_model_y_no['ascore2501_pro_Bin'] = df_model_y_no.apply(lambda x: '1.(17.028, 100.0]' if x.ascore2501_pro > 17.028 and x.ascore2501_pro <= 100
# #                                                      else '2.(15.24, 17.028]' if x.ascore2501_pro > 15.24 and x.ascore2501_pro <= 17.028
# #                                                      else '3.(14.059, 15.24]' if x.ascore2501_pro > 14.059 and x.ascore2501_pro <= 15.24
# #                                                      else '4.(13.182, 14.059]' if x.ascore2501_pro > 13.182 and x.ascore2501_pro <= 14.059
# #                                                      else '5.(12.394, 13.182]' if x.ascore2501_pro > 12.394  and x.ascore2501_pro <= 13.182
# #                                                      else '6.(11.702, 12.394]' if x.ascore2501_pro > 11.702  and x.ascore2501_pro <= 12.394
# #                                                      else '7.(11.13, 11.702]' if x.ascore2501_pro > 11.13 and x.ascore2501_pro <= 11.702
# #                                                      else '8.(10.531, 11.13]' if x.ascore2501_pro > 10.531  and x.ascore2501_pro <= 11.13
# #                                                      else '9.(9.835, 10.531]' if x.ascore2501_pro > 9.835  and x.ascore2501_pro <= 10.531
# #                                                      else '91.(0.0, 9.835]' if x.ascore2501_pro > 0  and x.ascore2501_pro <= 9.835
# #                                                      else 'other'
# #                                                      , axis = 1
# #                                                      )

# df_model_y_no['f2pd7'] = np.where(df_model_y_no.第二期逾期天数 >= df_model_y_no.第三期逾期天数, df_model_y_no.第二期逾期天数, df_model_y_no.第三期逾期天数)
# df_model_y_no['f3pd7'] = np.where(df_model_y_no.f2pd7 >= df_model_y_no.第四期逾期天数, df_model_y_no.f2pd7, df_model_y_no.第四期逾期天数)
# df_model_y_no['y_fraud_f2pd7'] = df_model_y_no.apply(lambda x: 1 if x.f2pd7 >= 7 
#                                                else 0 if x.f2pd7 >= 0 and x.f2pd7 <= 6
#                                                else 999, axis = 1)
# df_model_y_no['y_fraud_f3pd7'] = df_model_y_no.apply(lambda x: 1 if x.f3pd7 >= 7 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 6
#                                                else 999, axis = 1)
# df_model_y_no['y_fraud_fpd7'] = df_model_y_no.apply(lambda x: 1 if x.FPD >= 7 
#                                                else 0 if x.FPD >= 0 and x.FPD <= 6
#                                                else 999, axis = 1)
# df_model_y_no.shape
# df_model_y_no[:1]

# # （3-4）统计
# # （3-4-1）有标签 -- 首期应还日期为 2024.11.3-2024.12.14
# conditions_4 = (df_model_y_no.首期应还日期 >= '2024-11-03') & (df_model_y_no.首期应还日期 <= '2024-12-14')
# # conditions_5 = (df_model_y_no.ascore2501_base <= 17.691)
# conditions_5 = (df_model_y_no.ascore2501_base <= 18.1312)
# conditions_6 = (df_model_y_no.下单日期 >= '2024-12-15')

# condition_8 = (df_model_y_no.归属渠道 == '芝麻租物')
# # condition_9 = (df_model_y_no.归属渠道 == '搜索渠道')
# condition_9 = (df_model_y_no.归属渠道 != '芝麻租物')
# condition_10 = (~df_model_y_no.归属渠道.isin(['芝麻租物','搜索渠道']))

# pd.pivot_table(df_model_y_no[conditions_4 & condition_8],index='ascore2501_base_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y_no[conditions_4 & condition_9],index='ascore2501_base_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# # # pd.pivot_table(df_model_y_no[conditions_4],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y_no[conditions_4 & conditions_5 & condition_8],index='ascore2501_pro_Bin',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

# # （3-4-2）无标签 -- 进件日期为 2024.12.15-2025.1.14
# pd.pivot_table(df_model_y_no[conditions_6 & condition_8],index='下单日期',columns='ascore2501_base_Bin',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y_no[conditions_6 & condition_9],index='下单日期',columns='ascore2501_base_Bin',values='order_number',aggfunc='count')
# # # pd.pivot_table(df_model_y_no[conditions_6],index='下单日期',columns='ascore2501_pro_Bin',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y_no[conditions_6 & conditions_5 & condition_8],index='下单日期',columns='ascore2501_pro_Bin',values='order_number',aggfunc='count')

In [250]:
# # （6）更新时间范围
# conditions_4 = (df_model_y_no.首期应还日期 >= '2024-11-03') & (df_model_y_no.首期应还日期 <= '2024-11-30')
# conditions_5 = (df_model_y_no.首期应还日期 >= '2024-12-01') & (df_model_y_no.首期应还日期 <= '2024-12-31')
# # conditions_6 = (df_model_y_no.首期应还日期 >= '2024-11-03') & (df_model_y_no.首期应还日期 <= '2024-12-14')

# df_model_y_no['juece'] = df_model_y_no.apply(lambda x: '3.免审' if x.归属渠道 == '芝麻租物' and x.ascore2501_base >= 0 and x.ascore2501_base <= 14.104
#                                    else '2.信审' if x.归属渠道 == '芝麻租物' and x.ascore2501_base > 14.104 and x.ascore2501_base <= 18.1312
#                                    else '1.强拒' if x.归属渠道 == '芝麻租物' and x.ascore2501_base > 18.1312
#                                    else '3.免审' if x.归属渠道 != '芝麻租物' and x.ascore2501_base >= 0 and x.ascore2501_base <= 13.79
#                                    else '2.信审' if x.归属渠道 != '芝麻租物' and x.ascore2501_base >= 13.79 and x.ascore2501_base <= 18.098
#                                    else '1.强拒' if x.归属渠道 != '芝麻租物' and x.ascore2501_base > 18.098

#                                    else '-999'
#                                    ,axis = 1)

# # df_model_y_no
# pd.pivot_table(df_model_y_no[conditions_4],index='juece',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y_no[conditions_4],index='juece',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model_y_no[conditions_6],index='juece',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model_y_no[conditions_5],index='juece',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

In [251]:
# # （7）新旧模型对比
# # 2024.15及之后的样本，新模型的是含商家，旧模型的是自营，所有数量上稍有差异，另外新模型的小蚂蚁拒量的分可能不正确，看强拒、信审、免审 情况主要看2025.1.21之后的（即上线后的）
# # 2024.9-2024.11.14 的样本，旧模型按照订单号去重，数据有可能跟新模型稍有差异
# # （6-1）样本跟新模型的一致，下单日期为 2024.9及之后的
# path = 'C:/Users/Administrator/Desktop/mine/11_模型/2025.1/2.模型结果/'
# df_model_y.shape
# df_model_y_no.shape
# var_need = ['order_number','y_fraud_f2pd7','y_fraud_f3pd7','y_fraud_fpd7','首期应还日期','下单日期','归属渠道']
# df_model_1 = pd.concat([df_model_y[var_need], df_model_y_no[var_need]])
# df_model_1.shape
# df_model_1.drop_duplicates(subset = ['order_number']).shape

# path = 'C:/Users/Administrator/Desktop/mine/11_模型/'
# # 2025.1.22-2025.2.6的数据
# # score_data_new = pd.read_excel(path + '新模型上线后评分_20250106.xlsx')
# # 2025.2.5-2025.2.10的数据
# score_data_new = pd.read_excel(path + '新模型上线后评分_20250210.xlsx')
# score_data_new.shape
# score_data_new = score_data_new[(score_data_new.拒绝理由 != '命中非免押用户拒绝') & (score_data_new.下单日期 >= '2025-01-22')][['order_number','下单日期','归属渠道']]
# score_data_new.shape

# df_model_1 = pd.concat([df_model_1,score_data_new])
# df_model_1.shape

# # （6-2）
# sql = ''' 
# SELECT  om.id as order_id, om.order_number, tmu.id_card_num, tmu.mobile 
# , om.create_time
# from db_digua_business.t_order om
# left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
# where date_format(om.create_time, '%Y-%m-%d') >= '2024-08-01'
# ;
# '''

# df_zhys = query(sql)
# df_zhys["下单日期"] = df_zhys['create_time'].dt.date
# df_zhys["下单日期"] = pd.to_datetime(df_zhys["下单日期"],errors="coerce")
# df_zhys = df_zhys.drop(columns=['create_time']) 
# df_zhys.shape

# # （6-3）
# # 自有模型 2311
# sql1='''-- 风控-自有模型-ascore2311 
# SELECT trace_id, score
# from db_credit.risk_private_model
# ;
# '''
# mx_2311_df =query(sql1)
# mx_2311_df['model_score_2311'] = pd.to_numeric(mx_2311_df['score'], errors='coerce')
# mx_2311_df.shape

# # （6-4）自有模型 2401
# sql1='''-- 风控-自有模型-2401
# SELECT trace_id,  score
# from db_credit.risk_private_model_2401
# ;
# '''
# mx_2401_df =query(sql1)
# mx_2401_df['model_score_2401'] = pd.to_numeric(mx_2401_df['score'], errors='coerce')
# mx_2401_df.shape

# # （6-5）自有模型-银联
# sql1='''-- 风控-自有模型-yl
# SELECT trace_id,  score
# from db_credit.risk_private_model_yl
# ;
# '''
# mx_yl_df =query(sql1)
# mx_yl_df['model_score_yl'] = pd.to_numeric(mx_yl_df['score'], errors='coerce')
# mx_yl_df.shape

# # （6-6）
# sql1='''-- risk关联表 
# SELECT trace_id, id_card as id_card_num, mobile, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.is_model') end,'"','') as is_model
# from db_credit.risk
# ;
# '''
# risk_df = query(sql1)
# # risk_df = risk_df.sort_values('time', ascending=False).groupby('id_card_num').head(1)
# risk_df['下单日期'] = risk_df['time'].dt.date
# risk_df["下单日期"] = pd.to_datetime(risk_df["下单日期"],errors="coerce")
# risk_df = risk_df.drop(columns=['time']) 
# risk_df.shape
# risk_df.drop_duplicates(subset = ['trace_id']).shape

# # （6-7）表关联
# df_model = mx_2311_df[['trace_id', 'model_score_2311']].merge(mx_2401_df[['trace_id', 'model_score_2401']], on = 'trace_id', how='outer')
# df_model.shape

# df_model = df_model.merge(mx_yl_df[['trace_id', 'model_score_yl']], on = 'trace_id', how='outer')
# df_model.shape

# # df_model = df_model.merge(risk_df, on='', how='inner')
# df_model = df_model.merge(risk_df, on='trace_id', how='left')
# df_model.shape
# df_model.drop_duplicates(subset = ['trace_id']).shape
# df_model.drop_duplicates(subset = ['id_card_num','mobile','下单日期']).shape
# df_model.drop_duplicates(subset = ['id_card_num','mobile','下单日期','model_score_2311','model_score_2401','model_score_yl']).shape

# df_model = df_model.merge(df_zhys, on = ['id_card_num', 'mobile', '下单日期'], how = 'inner')
# df_model = df_model.drop(columns=['下单日期']) 
# df_model.shape
# df_model.drop_duplicates(subset = ['order_number']).shape
# df_model.drop_duplicates(subset = ['order_number','model_score_2311','model_score_2401','model_score_yl']).shape

# df_model = df_model_1.merge(df_model, on = ['order_number'], how = 'left')
# df_model.shape
# df_model.drop_duplicates(subset = ['order_number']).shape
# df_model.drop_duplicates(subset = ['order_number','model_score_2311','model_score_2401','model_score_yl']).shape
# # df_model.order_number.value_counts()
# # A20241022122614127
# # df_model[df_model.order_number == 'A2024122201230715']
# # 57092 - 54234

# df_model = df_model.drop_duplicates(subset = ['order_number'])
# df_model.shape
# df_model[:1]

# # # 来自df_model_data  12_差异化还款方式
# # df_model_data.shape
# # df_model = df_model.merge(df_model_data[['order_id','model_score_2311','model_score_2401','model_score_yl','is_model']], on = 'order_id', how = 'left')

# #（6-8）
# # # 0：没有使用模型
# # # 1：ascore2311
# # # 2：ascore_yl2311
# # # 3：ascore2401
# df_model['juece'] = df_model.apply(lambda x: '3.免审' if x.is_model == '1' and x.归属渠道 != '搜索渠道' and x.model_score_2311 >= 0 and x.model_score_2311 <= 18.321
#                                    else '2.信审' if x.is_model == '1' and x.归属渠道 != '搜索渠道' and x.model_score_2311 > 18.321 and x.model_score_2311 <= 23.297
#                                    else '1.强拒' if x.is_model == '1' and x.归属渠道 != '搜索渠道' and x.model_score_2311 > 23.297
#                                    else '3.免审' if x.is_model == '1' and x.归属渠道 == '搜索渠道' and x.model_score_2311 >= 0 and x.model_score_2311 <= 16.6
#                                    else '2.信审' if x.is_model == '1' and x.归属渠道 == '搜索渠道' and x.model_score_2311 >= 16.6 and x.model_score_2311 <= 21.435
#                                    else '1.强拒' if x.is_model == '1' and x.归属渠道 == '搜索渠道' and x.model_score_2311 > 21.435

#                                    else '3.免审' if x.is_model == '2' and x.归属渠道 != '搜索渠道' and x.model_score_yl >= 0 and x.model_score_yl <= 17.55
#                                    else '2.信审' if x.is_model == '2' and x.归属渠道 != '搜索渠道' and x.model_score_yl > 17.55 and x.model_score_yl <= 23.382
#                                    else '1.强拒' if x.is_model == '2' and x.归属渠道 != '搜索渠道' and x.model_score_yl > 23.382
#                                    else '3.免审' if x.is_model == '2' and x.归属渠道 == '搜索渠道' and x.model_score_yl >= 0 and x.model_score_yl <= 15.059
#                                    else '2.信审' if x.is_model == '2' and x.归属渠道 == '搜索渠道' and x.model_score_yl >= 15.059 and x.model_score_yl <= 21.661
#                                    else '1.强拒' if x.is_model == '2' and x.归属渠道 == '搜索渠道' and x.model_score_yl > 21.661

#                                    else '3.免审' if x.is_model == '3' and x.归属渠道 != '搜索渠道' and x.model_score_2401 >= 0 and x.model_score_2401 <= 11.04
#                                    else '2.信审' if x.is_model == '3' and x.归属渠道 != '搜索渠道' and x.model_score_2401 > 11.04 and x.model_score_2401 <= 16
#                                    else '1.强拒' if x.is_model == '3' and x.归属渠道 != '搜索渠道' and x.model_score_2401 > 16
#                                    else '3.免审' if x.is_model == '3' and x.归属渠道 == '搜索渠道' and x.model_score_2401 >= 0 and x.model_score_2401 <= 9.35
#                                    else '2.信审' if x.is_model == '3' and x.归属渠道 == '搜索渠道' and x.model_score_2401 >= 9.35 and x.model_score_2401 <= 14.93
#                                    else '1.强拒' if x.is_model == '3' and x.归属渠道 == '搜索渠道' and x.model_score_2401 > 14.93

#                                    else 'other' if x.is_model == '0'
#                                    else '-999'
#                                    ,axis = 1)

# df_model_online = df_model[df_model.下单日期 >= '2025-01-22']
# #（6-9）

# conditions_1 = (df_model.首期应还日期 >= '2024-09-03') & (df_model.首期应还日期 <= '2024-10-02')
# conditions_2 = (df_model.首期应还日期 >= '2024-10-03') & (df_model.首期应还日期 <= '2024-11-02')
# # conditions_3 = (df_model.首期应还日期 >= '2024-11-03') & (df_model.首期应还日期 <= '2024-12-14')
# # conditions_4 = (df_model.下单日期 >= '2024-12-15')

# conditions_4 = (df_model.首期应还日期 >= '2024-11-03') & (df_model.首期应还日期 <= '2024-11-30')
# conditions_5 = (df_model.首期应还日期 >= '2024-12-01') & (df_model.首期应还日期 <= '2024-12-31')

# pd.pivot_table(df_model[conditions_1],index='juece',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model[conditions_1],index='juece',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model[conditions_1],index='juece',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')

# pd.pivot_table(df_model[conditions_2],index='juece',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model[conditions_2],index='juece',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')

# # pd.pivot_table(df_model[conditions_3],index='juece',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_model[conditions_4],index='下单日期',columns='juece',values='order_number',aggfunc='count')
# pd.pivot_table(df_model[conditions_4],index='juece',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# pd.pivot_table(df_model[conditions_4],index='juece',columns='y_fraud_f2pd7',values='order_number',aggfunc='count')

# pd.pivot_table(df_model[conditions_5],index='juece',columns='y_fraud_fpd7',values='order_number',aggfunc='count')

In [252]:
# # （8-1）上线后模型运行情况
# # path = 'C:/Users/Administrator/Desktop/mine/11_模型/'
# # score_data_new = pd.read_excel(path + '新模型上线后评分_20250106.xlsx')
# # score_data_new.shape
# # score_data_new = score_data_new[(score_data_new.拒绝理由 != '命中非免押用户拒绝') & (score_data_new.下单日期 >= '2025-01-22')]
# # score_data_new.shape
# # score_data_new[score_data_new.score_base >= 0].shape
# # # score_data_new[:1]

# path = 'C:/Users/Administrator/Desktop/mine/11_模型/2025.1/2.模型结果/'
# score_data_new = pd.read_excel(path + 'ascore2501_base_线上线下分数对比.xlsx',sheet_name="Sheet1")
# score_data_new = score_data_new.rename(columns = {'score1': 'score_base'})
# score_data_new.shape
# # score_data_new[:1]

# df_model_online_old = df_model[df_model.下单日期 >= '2025-01-22']
# # df_model_online_old = df_model_online_old[['order_number', 'juece']]
# df_model_online_old = df_model_online_old[['order_number', 'trace_id', '下单日期', '归属渠道', 'juece']]
# df_model_online_old = df_model_online_old.rename(columns = {'juece': 'juece_old'})
# df_model_online_old.shape
# # score_data_new = score_data_new.merge(df_model_online_old, on = 'order_number', how = 'inner')
# score_data_new = score_data_new.merge(df_model_online_old, on = 'trace_id', how = 'left')
# score_data_new.shape
# score_data_new.drop_duplicates(subset = ['order_number']).shape

# score_data_new['juece'] = score_data_new.apply(lambda x: '3.免审' if x.归属渠道 == '芝麻租物' and x.score_base >= 0 and x.score_base <= 14.104
#                                    else '2.信审' if x.归属渠道 == '芝麻租物' and x.score_base > 14.104 and x.score_base <= 18.1312
#                                    else '1.强拒' if x.归属渠道 == '芝麻租物' and x.score_base > 18.1312
#                                    else '3.免审' if x.归属渠道 != '芝麻租物' and x.score_base >= 0 and x.score_base <= 13.79
#                                    else '2.信审' if x.归属渠道 != '芝麻租物' and x.score_base >= 13.79 and x.score_base <= 18.098
#                                    else '1.强拒' if x.归属渠道 != '芝麻租物' and x.score_base > 18.098

#                                    else '-999'
#                                    ,axis = 1)

# score_data_new[:1]
# pd.pivot_table(score_data_new, index='下单日期',columns='juece',values='order_number',aggfunc='count')
# pd.pivot_table(score_data_new, index='下单日期',columns='juece_old',values='order_number',aggfunc='count')

In [253]:
# （8-2）
# # pd.pivot_table(score_data_new[score_data_new.下单日期 == '2025-02-05'], index='juece_old',columns='juece',values='order_number',aggfunc='count')
# pd.pivot_table(score_data_new, index='juece_old',columns='juece',values='order_number',aggfunc='count')
# score_data_new[((score_data_new.juece_old == '3.免审') & (score_data_new.juece.isin(['2.信审','3.免审']))) | ((score_data_new.juece_old.isin(['2.信审','3.免审'])) & (score_data_new.juece == '3.免审'))].shape
# # score_data_new[((score_data_new.juece_old == '3.免审') & (score_data_new.juece.isin(['2.信审','3.免审']))) | ((score_data_new.juece_old.isin(['2.信审','3.免审'])) & (score_data_new.juece == '3.免审'))].to_excel('tmp.xlsx', index = False)

# score_data_new[(score_data_new.juece_old == '3.免审')].to_excel('tmp1.xlsx', index = False)

In [254]:
# （9）上线后风控决策结构分析
# 取免审策略涉及的指标
# （1）jx_fico_score
sql1='''-- 风控-极信-分数sc32007   fico  
SELECT * 
from db_credit.risk_jixin_score_sc32007
;
'''

jx_fico_df =query(sql1)
jx_fico_df.shape

jx_fico_df['jx_fico_score'] = jx_fico_df['rs_decode'].apply(lambda x: json.loads(x).get('data', {}).get('resultData', {}).get('score', None))  
jx_fico_df['jx_fico_score'] = pd.to_numeric(jx_fico_df['jx_fico_score'].replace('"null"', np.nan), errors='coerce')
jx_fico_df = jx_fico_df[['trace_id', 'jx_fico_score']]
jx_fico_df.shape
# jx_fico_df[:1]

# df_out_data = df_out_data.merge(jx_fico_df, on = 'trace_id', how = 'left')
# df_out_data.shape
# df_out_data[:1]
# （2）bj_qy_score
sql1='''-- 风控-冰鉴-青云分
SELECT * 
from db_credit.risk_icekredit_qy
;
'''

bj_qy_df =query(sql1)
bj_qy_df.shape

bj_qy_df['bj_qy_score'] = bj_qy_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('qing_yun22', {}).get('bj_score', None))  
bj_qy_df['bj_qy_score'] = bj_qy_df['bj_qy_score'].astype(int)
bj_qy_df = bj_qy_df[['trace_id', 'bj_qy_score']]
bj_qy_df.shape
# bj_qy_df[:1]

# df_out_data = df_out_data.merge(bj_qy_df, on = 'trace_id', how = 'left')
# df_out_data.shape
# df_out_data[:1]

# （3） jx_yl_UPPE082 
# 极信-银联
sql1='''-- 风控-极信-银联
SELECT * 
from db_credit.risk_jixin
;
'''

jx_yl_df =query(sql1)
jx_yl_df.shape

jx_yl_df['jx_yl_UPPE082'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE082'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE082'] = pd.to_numeric(jx_yl_df['jx_yl_UPPE082'].replace('"null"', np.nan), errors='coerce')

jx_yl_df = jx_yl_df[['trace_id','jx_yl_UPPE082']]
jx_yl_df.shape
# jx_yl_df[:1]
# df_out_data = df_out_data.merge(jx_yl_df, on = 'trace_id', how = 'left')
# df_out_data.shape
# df_out_data[:1]

# （4） xy_qj_C22180008
# 微聚未来-全景雷达
sql1='''-- 风控-微聚未来-全景雷达
SELECT * 
from db_credit.risk_weisuper_qjld
;
'''

xy_risk_weisuper_df =query(sql1)
xy_risk_weisuper_df.shape
# xy_risk_weisuper_df[:1]

def expand_json_columns(df, json_col,prefix='xy_qj_',sep='.'):
    # 只对非空的 JSON 列进行解析
    df['result_parsed'] = df[json_col].apply(lambda x:json.loads(x) if pd.notna(x) else None )

    # 对解析出来的 JSON 字段进行展开，并添加前缀
    df_json_expanded = pd.json_normalize(df['result_parsed'].dropna(),sep=sep)

    # 保留最内层字段名（去除层级前缀）
    df_json_expanded.columns = [col.split(sep)[-1] for col in df_json_expanded.columns]

    # 为所有字段名添加前缀
    df_json_expanded = df_json_expanded.add_prefix(prefix)

    # 将展开的字段拼接到原始 DataFrame 中
    df_final = pd.concat([df,df_json_expanded], axis=1)

     # 删除中间生成的列
    df_final = df_final.drop(columns='result_parsed')
    return  df_final

xy_risk_weisuper_df = expand_json_columns(xy_risk_weisuper_df,'rs','xy_qj_')

# # xy_qj_B22170034  去除百分比后  转为float  
xy_risk_weisuper_df['xy_qj_B22170034'] = pd.to_numeric(xy_risk_weisuper_df['xy_qj_B22170034'].str.replace('%', ''), errors='coerce') / 100
# # xy_risk_weisuper_df['xy_qj_B22170034'].value_counts()

xy_risk_weisuper_df['xy_qj_C22180008'] = pd.to_numeric(xy_risk_weisuper_df['xy_qj_C22180008'], errors='coerce')

xy_risk_weisuper_df = xy_risk_weisuper_df[['trace_id','xy_qj_C22180008']]
xy_risk_weisuper_df.shape
# xy_risk_weisuper_df[:1]

# （5）自有模型
sql1='''-- 风控-自有模型-ascore2311 
SELECT * 
from db_credit.risk_private_model
;
'''
mx_2311_df =query(sql1)
mx_2311_df.shape
mx_2311_df['model_score_2311'] = pd.to_numeric(mx_2311_df['score'], errors='coerce')
mx_2311_df = mx_2311_df[['trace_id', 'model_score_2311']]

sql1='''-- 风控-自有模型-2401
SELECT * 
from db_credit.risk_private_model_2401
;
'''
mx_2401_df =query(sql1)
mx_2401_df.shape
mx_2401_df['model_score_2401'] = pd.to_numeric(mx_2401_df['score'], errors='coerce')
mx_2401_df = mx_2401_df[['trace_id', 'model_score_2401']]

sql1='''-- 风控-自有模型-yl
SELECT * 
from db_credit.risk_private_model_yl
;
'''
mx_yl_df =query(sql1)
mx_yl_df.shape
mx_yl_df['model_score_yl'] = pd.to_numeric(mx_yl_df['score'], errors='coerce')
mx_yl_df = mx_yl_df[['trace_id', 'model_score_yl']]

# （6）中间表
sql1='''-- risk关联表 
SELECT trace_id,  replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.is_model') end,'"','') as is_model
from db_credit.risk
;
'''
risk_df = query(sql1)
risk_df.shape
risk_df.drop_duplicates(subset = ['trace_id']).shape

sql1='''-- trace关联表 
SELECT * 
from db_credit.order_association
;
'''
trace_df =query(sql1)
trace_df.shape

# （7）表关联 
df_var_1 = mx_2311_df.merge(mx_2401_df, on = 'trace_id', how='outer')
df_var_1 = df_var_1.merge(mx_yl_df, on = 'trace_id', how='outer')
df_var_1 = risk_df.merge(df_var_1, on='trace_id', how='left')
df_var_1 = df_var_1.rename(columns = {'trace_id': 'trace_id_risk'})

df_var_2 = jx_fico_df.merge(bj_qy_df, on = 'trace_id', how='outer')
df_var_2 = df_var_2.merge(jx_yl_df, on = 'trace_id', how='outer')
df_var_2 = df_var_2.merge(xy_risk_weisuper_df, on = 'trace_id', how='outer')
df_var_2 = trace_df.merge(df_var_2, on = 'trace_id', how='left')

df_var = df_var_1.merge(df_var_2, left_on = 'trace_id_risk', right_on = 'order_trace_id', how = 'outer')
# risk_trace_id 可能有空， 但 order_trace_id 较全面
# trace_id 来自 trace_df
# trace_id_risk 来自 risk_df
var = ['order_trace_id','risk_trace_id','trace_id','order_id','order_number','trace_id_risk','is_model','model_score_2311','model_score_2401','model_score_yl','jx_fico_score','bj_qy_score','jx_yl_UPPE082','xy_qj_C22180008']
df_var = df_var[var]	
df_var.shape
df_var[:1]

(66744, 6)

(66744, 2)

(249386, 5)

(249386, 2)

(170572, 6)

...

(259137, 2)

(259137, 2)

(1709848, 5)

(1720901, 14)

,order_trace_id,risk_trace_id,trace_id,order_id,order_number,trace_id_risk,is_model,model_score_2311,model_score_2401,model_score_yl,jx_fico_score,bj_qy_score,jx_yl_UPPE082,xy_qj_C22180008
0,000022fe497871cd841f51efa26810b5,None,000022fe497871cd841f51efa26810b5,1684396.0,A20241105104310307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [255]:
df_result = dfzh1.merge(df_var, on='order_number', how='left')
df_result.columns

Index(['id', 'order_id_x', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'co

In [256]:
# （9-1）关联数据
# 鑫仲导出最新数据
# path = 'C:/Users/Administrator/Desktop/mine/11_模型/2025.2/'
# df_result_1 = pd.read_excel(path + '模型上线前后对比分析_20250217_1.xlsx')
# df_result_1.shape
# # df_result_1[:1]

# df_result = df_result_1.merge(df_var, on = 'order_number', how = 'left')
# df_result.shape
# df_result[:1]

# 新旧免审策略结果
df_result['旧结果'] = df_result.apply(lambda x: '免人审' if x.is_model == '1' and x.归属渠道 != '搜索渠道' and x.model_score_2311 >= 0 and x.model_score_2311 <= 18.321
                                   else '信审' if x.is_model == '1' and x.归属渠道 != '搜索渠道' and x.model_score_2311 > 18.321 and x.model_score_2311 <= 23.297
                                   else '机审强拒' if x.is_model == '1' and x.归属渠道 != '搜索渠道' and x.model_score_2311 > 23.297
                                   else '免人审' if x.is_model == '1' and x.归属渠道 == '搜索渠道' and x.model_score_2311 >= 0 and x.model_score_2311 <= 16.6
                                   else '信审' if x.is_model == '1' and x.归属渠道 == '搜索渠道' and x.model_score_2311 >= 16.6 and x.model_score_2311 <= 21.435
                                   else '机审强拒' if x.is_model == '1' and x.归属渠道 == '搜索渠道' and x.model_score_2311 > 21.435

                                   else '免人审' if x.is_model == '2' and x.归属渠道 != '搜索渠道' and x.model_score_yl >= 0 and x.model_score_yl <= 17.55
                                   else '信审' if x.is_model == '2' and x.归属渠道 != '搜索渠道' and x.model_score_yl > 17.55 and x.model_score_yl <= 23.382
                                   else '机审强拒' if x.is_model == '2' and x.归属渠道 != '搜索渠道' and x.model_score_yl > 23.382
                                   else '免人审' if x.is_model == '2' and x.归属渠道 == '搜索渠道' and x.model_score_yl >= 0 and x.model_score_yl <= 15.059
                                   else '信审' if x.is_model == '2' and x.归属渠道 == '搜索渠道' and x.model_score_yl >= 15.059 and x.model_score_yl <= 21.661
                                   else '机审强拒' if x.is_model == '2' and x.归属渠道 == '搜索渠道' and x.model_score_yl > 21.661

                                   else '免人审' if x.is_model == '3' and x.归属渠道 != '搜索渠道' and x.model_score_2401 >= 0 and x.model_score_2401 <= 11.04
                                   else '信审' if x.is_model == '3' and x.归属渠道 != '搜索渠道' and x.model_score_2401 > 11.04 and x.model_score_2401 <= 16
                                   else '机审强拒' if x.is_model == '3' and x.归属渠道 != '搜索渠道' and x.model_score_2401 > 16
                                   else '免人审' if x.is_model == '3' and x.归属渠道 == '搜索渠道' and x.model_score_2401 >= 0 and x.model_score_2401 <= 9.35
                                   else '信审' if x.is_model == '3' and x.归属渠道 == '搜索渠道' and x.model_score_2401 >= 9.35 and x.model_score_2401 <= 14.93
                                   else '机审强拒' if x.is_model == '3' and x.归属渠道 == '搜索渠道' and x.model_score_2401 > 14.93

                                   else 'other', axis = 1)

# df_result['新结果'] = df_result.apply(lambda x: '免人审' if x.是否调用 == '已调用' and x.结果 != '机审强拒' and x.归属渠道 == '芝麻租物' and x.score_base >= 0 and x.score_base <= 14.104
#                                    else '信审' if x.是否调用 == '已调用' and x.结果 != '机审强拒' and x.归属渠道 == '芝麻租物' and x.score_base > 14.104 and x.score_base <= 18.1312
#                                    else '机审强拒' if x.是否调用 == '已调用' and x.结果 != '机审强拒' and x.归属渠道 == '芝麻租物' and x.score_base > 18.1312
#                                    else '免人审' if x.是否调用 == '已调用' and x.结果 != '机审强拒' and x.归属渠道 != '芝麻租物' and x.score_base >= 0 and x.score_base <= 13.79
#                                    else '信审' if x.是否调用 == '已调用' and x.结果 != '机审强拒' and x.归属渠道 != '芝麻租物' and x.score_base >= 13.79 and x.score_base <= 18.098
#                                    else '机审强拒' if x.是否调用 == '已调用' and x.结果 != '机审强拒' and x.归属渠道 != '芝麻租物' and x.score_base > 18.098
#                                    else x.旧结果,axis = 1)

df_result['旧结果'] = df_result.apply(lambda x: '免人审' if x.旧结果 != '机审强拒' and x.jx_fico_score > 604.5 and x.jx_yl_UPPE082 > 0.655
                                   else '免人审' if x.旧结果 != '机审强拒' and x.jx_fico_score > 604.5 and x.model_score_2401 <= 8.093
                                   else '免人审' if x.旧结果 != '机审强拒' and x.jx_fico_score > 561.5 and x.model_score_2401 <= 9.849
                                   else '免人审' if x.旧结果 != '机审强拒' and x.model_score_2401  <= 14.326 and x.xy_qj_C22180008 > 10.5
                                   else '免人审' if x.旧结果 != '机审强拒' and x.jx_fico_score > 561.5 and x.bj_qy_score > 433.5

                                   else x.旧结果, axis = 1)
# df_result['新结果'] = df_result.apply(lambda x: '免人审' if x.新结果 != '机审强拒' and x.jx_fico_score > 604.5 and x.jx_yl_UPPE082 > 0.655
#                                    else '免人审' if x.新结果 != '机审强拒' and x.jx_fico_score > 604.5 and x.model_score_2401 <= 8.093
#                                    else '免人审' if x.新结果 != '机审强拒' and x.jx_fico_score > 561.5 and x.model_score_2401 <= 9.849
#                                    else '免人审' if x.新结果 != '机审强拒' and x.model_score_2401  <= 14.326 and x.xy_qj_C22180008 > 10.5
#                                    else '免人审' if x.新结果 != '机审强拒' and x.jx_fico_score > 561.5 and x.bj_qy_score > 433.5

                                #    else x.新结果, axis = 1)

df_result['旧结果'] = np.where((df_result.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）' )&(df_result.旧结果 == '信审'), '机审强拒', df_result.旧结果)
df_result['旧结果'] = np.where((df_result.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）' )&(df_result.旧结果 == '免人审'), '信审', df_result.旧结果)

# df_result['新结果'] = np.where((df_result.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）' )&(df_result.新结果 == '信审'), '机审强拒', df_result.新结果)
# df_result['新结果'] = np.where((df_result.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）' )&(df_result.新结果 == '免人审'), '信审', df_result.新结果)

# df_result.to_excel(path + '模型上线前后对比分析_20250217_mine.xlsx', index = False)

In [279]:
df_result1 = df_result[(df_result.旧结果=='免人审')&(df_result.下单日期=='2025-02-21')]

In [280]:
lis = df_result[(df_result.is_vip=='1')&(df_result.status_result=='0')&(df_result.下单日期=='2025-02-21')]['order_number'].to_list()
df_result1[~df_result1.order_number.isin(lis)][['order_number', 'model_score_2311', 'model_score_yl', 'model_score_2401', 'jx_fico_score', 'jx_yl_UPPE082', 'bj_qy_score', 'xy_qj_C22180008', '归属渠道', 'is_model']]

,order_number,model_score_2311,model_score_yl,model_score_2401,jx_fico_score,jx_yl_UPPE082,bj_qy_score,xy_qj_C22180008,归属渠道,is_model
33000,A20250221134449299,12.141932,11.730752,6.042170,688.0,0.13,471.0,14.0,搜索渠道,1
33001,A20250221134726316,15.904682,12.317045,5.743183,613.0,NaN,494.0,NaN,搜索渠道,2
33005,A20250221141742152,12.365845,10.429466,6.734083,518.0,NaN,551.0,NaN,芝麻租物,2
33024,A20250221154949459,25.009340,23.786947,18.173917,625.0,NaN,457.0,NaN,搜索渠道,4
33055,A2025022119131271,14.218632,21.226937,9.357807,512.0,NaN,458.0,4.0,抖音渠道,1
33057,A20250221195652246,20.888665,19.364295,11.297704,704.0,NaN,459.0,NaN,芝麻租物,3
33058,A20250221205601177,19.208441,16.067490,8.412486,585.0,NaN,526.0,NaN,搜索渠道,2


In [ ]:
df_result['仅新模型结果'] = df_result.apply(lambda x: '免人审' if x.归属渠道 == '芝麻租物' and x.score_base >= 0 and x.score_base <= 14.104
                                   else '信审' if x.归属渠道 == '芝麻租物' and x.score_base > 14.104 and x.score_base <= 18.1312
                                   else '机审强拒' if x.归属渠道 == '芝麻租物' and x.score_base > 18.1312
                                   else '免人审' if x.归属渠道 != '芝麻租物' and x.score_base >= 0 and x.score_base <= 13.79
                                   else '信审' if x.归属渠道 != '芝麻租物' and x.score_base >= 13.79 and x.score_base <= 18.098
                                   else '机审强拒' if x.归属渠道 != '芝麻租物' and x.score_base > 18.098

                                   else '-999'
                                   ,axis = 1)

df_result['仅新模型结果_1'] = df_result.apply(lambda x: '免人审' if x.仅新模型结果 != '机审强拒' and x.jx_fico_score > 604.5 and x.jx_yl_UPPE082 > 0.655
                                   else '免人审' if x.仅新模型结果 != '机审强拒' and x.jx_fico_score > 604.5 and x.model_score_2401 <= 8.093
                                   else '免人审' if x.仅新模型结果 != '机审强拒' and x.jx_fico_score > 561.5 and x.model_score_2401 <= 9.849
                                   else '免人审' if x.仅新模型结果 != '机审强拒' and x.model_score_2401  <= 14.326 and x.xy_qj_C22180008 > 10.5
                                   else '免人审' if x.仅新模型结果 != '机审强拒' and x.jx_fico_score > 561.5 and x.bj_qy_score > 433.5

                                   else x.仅新模型结果, axis = 1)

df_result[['order_number','仅新模型结果','仅新模型结果_1']].to_excel(path + 'tmp.xlsx', index = False)

In [ ]:
# tmp = df_result_1.merge(df_var, on = 'order_number', how = 'inner')
# (23461, 21)

tmp = df_result_1.merge(df_var, on = 'trace_id_risk', how = 'inner')
tmp.shape

(36618548, 21)

In [ ]:
df_var.drop_duplicates(subset = ['order_number']).shape

(1673304, 14)

# 6_白名单分析

In [125]:
# （1）数据导进
# 导进风控通过数据
# 184 件的为包含错误名单的风控通过件
# path = 'C:/Users/Administrator/Desktop/mine/6_白名单/1_白名单转化分析_20241218/'
# df_bmd_data = pd.read_excel(path + '白名单风控通过_鑫仲03.xlsx')
# df_bmd_data.shape
path = 'C:/Users/Administrator/Desktop/mine/6_白名单/1_白名单转化分析_20241218/'
df_bmd_data = pd.read_excel(path + '白名单呼叫申请明细清单_mine.xlsx',sheet_name="3_出库订单")
df_bmd_data.shape

# （2）关联贷后表现
df_bmd_data = df_bmd_data.merge(df_analy_data, on = 'order_number', how = 'inner')
df_bmd_data.shape
# 出库分布
df_bmd_data['search_time'].value_counts()
df_bmd_data[:1]

# df_bmd_data[['order_number']].to_excel('tmp.xlsx', index = False)

(151, 1)

(151, 16)

search_time
2024-12    48
2024-11    30
2025-01    27
2024-09    20
2024-10    17
2025-02     9
Name: count, dtype: int64

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,首期应还日期,overdue_type
0,A20241217111946186,1876628,2024-12-17,2024-12,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0,2024-12-20,


In [126]:
# （2）历史逾期统计
df_bmd_data['tmp1'] = np.where(df_bmd_data.overdue_type.isin(['M0','M1','M2','M3','M6']), 1, 0)
df_bmd_data['tmp2'] = np.where(df_bmd_data.FPD >= 7, 1, 0)
df_bmd_data['tmp3'] = np.where(df_bmd_data.历史最大逾期天数 >= 1, 1, 0)
df_bmd_data['tmp4'] = np.where(df_bmd_data.历史最大逾期天数 > 30, 1, 0)
df_bmd_data.groupby(['search_time']).agg({'order_number':'count','tmp1':'sum','tmp2':'sum','tmp3':'sum','tmp4':'sum'}).reset_index()

,search_time,order_number,tmp1,tmp2,tmp3,tmp4
0,2024-09,20,4,2,8,3
1,2024-10,17,2,0,4,1
2,2024-11,30,1,1,2,1
3,2024-12,48,2,1,5,0
4,2025-01,27,0,0,1,0
5,2025-02,9,0,0,0,0


In [304]:
# # （3）当前逾期统计
# sql = '''-- 租后应收监控  
# SELECT  tprm.overdue_type,order_number,search_time
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# where order_number in ('A20241203183750152','A20241202153400238','A20241130174846195','A202410020507027','A20240926154447402','A20240926114911257','A20240925154208280','A20240914183909150','A20240912112753125','A2024090316083459')
# ;
# '''
# query(sql)

,overdue_type,order_number,search_time
0,M3,A2024090316083459,2024-09
1,M0,A20240912112753125,2024-09
2,,A20240914183909150,2024-09
3,,A20240925154208280,2024-09
4,M2,A20240926114911257,2024-09
5,,A20240926154447402,2024-09
6,M1,A202410020507027,2024-10
7,M0,A20241130174846195,2024-11
8,,A20241202153400238,2024-12
9,M0,A20241203183750152,2024-12


In [71]:
# # （4）总体历史逾期统计
# conditions = (df_analy_data.search_time == '2024-09')
# df_analy_data[conditions].shape
# df_analy_data[(df_analy_data.第二期逾期天数 >= 7) & conditions].shape
# df_analy_data[(df_analy_data.历史最大逾期天数 >= 1) & conditions].shape
# df_analy_data[(df_analy_data.历史最大逾期天数 >= 31) & conditions].shape

(2599, 12)

(100, 12)

(518, 12)

(101, 12)

In [74]:
# # （5）总体当前逾期统计
# sql = '''-- 租后应收监控  
# SELECT  order_number,overdue_type
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# where search_time in ('2024-09','2024-10','2024-11')
# '''
# tmp = query(sql)
# # tmp.search_time.value_counts()
# tmp1 = df_analy_data.merge(tmp, on = 'order_number', how = 'left')

# conditions = (tmp1.search_time == '2024-09')
# tmp1[conditions].overdue_type.value_counts()

overdue_type
      2363
M0     143
M1      68
M2      25
Name: count, dtype: int64

# 7_多头豁免分析

In [92]:
# （1）豁免清单
# 多头豁免不带锁于2024.10.27上线，分析豁免客群的租后表现
# 鑫仲提供多头豁免清单
path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'
# df_5t2_data = pd.read_excel(path + '总体命中多头20241218.xlsx')
# df_5t2_data = pd.read_excel(path + '总体命中多头20250106.xlsx')
df_5t2_data = pd.read_excel(path + '总体命中多头20250210.xlsx')
df_5t2_data["下单日期"] = df_5t2_data['create_time'].dt.date
df_5t2_data["下单日期"] = pd.to_datetime(df_5t2_data["下单日期"],errors="coerce")

df_5t2_data.shape
# df_5t2_data.drop_duplicates(subset = ['order_number']).shape
df_5t2_data[:2]
df_5t2_data = df_5t2_data[['order_number']]

(649, 4)

,create_time,order_number,tips,下单日期
0,2024-10-28 18:20:27,A2024102818202766,"[30天多头>9并且90天多头>30, IP与定位地址校验不一致:IP:上海上海、定位:上海...",2024-10-28
1,2024-10-28 18:41:18,A20241028184117143,"[30天多头>9并且90天多头>30, 7日内GPS(1000)米内下单人数:1, GPS最...",2024-10-28


In [ ]:
# （2）关联历史逾期
df_5t2_data = df_5t2_data.merge(df_analy_data, on = 'order_number', how = 'left')
df_5t2_data[:1]
df_5t2_data.shape
# # df_5t2_data[df_5t2_data.历史最大逾期天数 > 0].shape
# # df_5t2_data[df_5t2_data.历史最大逾期天数 > 30].shape

df_5t2_data['tmp1'] = np.where(df_5t2_data.overdue_type.isin(['M0','M1','M2','M3','M6']), 1, 0)
df_5t2_data['tmp2'] = np.where(df_5t2_data.FPD >= 7, 1, 0)
df_5t2_data['tmp3'] = np.where(df_5t2_data.历史最大逾期天数 >= 1, 1, 0)
df_5t2_data['tmp4'] = np.where(df_5t2_data.历史最大逾期天数 > 30, 1, 0)
df_5t2_data.groupby(['search_time']).agg({'order_id':'count','tmp1':'sum','tmp2':'sum','tmp3':'sum','tmp4':'sum'}).reset_index()

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,首期应还日期,overdue_type
0,A2024102818202766,1656599.0,2024-10-28,2024-10,搜索渠道,全新,未带锁,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-10-31,


,search_time,order_id,tmp1,tmp2,tmp3,tmp4
0,2024-10,10,1,1,1,1
1,2024-11,208,13,2,31,4
2,2024-12,263,11,9,23,3
3,2025-01,129,1,0,2,0
4,2025-02,33,0,0,0,0


In [299]:
# （3）关联当前逾期
# order_number 为上面的cell的历史最大逾期天数>0的订单
sql = '''-- 租后应收监控  
SELECT  tprm.overdue_type,order_number,search_time
from db_digua_business.t_postlease_receivables_monitoring   tprm 
where order_number in ('A2024103113112044','A20241101151421103','A2024110118223990','A20241102145325238','A20241103144954159','A20241113104400333','A20241114152152118','A20241114161700106','A20241128195114121','A20241201145752358','A2024120314074744')
;
'''
query(sql)

# M0：1-30
# M1:31-60
# M2：61-90
# M3:91-180
# M6：>180

,overdue_type,order_number,search_time
0,M1,A2024103113112044,2024-10
1,M0,A20241101151421103,2024-11
2,M0,A2024110118223990,2024-11
3,M0,A20241102145325238,2024-11
4,M0,A20241103144954159,2024-11
5,M0,A20241113104400333,2024-11
6,,A20241114152152118,2024-11
7,,A20241114161700106,2024-11
8,,A20241128195114121,2024-11
9,M0,A20241201145752358,2024-12


# 8_512转256分析

In [72]:
# （1）清单导入
# 2024.6 512可转256出库

# 鑫仲提供512转256客户清单
path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'

# df_5t2_data = pd.read_excel(path + '512转256_20241229.xlsx')
# df_5t2_data = pd.read_excel(path + '512转256_20250106.xlsx')
df_5t2_data = pd.read_excel(path + '512转256_20250210.xlsx')
df_5t2_data.shape
df_5t2_data = df_5t2_data.rename(columns = {'order_number_256': 'order_number'})
df_5t2_data.drop_duplicates(subset = 'order_number').shape
df_5t2_data[:2]

(1802, 1)

(1802, 1)

,order_number
0,A2024060109410855
1,A2024060110230259


In [73]:
# （2）关联历史逾期
df_5t2_data = df_5t2_data.merge(df_analy_data, on = 'order_number', how = 'left')
df_5t2_data.shape

df_5t2_data.groupby(['search_time']).agg({'order_id':'count'}).reset_index()
df_5t2_data[df_5t2_data.历史最大逾期天数 > 0].shape

# 未发货
# df_5t2_data[df_5t2_data.search_time.isnull()].shape

(1802, 16)

,search_time,order_id
0,2024-06,120
1,2024-07,186
2,2024-08,258
3,2024-09,235
4,2024-10,169
5,2024-11,316
6,2024-12,322
7,2025-01,160
8,2025-02,32


(388, 16)

In [76]:
# （3）统计历史逾期
conditions = (df_5t2_data.search_time == '2024-06')
df_5t2_data[conditions].shape
df_5t2_data[(df_5t2_data.第二期逾期天数 >= 7) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 1) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_5t2_data.search_time == '2024-07')
df_5t2_data[conditions].shape
df_5t2_data[(df_5t2_data.第二期逾期天数 >= 7) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 1) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_5t2_data.search_time == '2024-08')
df_5t2_data[conditions].shape
df_5t2_data[(df_5t2_data.第二期逾期天数 >= 7) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 1) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_5t2_data.search_time == '2024-09')
df_5t2_data[conditions].shape
df_5t2_data[(df_5t2_data.第二期逾期天数 >= 7) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 1) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_5t2_data.search_time == '2024-10')
df_5t2_data[conditions].shape
df_5t2_data[(df_5t2_data.第二期逾期天数 >= 7) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 1) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_5t2_data.search_time == '2024-11')
df_5t2_data[conditions].shape
df_5t2_data[(df_5t2_data.第二期逾期天数 >= 7) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 1) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_5t2_data.search_time == '2024-12')
df_5t2_data[conditions].shape
df_5t2_data[(df_5t2_data.第二期逾期天数 >= 7) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 1) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_5t2_data.search_time == '2025-01')
df_5t2_data[conditions].shape
df_5t2_data[(df_5t2_data.第二期逾期天数 >= 7) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 1) & conditions].shape
df_5t2_data[(df_5t2_data.历史最大逾期天数 >= 31) & conditions].shape

(120, 16)

(4, 16)

(43, 16)

(24, 16)

(186, 16)

...

(2, 16)

(160, 16)

(0, 16)

(5, 16)

(0, 16)

In [75]:
# （4）当前逾期统计
# sql = '''-- 租后应收监控  
# SELECT  tprm.overdue_type,tprm.order_number
# from db_digua_business.t_postlease_receivables_monitoring  tprm 
# '''
# tmp1 = query(sql)
# tmp1.shape

# df_5t2_data = df_5t2_data.merge(tmp1, on = 'order_number', how = 'left')
# df_5t2_data.shape

conditions = (df_5t2_data.search_time == '2024-06')
df_5t2_data[conditions].overdue_type.value_counts()

conditions = (df_5t2_data.search_time == '2024-07')
df_5t2_data[conditions].overdue_type.value_counts()

conditions = (df_5t2_data.search_time == '2024-08')
df_5t2_data[conditions].overdue_type.value_counts()

conditions = (df_5t2_data.search_time == '2024-09')
df_5t2_data[conditions].overdue_type.value_counts()

conditions = (df_5t2_data.search_time == '2024-10')
df_5t2_data[conditions].overdue_type.value_counts()

conditions = (df_5t2_data.search_time == '2024-11')
df_5t2_data[conditions].overdue_type.value_counts()

conditions = (df_5t2_data.search_time == '2024-12')
df_5t2_data[conditions].overdue_type.value_counts()

conditions = (df_5t2_data.search_time == '2025-01')
df_5t2_data[conditions].overdue_type.value_counts()

overdue_type
      94
M3    10
M0     6
M2     4
M1     3
M6     3
Name: count, dtype: int64

overdue_type
      142
M3     20
M1      9
M0      8
M2      6
M6      1
Name: count, dtype: int64

overdue_type
      207
M1     17
M3     15
M0     11
M2      8
Name: count, dtype: int64

overdue_type
      194
M0     14
M2     12
M1      9
M3      6
Name: count, dtype: int64

overdue_type
      155
M0      6
M1      5
M2      3
Name: count, dtype: int64

overdue_type
      294
M0     15
M1      6
M2      1
Name: count, dtype: int64

overdue_type
      310
M0     10
M1      2
Name: count, dtype: int64

overdue_type
      158
M0      2
Name: count, dtype: int64

In [77]:
# （5）总体历史逾期统计
conditions = (df_analy_data.search_time == '2024-06')
df_analy_data[conditions].shape
df_analy_data[(df_analy_data.第二期逾期天数 >= 7) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 1) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_analy_data.search_time == '2024-07')
df_analy_data[conditions].shape
df_analy_data[(df_analy_data.第二期逾期天数 >= 7) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 1) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_analy_data.search_time == '2024-08')
df_analy_data[conditions].shape
df_analy_data[(df_analy_data.第二期逾期天数 >= 7) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 1) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_analy_data.search_time == '2024-09')
df_analy_data[conditions].shape
df_analy_data[(df_analy_data.第二期逾期天数 >= 7) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 1) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_analy_data.search_time == '2024-10')
df_analy_data[conditions].shape
df_analy_data[(df_analy_data.第二期逾期天数 >= 7) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 1) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_analy_data.search_time == '2024-11')
df_analy_data[conditions].shape
df_analy_data[(df_analy_data.第二期逾期天数 >= 7) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 1) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_analy_data.search_time == '2024-12')
df_analy_data[conditions].shape
df_analy_data[(df_analy_data.第二期逾期天数 >= 7) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 1) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 31) & conditions].shape

conditions = (df_analy_data.search_time == '2025-01')
df_analy_data[conditions].shape
df_analy_data[(df_analy_data.第二期逾期天数 >= 7) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 1) & conditions].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >= 31) & conditions].shape

(1626, 16)

(75, 16)

(670, 16)

(306, 16)

(2061, 16)

...

(17, 16)

(1869, 16)

(1, 16)

(40, 16)

(0, 16)

In [81]:
# （5）总体当前逾期统计
# sql = '''-- 租后应收监控  
# SELECT  order_number,overdue_type
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# '''
# tmp = query(sql)
# # tmp.search_time.value_counts()
# tmp1 = df_analy_data.merge(tmp, on = 'order_number', how = 'left')
tmp1 = df_analy_data.copy()

conditions = (tmp1.search_time == '2024-06')
tmp1[conditions].shape
tmp1[conditions].overdue_type.value_counts()

conditions = (tmp1.search_time == '2024-07')
tmp1[conditions].shape
tmp1[conditions].overdue_type.value_counts()

conditions = (tmp1.search_time == '2024-08')
tmp1[conditions].shape
tmp1[conditions].overdue_type.value_counts()

conditions = (tmp1.search_time == '2024-09')
tmp1[conditions].shape
tmp1[conditions].overdue_type.value_counts()

conditions = (tmp1.search_time == '2024-10')
tmp1[conditions].shape
tmp1[conditions].overdue_type.value_counts()

conditions = (tmp1.search_time == '2024-11')
tmp1[conditions].shape
tmp1[conditions].overdue_type.value_counts()

conditions = (tmp1.search_time == '2024-12')
tmp1[conditions].shape
tmp1[conditions].overdue_type.value_counts()

conditions = (tmp1.search_time == '2025-01')
tmp1[conditions].shape
tmp1[conditions].overdue_type.value_counts()

(1626, 16)

overdue_type
      1275
M3     124
M0      70
M1      56
M6      52
M2      49
Name: count, dtype: int64

(2061, 16)

overdue_type
      1638
M3     158
M0     111
M2      76
M1      61
M6      17
Name: count, dtype: int64

(2246, 16)

...

overdue_type
      2303
M0     136
M1      53
M2      10
Name: count, dtype: int64

(3194, 16)

overdue_type
      3054
M0     123
M1      17
Name: count, dtype: int64

(1869, 16)

overdue_type
      1847
M0      22
Name: count, dtype: int64

# 9_单人会话

In [84]:
# （1）样本分析
# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

df_analy_data.shape
df_analy_data[(df_analy_data.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）')].shape
train_condition = (df_analy_data.search_time>='2023-02')&(df_analy_data.search_time<='2024-01')
df_analy_data[(df_analy_data.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）') & train_condition].shape
# 2323.2-2024.1 有出库，2024.2-2024.5 没有出库，2024.6-2024.12 有出库
# vintage 显示 mob9 稳定
df_analy_data[(df_analy_data.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）') & train_condition & (df_analy_data.历史最大逾期天数 >= 31)].shape
df_analy_data[(df_analy_data.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）') & (df_analy_data.第二期逾期天数 >= 7)].shape
df_analy_data[(df_analy_data.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）') & (df_analy_data.第二期逾期天数 >= 7)&(df_analy_data.search_time<='2024-01')].shape
# df_analy_data[(df_analy_data.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）') & (df_analy_data.第三期逾期天数 >= 7)].shape
# df_analy_data[(df_analy_data.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）') & (df_analy_data.第四期逾期天数 >= 7)].shape
# df_analy_data[(df_analy_data.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）') & (df_analy_data.第二期逾期天数 >= 7)& (df_analy_data.search_time >= '2024-07')].shape
# df_analy_data[(df_analy_data.归属渠道 == '单人聊天会话中的小程序消息卡片（分享）') & (df_analy_data.第三期逾期天数 >= 7)].tail()
137/320
(25-3)/320

(28989, 12)

(555, 12)

(320, 12)

(137, 12)

(25, 12)

(22, 12)

0.428125

0.06875

# 10_火眸分

<!-- （1）搜索渠道目前未使用火眸分作为规则
（2）非搜索渠道火眸分cutoff为500 -->

In [66]:
# （1）trace关联
# T + 1
sql1='''-- trace关联表 
SELECT order_trace_id, risk_trace_id, trace_id, order_id
from db_credit.order_association
;
'''
trace_df = query(sql1)
trace_df.shape
trace_df[:2]

# order_trace_id 有重复，没有空
# order_id 无重复，没有空
# trace_df[trace_df['order_trace_id'] == '441581199811230010']

# 三者非空的时候，order_trace_id = risk_trace_id，但 trace_id 不一定和risk_trace_id/order_trace_id相等
# tmp1 = trace_df[(trace_df.trace_id.notna()) & (trace_df.order_trace_id.notna()) & (trace_df.risk_trace_id.notna())]
# tmp1.shape
# tmp1[tmp1.trace_id== tmp1.order_trace_id].shape
# tmp1[tmp1.trace_id== tmp1.risk_trace_id].shape
# tmp1[tmp1.order_trace_id== tmp1.risk_trace_id].shape
# tmp1[tmp1.trace_id!= tmp1.risk_trace_id][:1]

(1535678, 4)

,order_trace_id,risk_trace_id,trace_id,order_id
0,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,88929
1,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,88931


In [68]:
# （2）火眸分
# 实时
sql1='''-- 风控-冰鉴-火眸分
SELECT trace_id,  rs
from db_credit.risk_icekredit_cds
;
'''
bj_hm_df =query(sql1)
bj_hm_df['bj_hm_score'] = bj_hm_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('card_device_score22', {}).get('bj_score', None))  
bj_hm_df['bj_hm_score'] = bj_hm_df['bj_hm_score'].astype(int)
df_rule = trace_df.merge(bj_hm_df[['trace_id', 'bj_hm_score']], on='trace_id', how='outer')
df_rule.shape
bj_hm_df.shape
df_rule.tail()

# 有 14039 条记录没在中间表
# trace_df[trace_df.risk_trace_id == 'b1d32be1bcaa13902e4c0e04ec0fcac3']
# bj_hm_df[bj_hm_df.trace_id == 'b1d32be1bcaa13902e4c0e04ec0fcac3']
# 14039 = 1549717 - 1535678

(1549717, 5)

(823266, 3)

,order_trace_id,risk_trace_id,trace_id,order_id,bj_hm_score
1549712,NaN,NaN,be12d0fdefb5764a93bb074eaf4a697d,NaN,481.0
1549713,NaN,NaN,b1d32be1bcaa13902e4c0e04ec0fcac3,NaN,431.0
1549714,NaN,NaN,8635c2354dfed8496f3b2bf5cfdf1979,NaN,543.0
1549715,NaN,NaN,9efa0d9b64b5bad600f301fce877c6b4,NaN,685.0
1549716,NaN,NaN,e28a61561bfe1fe64c8f5ff3d935c828,NaN,432.0


In [75]:
# （3）关联租后表现
df_hmscore = df_analy_data.merge(df_rule, on = 'order_id', how = 'inner')
df_hmscore.shape
df_hmscore.drop_duplicates(subset = ['order_id']).shape
df_hmscore[:1]

(29271, 16)

(29271, 16)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,第二期逾期天数,第三期逾期天数,第四期逾期天数,首期应还日期,order_trace_id,risk_trace_id,trace_id,bj_hm_score
0,A202301090006044,88931,2023-01-09,2023-01,芝麻租物,全新,未带锁,5,0.0,5.0,0.0,2023-01-09,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,627.0


In [ ]:
# （4）打标签
# 标签定义：租后9期内历史最大逾期超过30天
# train 2023.7-2024.3
# test 2024.4-2024.11.21
df_hmscore['bj_hm_score'] = df_hmscore['bj_hm_score'].replace(-2, -999)
df_hmscore['bj_hm_score'] = df_hmscore['bj_hm_score'].fillna(-999)
df_hmscore['y_fraud'] = df_hmscore.apply(lambda x: 1 if x.第二期逾期天数 >= 7 
                                       else 0.5 if x.第二期逾期天数 > 0 and x.第二期逾期天数 < 7
                                       else 0, axis = 1)
df_hmscore['y_credit'] = df_hmscore.apply(lambda x: 1 if x.历史最大逾期天数 >= 31 
                                       else 0.5 if x.历史最大逾期天数 > 0 and x.历史最大逾期天数 < 31
                                       else 0, axis = 1)

conditions_1 = (df_hmscore.下单日期 <= '2024-11-21')
pd.pivot_table(df_hmscore[conditions_1],index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(df_hmscore,index='search_time',columns='y_credit',values='order_number',aggfunc='count')

pd.pivot_table(df_hmscore[(df_hmscore.归属渠道 != '搜索渠道') & conditions_1],index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(df_hmscore[df_hmscore.归属渠道 != '搜索渠道'],index='search_time',columns='y_credit',values='order_number',aggfunc='count')

pd.pivot_table(df_hmscore[(df_hmscore.归属渠道 == '搜索渠道') & conditions_1],index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(df_hmscore[df_hmscore.归属渠道 == '搜索渠道'],index='search_time',columns='y_credit',values='order_number',aggfunc='count')

y_fraud,0.0,0.5,1.0
search_time,,,
2023-01,125,27,13
2023-02,258,36,38
2023-03,276,9,31
2023-04,557,40,37
2023-05,425,40,23
2023-06,502,56,30
2023-07,760,68,54
2023-08,759,52,68
2023-09,790,54,46


y_credit,0.0,0.5,1.0
search_time,,,
2023-01,27.0,69.0,69.0
2023-02,103.0,108.0,121.0
2023-03,112.0,81.0,123.0
2023-04,222.0,182.0,230.0
2023-05,153.0,142.0,193.0
2023-06,162.0,180.0,246.0
2023-07,260.0,275.0,347.0
2023-08,301.0,249.0,329.0
2023-09,335.0,248.0,307.0


y_fraud,0.0,0.5,1.0
search_time,,,
2023-01,59,7,2
2023-02,110,11,11
2023-03,151,4,11
2023-04,390,27,26
2023-05,309,26,14
2023-06,409,38,20
2023-07,642,55,40
2023-08,584,42,51
2023-09,689,43,41


y_credit,0.0,0.5,1.0
search_time,,,
2023-01,12.0,26.0,30.0
2023-02,51.0,44.0,37.0
2023-03,71.0,42.0,53.0
2023-04,159.0,130.0,154.0
2023-05,120.0,107.0,122.0
2023-06,140.0,150.0,177.0
2023-07,224.0,232.0,281.0
2023-08,234.0,202.0,241.0
2023-09,306.0,211.0,256.0


y_fraud,0.0,0.5,1.0
search_time,,,
2023-01,66,20,11
2023-02,148,25,27
2023-03,125,5,20
2023-04,167,13,11
2023-05,116,14,9
2023-06,93,18,10
2023-07,118,13,14
2023-08,175,10,17
2023-09,101,11,5


y_credit,0.0,0.5,1.0
search_time,,,
2023-01,15.0,43.0,39.0
2023-02,52.0,64.0,84.0
2023-03,41.0,39.0,70.0
2023-04,63.0,52.0,76.0
2023-05,33.0,35.0,71.0
2023-06,22.0,30.0,69.0
2023-07,36.0,43.0,66.0
2023-08,67.0,47.0,88.0
2023-09,29.0,37.0,51.0


In [103]:
# （5）样本划分
train_sousu_no = df_hmscore[(df_hmscore.归属渠道 != '搜索渠道') & (df_hmscore.下单日期 >= '2023-07-01') & (df_hmscore.下单日期 <= '2024-03-31')]
test_sousu_no = df_hmscore[(df_hmscore.归属渠道 != '搜索渠道') & (df_hmscore.下单日期 >= '2024-04-01') & (df_hmscore.下单日期 <= '2024-11-21')]
train_sousu_no.shape
test_sousu_no.shape

train_sousu = df_hmscore[(df_hmscore.归属渠道 == '搜索渠道') & (df_hmscore.下单日期 >= '2023-07-01') & (df_hmscore.下单日期 <= '2024-03-31')]
test_sousu = df_hmscore[(df_hmscore.归属渠道 == '搜索渠道') & (df_hmscore.下单日期 >= '2024-04-01') & (df_hmscore.下单日期 <= '2024-11-21')]
train_sousu.shape
test_sousu.shape

(7038, 18)

(10721, 18)

(1280, 18)

(3761, 18)

In [104]:
# （6-1）分箱-非搜索渠道
%run C:/Users/Administrator/Desktop/mine/4_专题分析/1_回捞分析/2.model_func.ipynb
continous_merged_dict = {}
var_bin_list = []
trainData_kf_Bin = train_sousu_no[train_sousu_no.y_credit.isin([0,1])]
trainData_kf_Bin['y'] = trainData_kf_Bin['y_credit']
numeric_desc_var = ['bj_hm_score']

for col in numeric_desc_var:
    print ("{} is in processing".format(col))
    if -1111 not in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]): 
        max_interval = 5 
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval,special_attribute=[],minBinPcnt=0)
        trainData_kf_Bin[col+'_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff,special_attribute=[]))
        monotone = BadRateMonotone(trainData_kf_Bin, col+'_Bin', 'y')
        while(not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute = [],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = []))
            if max_interval == 2:
                break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y')
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [])) 
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    else:
        max_interval = 7 #正常取值最多分5箱
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval = max_interval, special_attribute = [-1111, -999],
                                      minBinPcnt = 0)
        trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1111, -999]))
        monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2'])
        while (not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute=[-1111, -999],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))
            if max_interval == 3:
                break
#             if max_interval == 3 and -1111 in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 3 and -1111 not in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 4 and -1111 in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2']) 
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))  
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    continous_merged_dict[col] = cutOff
    

bj_hm_score is in processing


In [111]:
# （6-2）改变映射方式-非搜索渠道
trainData_kf_Bin_1 = train_sousu_no[train_sousu_no.y_credit.isin([0,1])]
trainData_kf_Bin_1['y'] = trainData_kf_Bin_1['y_credit']

for col in numeric_desc_var:
    if -1111 not in set(trainData_kf_Bin_1[col]) and -999 not in set(trainData_kf_Bin_1[col]):
        trainData_kf_Bin_1[col+'_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col]))
        # del trainData_kf_Bin_1[col]
    else:
        trainData_kf_Bin_1[col + '_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col],[-1111, -999]))
        # del trainData_kf_Bin_1[col]

In [114]:
# （6-3）映射到test-非搜索渠道
# start_time = time.time()
test = test_sousu_no.copy()
short_list_ori = numeric_desc_var

for var in short_list_ori:
    if -1111 not in set(test[var]) and -999 not in set(test[var]):
        test[var + '_Bin'] = test[var].map(lambda x: AssignBin_me(x, continous_merged_dict[var]))
        del test[var]
    else:
        test[var + '_Bin'] = test[var].map(lambda x: AssignBin_me(x, continous_merged_dict[var],[-1111, -999]))
        del test[var]

# print("- Time consumed: %s seconds" % int(time.time() - start_time))

test.shape

(10721, 18)

In [ ]:
# （6-4）结果统计-非搜索渠道
train_sousu_no_bin = trainData_kf_Bin_1.copy()
# train_sousu_no_bin['bj_hm_score_Bin_new'] = train_sousu_no_bin.apply(lambda x: '[0,499]' if x.bj_hm_score >= 0 and x.bj_hm_score  <= 499
#                                                                              else '(499,529]' if x.bj_hm_score > 499 and x.bj_hm_score  <= 529
#                                                                              else '(529,563]' if x.bj_hm_score > 529 and x.bj_hm_score  <= 563
#                                                                              else '(563,600]' if x.bj_hm_score > 563 and x.bj_hm_score  <= 600
#                                                                              else '(600,658]' if x.bj_hm_score > 600 and x.bj_hm_score  <= 658
#                                                                              else '(658,inf)' if x.bj_hm_score > 658
#                                                                              else '[-999,-999]' if x.bj_hm_score == -999
#                                                                              else 'other'
#                                                                              ,axis = 1)

pd.pivot_table(trainData_kf_Bin_1,index='bj_hm_score_Bin',columns='y',values='order_number',aggfunc='count')
# pd.pivot_table(train_sousu_no_bin,index='bj_hm_score_Bin_new',columns='y',values='order_number',aggfunc='count')

# test_sousu_no[(test_sousu_no.bj_hm_score >= 0) & (test_sousu_no.bj_hm_score <= 499)].归属渠道.value_counts()

pd.pivot_table(test, index='bj_hm_score_Bin',columns='search_time',values='order_number',aggfunc='count')

pd.pivot_table(test[test.search_time == '2024-04'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-05'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-06'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-07'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-08'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-09'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-10'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-11'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')


y,0.0,1.0
bj_hm_score_Bin,,
"(-999,529.0]",212,239
"(529.0,563.0]",472,423
"(563.0,658.0]",1477,1195
"(658.0,inf)",287,168
"[-999,-999]",353,194


search_time,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11
bj_hm_score_Bin,,,,,,,,
"(-999,529.0]",196,172,213,277,326,374,157,175
"(529.0,563.0]",272,262,297,312,395,473,226,225
"(563.0,658.0]",526,408,595,621,765,837,395,409
"(658.0,inf)",61,74,81,99,92,117,61,63
"[-999,-999]",103,72,136,214,237,223,85,95


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin,,,
"(-999,529.0]",183,9,4
"(529.0,563.0]",243,14,15
"(563.0,658.0]",475,36,15
"(658.0,inf)",54,6,1
"[-999,-999]",95,3,5


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin,,,
"(-999,529.0]",151.0,13.0,8.0
"(529.0,563.0]",247.0,11.0,4.0
"(563.0,658.0]",381.0,16.0,11.0
"(658.0,inf)",70.0,3.0,1.0
"[-999,-999]",70.0,2.0,NaN


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin,,,
"(-999,529.0]",187,13,13
"(529.0,563.0]",262,21,14
"(563.0,658.0]",525,42,28
"(658.0,inf)",73,5,3
"[-999,-999]",118,13,5


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin,,,
"(-999,529.0]",243,25,9
"(529.0,563.0]",290,14,8
"(563.0,658.0]",557,39,25
"(658.0,inf)",89,5,5
"[-999,-999]",191,12,11


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin,,,
"(-999,529.0]",304,7,15
"(529.0,563.0]",357,17,21
"(563.0,658.0]",700,33,32
"(658.0,inf)",86,1,5
"[-999,-999]",200,13,24


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin,,,
"(-999,529.0]",341,21,12
"(529.0,563.0]",434,23,16
"(563.0,658.0]",778,31,28
"(658.0,inf)",110,3,4
"[-999,-999]",195,11,17


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin,,,
"(-999,529.0]",146.0,5.0,6.0
"(529.0,563.0]",208.0,7.0,11.0
"(563.0,658.0]",374.0,11.0,10.0
"(658.0,inf)",59.0,2.0,NaN
"[-999,-999]",72.0,10.0,3.0


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin,,,
"(-999,529.0]",166,6,3
"(529.0,563.0]",213,9,3
"(563.0,658.0]",382,15,12
"(658.0,inf)",55,7,1
"[-999,-999]",81,8,6


In [125]:
# （7-1）分箱-搜索渠道
%run C:/Users/Administrator/Desktop/mine/4_专题分析/1_回捞分析/2.model_func.ipynb
continous_merged_dict = {}
var_bin_list = []
trainData_kf_Bin = train_sousu[train_sousu.y_credit.isin([0,1])]
trainData_kf_Bin['y'] = trainData_kf_Bin['y_credit']
numeric_desc_var = ['bj_hm_score']

for col in numeric_desc_var:
    print ("{} is in processing".format(col))
    if -1111 not in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]): 
        max_interval = 5 
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval,special_attribute=[],minBinPcnt=0)
        trainData_kf_Bin[col+'_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff,special_attribute=[]))
        monotone = BadRateMonotone(trainData_kf_Bin, col+'_Bin', 'y')
        while(not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute = [],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = []))
            if max_interval == 2:
                break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y')
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [])) 
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    else:
        max_interval = 7 #正常取值最多分5箱
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval = max_interval, special_attribute = [-1111, -999],
                                      minBinPcnt = 0)
        trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1111, -999]))
        monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2'])
        while (not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute=[-1111, -999],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))
            if max_interval == 3:
                break
#             if max_interval == 3 and -1111 in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 3 and -1111 not in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 4 and -1111 in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2']) 
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))  
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    continous_merged_dict[col] = cutOff

bj_hm_score is in processing


In [126]:
# （7-2）改变映射方式-搜索渠道
trainData_kf_Bin_1 = train_sousu[train_sousu.y_credit.isin([0,1])]
trainData_kf_Bin_1['y'] = trainData_kf_Bin_1['y_credit']

for col in numeric_desc_var:
    if -1111 not in set(trainData_kf_Bin_1[col]) and -999 not in set(trainData_kf_Bin_1[col]):
        trainData_kf_Bin_1[col+'_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col]))
        # del trainData_kf_Bin_1[col]
    else:
        trainData_kf_Bin_1[col + '_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col],[-1111, -999]))
        # del trainData_kf_Bin_1[col]

In [130]:
# （7-3）映射到test-搜索渠道
# start_time = time.time()
test = test_sousu.copy()
short_list_ori = numeric_desc_var

for var in short_list_ori:
    if -1111 not in set(test[var]) and -999 not in set(test[var]):
        test[var + '_Bin'] = test[var].map(lambda x: AssignBin_me(x, continous_merged_dict[var]))
        # del test[var]
    else:
        test[var + '_Bin'] = test[var].map(lambda x: AssignBin_me(x, continous_merged_dict[var],[-1111, -999]))
        # del test[var]

# print("- Time consumed: %s seconds" % int(time.time() - start_time))


In [135]:
# （7-4）结果统计-非搜索渠道

# pd.pivot_table(trainData_kf_Bin_1,index='bj_hm_score_Bin',columns='y',values='order_number',aggfunc='count')

# pd.pivot_table(test, index='bj_hm_score_Bin',columns='search_time',values='order_number',aggfunc='count')

# pd.pivot_table(test[test.search_time == '2024-04'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(test[test.search_time == '2024-05'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(test[test.search_time == '2024-06'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(test[test.search_time == '2024-07'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(test[test.search_time == '2024-08'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(test[test.search_time == '2024-09'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(test[test.search_time == '2024-10'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(test[test.search_time == '2024-11'],index='bj_hm_score_Bin',columns='y_fraud',values='order_number',aggfunc='count')

# 改变分组
trainData_kf_Bin_1['bj_hm_score_Bin_new'] = trainData_kf_Bin_1.apply(lambda x: '[0,499]' if x.bj_hm_score >= 0 and x.bj_hm_score  <= 499
                                                                             else '(499,599]' if x.bj_hm_score > 499 and x.bj_hm_score  <= 599
                                                                             else '(599,680]' if x.bj_hm_score > 599 and x.bj_hm_score  <= 680
                                                                             else '(680,inf)' if x.bj_hm_score > 680
                                                                             else '[-999,-999]' if x.bj_hm_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)
test['bj_hm_score_Bin_new'] = test.apply(lambda x: '[0,499]' if x.bj_hm_score >= 0 and x.bj_hm_score  <= 499
                                                                             else '(499,599]' if x.bj_hm_score > 499 and x.bj_hm_score  <= 599
                                                                             else '(599,680]' if x.bj_hm_score > 599 and x.bj_hm_score  <= 680
                                                                             else '(680,inf)' if x.bj_hm_score > 680
                                                                             else '[-999,-999]' if x.bj_hm_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)

pd.pivot_table(trainData_kf_Bin_1,index='bj_hm_score_Bin_new',columns='y',values='order_number',aggfunc='count')
pd.pivot_table(test, index='bj_hm_score_Bin_new',columns='search_time',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-04'],index='bj_hm_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-05'],index='bj_hm_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-06'],index='bj_hm_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-07'],index='bj_hm_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-08'],index='bj_hm_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-09'],index='bj_hm_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-10'],index='bj_hm_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(test[test.search_time == '2024-11'],index='bj_hm_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

y,0.0,1.0
bj_hm_score_Bin_new,,
"(499,599]",232,269
"(599,680]",199,163
"(680,inf)",21,15
"[-999,-999]",9,23


search_time,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11
bj_hm_score_Bin_new,,,,,,,,
"(499,599]",298.0,199.0,230.0,239.0,243.0,316.0,390.0,395.0
"(599,680]",102.0,62.0,48.0,73.0,75.0,100.0,120.0,118.0
"(680,inf)",11.0,7.0,8.0,6.0,4.0,4.0,19.0,10.0
"[-999,-999]",1.0,2.0,6.0,36.0,13.0,28.0,35.0,26.0
"[0,499]",NaN,NaN,NaN,57.0,96.0,127.0,118.0,139.0


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin_new,,,
"(499,599]",267.0,20.0,11.0
"(599,680]",93.0,4.0,5.0
"(680,inf)",10.0,1.0,NaN
"[-999,-999]",1.0,NaN,NaN


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin_new,,,
"(499,599]",184.0,9.0,6.0
"(599,680]",59.0,3.0,NaN
"(680,inf)",5.0,1.0,1.0
"[-999,-999]",2.0,NaN,NaN


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin_new,,,
"(499,599]",203.0,17.0,10.0
"(599,680]",44.0,2.0,2.0
"(680,inf)",7.0,1.0,NaN
"[-999,-999]",5.0,1.0,NaN


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin_new,,,
"(499,599]",212.0,19.0,8.0
"(599,680]",68.0,4.0,1.0
"(680,inf)",6.0,NaN,NaN
"[-999,-999]",30.0,5.0,1.0
"[0,499]",53.0,3.0,1.0


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin_new,,,
"(499,599]",227.0,12.0,4.0
"(599,680]",67.0,4.0,4.0
"(680,inf)",3.0,NaN,1.0
"[-999,-999]",9.0,4.0,NaN
"[0,499]",84.0,8.0,4.0


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin_new,,,
"(499,599]",284.0,21.0,11.0
"(599,680]",95.0,1.0,4.0
"(680,inf)",4.0,NaN,NaN
"[-999,-999]",24.0,NaN,4.0
"[0,499]",119.0,4.0,4.0


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin_new,,,
"(499,599]",363.0,20.0,7.0
"(599,680]",116.0,1.0,3.0
"(680,inf)",18.0,NaN,1.0
"[-999,-999]",30.0,3.0,2.0
"[0,499]",110.0,5.0,3.0


y_fraud,0.0,0.5,1.0
bj_hm_score_Bin_new,,,
"(499,599]",374,16,5
"(599,680]",106,10,2
"(680,inf)",8,1,1
"[-999,-999]",20,2,4
"[0,499]",131,3,5


In [ ]:
# （8）未命中前置拦截且未进件的去重订单数分布
path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/'
order_hmscore = pd.read_excel(path + '火眸分20241231.xlsx')
order_hmscore.shape

order_hmscore_sousu_no = order_hmscore[order_hmscore.归属渠道 != '搜索渠道']
order_hmscore_sousu = order_hmscore[order_hmscore.归属渠道 == '搜索渠道']

order_hmscore_sousu_no['bj_hm_score_Bin_new'] = order_hmscore_sousu_no.apply(lambda x: '(-999,529]' if x.bj_hm_score >= 0 and x.bj_hm_score  <= 529
                                                                             else '(529,563]' if x.bj_hm_score > 529 and x.bj_hm_score  <= 563
                                                                             else '(563,658]' if x.bj_hm_score > 563 and x.bj_hm_score  <= 658
                                                                             else '(658,inf)' if x.bj_hm_score > 658
                                                                             else '[-999,-999]' if x.bj_hm_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)
order_hmscore_sousu['bj_hm_score_Bin_new'] = order_hmscore_sousu.apply(lambda x: '[0,499]' if x.bj_hm_score >= 0 and x.bj_hm_score  <= 499
                                                                             else '(499,599]' if x.bj_hm_score > 499 and x.bj_hm_score  <= 599
                                                                             else '(599,680]' if x.bj_hm_score > 599 and x.bj_hm_score  <= 680
                                                                             else '(680,inf)' if x.bj_hm_score > 680
                                                                             else '[-999,-999]' if x.bj_hm_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)

order_hmscore_sousu_no.shape
order_hmscore_sousu.shape
order_hmscore[:1]

pd.pivot_table(order_hmscore_sousu_no, index='bj_hm_score_Bin_new',columns='下单月份',values='order_number',aggfunc='count')
pd.pivot_table(order_hmscore_sousu, index='bj_hm_score_Bin_new',columns='下单月份',values='order_number',aggfunc='count')

(395378, 4)

(302873, 5)

(92505, 5)

,下单月份,order_number,bj_hm_score,归属渠道
0,2024-01,A202401010025406,628.0,芝麻租物


下单月份,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
bj_hm_score_Bin_new,,,,,,,,,,,,
"(-999,529]",1000,787,1490,2590,3071,4627,4446,5263,6428,5242,4841,5221
"(529,563]",1567,1185,2249,3684,3889,5861,5527,6375,7363,5961,5291,5694
"(563,658]",4477,3361,6361,8547,7398,11690,11151,11784,13554,11000,9872,9957
"(658,inf)",526,500,829,981,717,1195,1199,1247,1438,1161,1001,1032
other,1811,1388,3057,4396,4629,8397,10827,10010,11288,9263,8370,8807


下单月份,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
bj_hm_score_Bin_new,,,,,,,,,,,,
"(499,599]",2000.0,879.0,1846.0,3041.0,2825.0,2427.0,3100.0,3543.0,4207.0,5413.0,6959.0,8750.0
"(599,680]",1063.0,499.0,841.0,1169.0,795.0,734.0,919.0,971.0,1158.0,1481.0,1823.0,2260.0
"(680,inf)",97.0,47.0,79.0,118.0,73.0,68.0,84.0,98.0,109.0,119.0,162.0,200.0
"[0,499]",NaN,NaN,NaN,NaN,NaN,137.0,1762.0,2062.0,2495.0,3090.0,4357.0,4410.0
other,661.0,308.0,549.0,658.0,744.0,608.0,1055.0,1114.0,1428.0,1773.0,2215.0,3122.0


# 11_12月风险监控

In [397]:
# （5）总体当前逾期统计
sql = '''-- 租后应收监控  
SELECT  order_number,overdue_type
from db_digua_business.t_postlease_receivables_monitoring   tprm 
'''
tmp = query(sql)
# tmp.search_time.value_counts()
tmp1 = df_analy_data.merge(tmp, on = 'order_number', how = 'left')


conditions = (tmp1.search_time == '2024-12')
tmp1[conditions].shape
tmp1[conditions & (tmp1.overdue_type == 'M0')].shape
tmp1[conditions & (tmp1.下单日期 <= '2024-12-10')].shape
45/1111
# tmp1[conditions & ((tmp1.overdue_type == 'M0') | (tmp1.历史最大逾期天数 > 0))]
tmp1[conditions & (tmp1.overdue_type == 'M0')]

(3194, 16)

(45, 16)

(1111, 16)

0.0405040504050405

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,首期应还日期,overdue_type
26683,A20241201145752358,1796364,2024-12-01,2024-12,搜索渠道,全新,未带锁,4,4.0,0.0,0.0,4.0,0.0,0.0,2024-12-04,M0
26752,A2024120211081542,1799338,2024-12-02,2024-12,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,3,3.0,0.0,0.0,3.0,0.0,0.0,2024-12-05,M0
26831,A20241202163230256,1801757,2024-12-02,2024-12,搜索渠道,全新,带锁,3,3.0,0.0,0.0,3.0,0.0,0.0,2024-12-05,M0
26833,A20241202163507283,1801784,2024-12-02,2024-12,搜索渠道,全新,未带锁,3,3.0,0.0,0.0,3.0,0.0,0.0,2024-12-05,M0
26850,A2024120303145916,1803381,2024-12-03,2024-12,芝麻租物,全新,未带锁,2,2.0,0.0,0.0,2.0,0.0,0.0,2024-12-06,M0
26852,A2024120309283384,1803757,2024-12-03,2024-12,付费灯火,全新,未带锁,2,2.0,0.0,0.0,2.0,0.0,0.0,2024-12-06,M0
26882,A20241203132058120,1805344,2024-12-03,2024-12,芝麻租物,全新,未带锁,2,2.0,0.0,0.0,2.0,0.0,0.0,2024-12-06,M0
26895,A2024120314074744,1805670,2024-12-03,2024-12,搜索渠道,全新,未带锁,2,2.0,0.0,0.0,2.0,0.0,0.0,2024-12-06,M0
26911,A2024120315021333,1806150,2024-12-03,2024-12,搜索渠道,全新,未带锁,2,2.0,0.0,0.0,2.0,0.0,0.0,2024-12-06,M0
26915,A20241203151326132,1806249,2024-12-03,2024-12,搜索渠道,全新,未带锁,2,2.0,0.0,0.0,2.0,0.0,0.0,2024-12-06,M0


# 12_差异化还款方式

In [100]:
# （1）trace关联
sql1='''-- trace关联表 
SELECT order_trace_id, risk_trace_id, trace_id, order_id
from db_credit.order_association
;
'''
trace_df = query(sql1)
trace_df.shape
trace_df[:2]

# order_trace_id 有重复，没有空
# order_id 无重复，没有空
# trace_df[trace_df['order_trace_id'] == '441581199811230010']

# 三者非空的时候，order_trace_id = risk_trace_id，但 trace_id 不一定和risk_trace_id/order_trace_id相等
# tmp1 = trace_df[(trace_df.trace_id.notna()) & (trace_df.order_trace_id.notna()) & (trace_df.risk_trace_id.notna())]
# tmp1.shape
# tmp1[tmp1.trace_id== tmp1.order_trace_id].shape
# tmp1[tmp1.trace_id== tmp1.risk_trace_id].shape
# tmp1[tmp1.order_trace_id== tmp1.risk_trace_id].shape
# tmp1[tmp1.trace_id!= tmp1.risk_trace_id][:1]

# （2）区别用哪个模型
# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
sql1='''-- risk关联表 
SELECT trace_id, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.is_model') end,'"','') as is_model
from db_credit.risk
;
'''
risk_df = query(sql1)
risk_df.shape
risk_df[:2]

(1639943, 4)

,order_trace_id,risk_trace_id,trace_id,order_id
0,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,88929
1,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,88931


(246846, 2)

,trace_id,is_model
0,91535f1aafe335dbddfa46a1317dd939,None
1,769ce80d7b55eb8f4e8d64cb3a9e9cef,None


In [101]:
# （2）自有模型 2311
sql1='''-- 风控-自有模型-ascore2311 
SELECT trace_id, score
from db_credit.risk_private_model
;
'''
mx_2311_df =query(sql1)
mx_2311_df['model_score_2311'] = pd.to_numeric(mx_2311_df['score'], errors='coerce')
mx_2311_df.shape
# df_rule4 = trace_df.merge(mx_2311_df[['trace_id', 'model_score_2311']], left_on='order_trace_id', right_on='trace_id', how='outer')
# df_rule4['trace_id_x'] = np.where(df_rule4['trace_id_y'].isna(), df_rule4['trace_id_x'], df_rule4['trace_id_y'])
# df_rule4.drop('trace_id_y', axis=1, inplace=True)
# df_rule4.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
# df_rule4.shape

# 153727
# df_rule4[(df_rule4.trace_id_x.notna()) & (df_rule4.trace_id_y.notna()) & (df_rule4.trace_id_x != df_rule4.trace_id_y)].shape
# df_rule4[(df_rule4.trace_id_x.notna()) & (df_rule4.trace_id_y.notna()) & (df_rule4.trace_id_x != df_rule4.trace_id_y)][:2]
# df_rule4[df_rule4.trace_id == '12f8bdc71e41d304140df3a262cc4951']
# trace_df[trace_df.trace_id == 'd7458a0a0c4be099fc2d1f1bfbf94908']
# d7458a0a0c4be099fc2d1f1bfbf94908
# 12f8bdc71e41d304140df3a262cc4951
# df_rule4[df_rule4.order_trace_id != df_rule4.trace_id].shape

(176474, 3)

In [102]:
# （3）自有模型 2401
sql1='''-- 风控-自有模型-2401
SELECT trace_id,  score
from db_credit.risk_private_model_2401
;
'''
mx_2401_df =query(sql1)
mx_2401_df['model_score_2401'] = pd.to_numeric(mx_2401_df['score'], errors='coerce')
mx_2401_df.shape
# df_rule5 = df_rule4.merge(mx_2401_df[['trace_id', 'model_score_2401']], left_on='order_trace_id', right_on='trace_id', how='outer')
# df_rule5['trace_id_x'] = np.where(df_rule5['trace_id_y'].isna(), df_rule5['trace_id_x'], df_rule5['trace_id_y'])
# df_rule5.drop('trace_id_y', axis=1, inplace=True)
# df_rule5.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
# df_rule4.shape
# df_rule5.shape

# df_rule5[(df_rule5['trace_id_y'].notnull()) & (df_rule5['trace_id_x'].notnull())].shape
# df_rule5[(df_rule5.trace_id_x.notna()) & (df_rule5.trace_id_y.notna()) & (df_rule5.trace_id_x != df_rule5.trace_id_y)].shape
# df_rule5[(df_rule5.trace_id_x.isna())].shape
# df_rule5[(df_rule5.trace_id_y.isna())].shape
# df_rule5[(df_rule5.trace_id_y.isna()) & (df_rule5.trace_id_x.isna())].shape

# df_rule5[(df_rule5['trace_id_y'].isna())].shape
# df_rule5[(df_rule5['trace_id_y'].isna()) & (df_rule5['trace_id_x'].notna())].shape
# df_rule5[(df_rule5['trace_id_y'].isna()) & (df_rule5['trace_id_x'].notna())][:1]
# df_rule5[(df_rule5['trace_id_y'].notnull()) & (df_rule5['trace_id_x'].isna())].shape
# df_rule5[(df_rule5['trace_id_y'].notnull()) & (df_rule5['trace_id_x'].isna())][:2]

# df_rule5[(df_rule5['trace_id_y'].notnull()) & (df_rule5['trace_id_x'].isna())][:2]
# df_rule5[df_rule5.trace_id == '807b44aedc4fe11242e7059a8ab4233a']
# mx_2401_df[mx_2401_df.trace_id == '807b44aedc4fe11242e7059a8ab4233a']
# df_rule4[df_rule4.trace_id == '807b44aedc4fe11242e7059a8ab4233a']
# mx_2311_df[mx_2311_df.trace_id == '807b44aedc4fe11242e7059a8ab4233a']
# mx_yl_df[mx_yl_df.trace_id == '807b44aedc4fe11242e7059a8ab4233a']

# trace_df[trace_df.trace_id == '807b44aedc4fe11242e7059a8ab4233a']
# trace_df[trace_df.order_trace_id == '807b44aedc4fe11242e7059a8ab4233a']

(153875, 3)

In [103]:
# （4）自有模型-银联
sql1='''-- 风控-自有模型-yl
SELECT trace_id,  score
from db_credit.risk_private_model_yl
;
'''
mx_yl_df =query(sql1)
mx_yl_df['model_score_yl'] = pd.to_numeric(mx_yl_df['score'], errors='coerce')
mx_yl_df.shape
# df_rule6 = df_rule5.merge(mx_yl_df[['trace_id', 'model_score_yl']], left_on='order_trace_id', right_on='trace_id', how='outer')
# df_rule6.drop('trace_id_y', axis=1, inplace=True)
# df_rule6.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
# df_rule5.shape
# df_rule6.shape

(176390, 3)

In [77]:
# （5）表关联-x
# 模型之间表关联
df_model = mx_2311_df[['trace_id', 'model_score_2311']].merge(mx_2401_df[['trace_id', 'model_score_2401']], on = 'trace_id', how='outer')
df_model.shape

df_model = df_model.merge(mx_yl_df[['trace_id', 'model_score_yl']], on = 'trace_id', how='outer')
df_model.shape

# df_model[df_model.trace_id.isnull()].shape
df_model_data = trace_df.merge(df_model, left_on='order_trace_id', right_on='trace_id', how='outer')
# df_model_data['trace_id_x'] = np.where(df_model_data['trace_id_y'].isna(), df_model_data['trace_id_x'], df_model_data['trace_id_y'])
df_model_data.drop('trace_id_y', axis=1, inplace=True)
df_model_data.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
df_model_data.shape
# df_model_data[:2]

# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
df_model_data = df_model_data.merge(risk_df, left_on='order_trace_id', right_on='trace_id', how='left')
df_model_data.drop('trace_id_y', axis=1, inplace=True)
df_model_data.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
df_model_data.shape

# 905794
# 130529199408200314
# c0c146b56d21e4ac76294738e469314e  --order_trace_id，中间表
# 11ff9a370c5d4faca8342a7520ec1590  --trace_id，中间表

df_model_data[df_model_data.trace_id.isin(['c0c146b56d21e4ac76294738e469314e','807b44aedc4fe11242e7059a8ab4233a','d7458a0a0c4be099fc2d1f1bfbf94908','12f8bdc71e41d304140df3a262cc4951'])]
df_model_data[df_model_data.order_trace_id.isin(['c0c146b56d21e4ac76294738e469314e','807b44aedc4fe11242e7059a8ab4233a','d7458a0a0c4be099fc2d1f1bfbf94908','12f8bdc71e41d304140df3a262cc4951'])]
# trace_df[trace_df.order_trace_id == 'c0c146b56d21e4ac76294738e469314e']

# trace_df[trace_df.trace_id.isna()].shape
# df_model_data[df_model_data.trace_id_x.isna()].shape
# df_model_data[df_model_data.trace_id_y.isna()].shape
# df_model_data[(df_model_data.trace_id_x.notna()) & (df_model_data.trace_id_y.notna()) & (df_model_data.trace_id_x != df_model_data.trace_id_y)].shape
# df_model_data[(df_model_data.trace_id_x.notna()) & (df_model_data.trace_id_y.notna()) & (df_model_data.trace_id_x != df_model_data.trace_id_y)][:2]

# trace_df 中无信息，df_model 有
# df_model_data[df_model_data.order_id.isna()].shape
# df_model_data[df_model_data.order_id.isna()][:2]

(176421, 3)

(176421, 4)

(1648769, 7)

(1648769, 8)

,order_trace_id,risk_trace_id,trace_id,order_id,model_score_2311,model_score_2401,model_score_yl,is_model
548670,12f8bdc71e41d304140df3a262cc4951,12f8bdc71e41d304140df3a262cc4951,d7458a0a0c4be099fc2d1f1bfbf94908,782401.0,19.514414,NaN,16.430973,None


,order_trace_id,risk_trace_id,trace_id,order_id,model_score_2311,model_score_2401,model_score_yl,is_model
548670,12f8bdc71e41d304140df3a262cc4951,12f8bdc71e41d304140df3a262cc4951,d7458a0a0c4be099fc2d1f1bfbf94908,782401.0,19.514414,NaN,16.430973,None
658459,c0c146b56d21e4ac76294738e469314e,c0c146b56d21e4ac76294738e469314e,11ff9a370c5d4faca8342a7520ec1590,905794.0,20.137972,8.230204,26.646283,1


In [72]:
# （6）表关联-y
df_model_data = df_model_data[df_model_data.order_id.notnull()]
df_model_data.shape
df_model_data.drop_duplicates(subset='order_id').shape

df_model_data = df_model_data.merge(df_analy_data, on = 'order_id', how = 'left')
df_model_data.shape
df_analy_data.shape
df_model_data[df_model_data.历史最大逾期天数 >= 0].shape
df_model_data[df_model_data.order_id.isin([508404,1678492,1692916,12449])]

# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]
# trace_df[trace_df.order_id.isin([508404,1678492,1692916,12449])]

(1637557, 8)

(1637557, 8)

(1637559, 23)

(31803, 16)

(31333, 23)

,order_trace_id,risk_trace_id,trace_id,order_id,model_score_2311,model_score_2401,model_score_yl,is_model,order_number,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,首期应还日期,overdue_type
306928,cd34506b628cd3415a627dbc034ec150,cd34506b628cd3415a627dbc034ec150,cd34506b628cd3415a627dbc034ec150,508404.0,NaN,NaN,NaN,None,A2023073114301076,2023-07-31,2023-07,芝麻租物,全新,未带锁,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-03,
1307221,121ff3d22a97dc24a5b32881520a53ec,121ff3d22a97dc24a5b32881520a53ec,3a50eeaf674eb1d46d1d0c2ccbee9a65,1678492.0,24.828763,16.362452,20.876086,2,A20241103155713186,2024-11-03,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,62.0,62.0,0.0,0.0,62.0,31.0,0.0,2024-11-06,M2
1319296,1f29a38e64ff9266c9ba7d75440d9bbc,1f29a38e64ff9266c9ba7d75440d9bbc,bfe2abcf2de19c099fde0948d9333636,1692916.0,16.494677,12.835012,19.159146,2,A20241107103123201,2024-11-07,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,8.0,8.0,0.0,0.0,8.0,0.0,0.0,2024-11-10,


In [143]:
# （7）标签定义
# 第N期逾期天数：为截止2024.1.8的逾期天数
# 反欺诈 FPD、SPD、TPD >=7
# 信用 （1）第二、三、四期逾期天数>=31； （2）历史最大逾期 >=31
df_model_data['fraud_tmp'] = np.where(df_model_data.FPD >= df_model_data.SPD, df_model_data.FPD, df_model_data.SPD)
df_model_data['fraud_tmp'] = np.where(df_model_data.fraud_tmp >= df_model_data.TPD, df_model_data.fraud_tmp, df_model_data.TPD)
df_model_data['y_fraud'] = df_model_data.apply(lambda x: 0 if x.fraud_tmp == 0 
                                               else 0.5 if x.fraud_tmp >0 and x.fraud_tmp < 7
                                               else 1 if x.fraud_tmp >= 7
                                               else 2, axis = 1)
df_model_data['credit_tmp'] = np.where(df_model_data.第二期逾期天数 >= df_model_data.第三期逾期天数, df_model_data.第二期逾期天数, df_model_data.第三期逾期天数)
df_model_data['credit_tmp'] = np.where(df_model_data.credit_tmp >= df_model_data.第四期逾期天数, df_model_data.credit_tmp, df_model_data.第四期逾期天数)
df_model_data['y_credit'] = df_model_data.apply(lambda x: 0 if x.credit_tmp == 0 
                                               else 0.5 if x.credit_tmp >0 and x.credit_tmp < 31
                                               else 1 if x.credit_tmp >= 31
                                               else 2, axis = 1)

df_model_data.drop(columns=['fraud_tmp','credit_tmp'],inplace = True)

df_model_data['y_fraud'].value_counts()
df_model_data['y_credit'].value_counts()

y_fraud
2.0    1533613
0.0      23416
0.5       3714
1.0       2800
Name: count, dtype: int64

y_credit
2.0    1533614
0.0      23416
0.5       4216
1.0       2297
Name: count, dtype: int64

In [144]:
# （8）样本统计
# df_model_data[(df_model_data.y_fraud == 1) & (df_model_data.y_credit == 1) & (df_model_data.search_time == '2024-01')].shape
pd.pivot_table(df_model_data,index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(df_model_data,index='search_time',columns='y_credit',values='order_number',aggfunc='count')

y_fraud,0.0,0.5,1.0
search_time,,,
2023-01,92.0,47.0,26.0
2023-02,216.0,54.0,62.0
2023-03,209.0,46.0,61.0
2023-04,404.0,125.0,105.0
2023-05,318.0,102.0,68.0
2023-06,368.0,133.0,87.0
2023-07,602.0,146.0,134.0
2023-08,595.0,154.0,130.0
2023-09,637.0,149.0,104.0


y_credit,0.0,0.5,1.0,2.0
search_time,,,,
2023-01,92.0,50.0,23.0,NaN
2023-02,216.0,62.0,54.0,NaN
2023-03,209.0,57.0,50.0,NaN
2023-04,404.0,157.0,73.0,NaN
2023-05,318.0,109.0,61.0,NaN
2023-06,368.0,139.0,81.0,NaN
2023-07,602.0,151.0,129.0,NaN
2023-08,595.0,167.0,116.0,1.0
2023-09,637.0,166.0,87.0,NaN


In [200]:
# （9）样本划分
# 0：没有使用模型
# 1：ascore2311       18.321 16.6--搜索渠道，2024.4.27上线
# 2：ascore_yl2311    17.55  15.059--搜索渠道，2024.4.27上线
# 3：ascore2401       11.04  9.35--搜索渠道，2024.4.27上线

# 2    79476
# 1    36520
# 3    29039
# 0     8908

conditions_1 = ((df_model_data.search_time >= '2023-11') & (df_model_data.search_time <= '2024-09'))
conditions_2 = (df_model_data.y_fraud.isin([0,1]))
conditions_3 = (df_model_data.y_credit.isin([0,1]))
train_0 = df_model_data[conditions_1 & conditions_2 & (df_model_data.is_model == '0')]
train_1 = df_model_data[conditions_1 & conditions_2 & (df_model_data.is_model == '1')]
train_2 = df_model_data[conditions_1 & conditions_2 & (df_model_data.is_model == '2')]
train_3 = df_model_data[conditions_1 & conditions_2 & (df_model_data.is_model == '3')]
# df_model_data[conditions_1 & conditions_2 & (df_model_data.is_model.isnull())].shape

train_0.shape
train_1.shape
train_2.shape
train_3.shape
412 + 2547 + 5812 + 2489

df_model_data[df_model_data.is_model == '0'].下单日期.max()
df_model_data[df_model_data.is_model == '1'].下单日期.max()
df_model_data[df_model_data.is_model == '2'].下单日期.max()
df_model_data[df_model_data.is_model == '3'].下单日期.max()
pd.pivot_table(df_model_data[df_model_data.is_model == '1'],index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(df_model_data[df_model_data.is_model == '2'],index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(df_model_data[df_model_data.is_model == '3'],index='search_time',columns='y_fraud',values='order_number',aggfunc='count')

pd.pivot_table(df_model_data[df_model_data.is_model == '1'],index='search_time',columns='y_credit',values='order_number',aggfunc='count')
pd.pivot_table(df_model_data[df_model_data.is_model == '2'],index='search_time',columns='y_credit',values='order_number',aggfunc='count')
pd.pivot_table(df_model_data[df_model_data.is_model == '3'],index='search_time',columns='y_credit',values='order_number',aggfunc='count')

(859, 24)

(3298, 24)

(6116, 24)

(2489, 24)

11260

...

y_fraud,0.0,0.5,1.0
search_time,,,
2023-11,126.0,29.0,23.0
2023-12,134.0,27.0,21.0
2024-01,157.0,23.0,15.0
2024-02,99.0,15.0,11.0
2024-03,147.0,33.0,17.0
2024-04,409.0,85.0,62.0
2024-05,541.0,108.0,81.0
2024-06,669.0,135.0,100.0
2024-07,860.0,149.0,107.0


y_fraud,0.0,0.5,1.0
search_time,,,
2024-02,1.0,NaN,NaN
2024-03,242.0,47.0,35.0
2024-04,454.0,120.0,70.0
2024-05,184.0,38.0,24.0
2024-06,252.0,55.0,30.0
2024-07,297.0,66.0,44.0
2024-08,347.0,49.0,59.0
2024-09,392.0,71.0,58.0
2024-10,273.0,31.0,21.0


y_credit,0.0,0.5,1.0
search_time,,,
2023-11,262.0,44.0,46.0
2023-12,396.0,64.0,43.0
2024-01,227.0,42.0,20.0
2024-02,116.0,29.0,5.0
2024-03,130.0,28.0,12.0
2024-04,241.0,73.0,34.0
2024-05,209.0,52.0,21.0
2024-06,279.0,61.0,30.0
2024-07,303.0,64.0,43.0


y_credit,0.0,0.5,1.0
search_time,,,
2023-11,126.0,31.0,21.0
2023-12,134.0,31.0,17.0
2024-01,157.0,23.0,15.0
2024-02,99.0,17.0,9.0
2024-03,147.0,31.0,19.0
2024-04,409.0,86.0,61.0
2024-05,541.0,125.0,64.0
2024-06,669.0,148.0,87.0
2024-07,860.0,166.0,90.0


y_credit,0.0,0.5,1.0
search_time,,,
2024-02,1.0,NaN,NaN
2024-03,242.0,49.0,33.0
2024-04,454.0,130.0,60.0
2024-05,184.0,41.0,21.0
2024-06,252.0,52.0,33.0
2024-07,297.0,73.0,37.0
2024-08,347.0,53.0,55.0
2024-09,392.0,97.0,32.0
2024-10,273.0,46.0,6.0


In [ ]:
# （10）分箱
# model_score_2311
# model_score_yl
# model_score_2401

# 1：ascore2311       18.321 16.6--搜索渠道，2024.4.27上线
# 2：ascore_yl2311    17.55  15.059--搜索渠道，2024.4.27上线
# 3：ascore2401       11.04  9.35--搜索渠道，2024.4.27上线

# df_model_data[['model_score_2311','model_score_yl','model_score_2401']][:1]

%run C:/Users/Administrator/Desktop/mine/4_专题分析/1_回捞分析/2.model_func.ipynb
continous_merged_dict = {}
var_bin_list = []
# 渠道和产品都要修改
trainData_kf_Bin = train_3[train_3.归属渠道 != '搜索渠道'].copy()
# trainData_kf_Bin = train_3[train_3.归属渠道 == '搜索渠道'].copy()
trainData_kf_Bin['y'] = trainData_kf_Bin['y_fraud']
numeric_desc_var = ['model_score_2401']

# start_time = time.time()

for col in numeric_desc_var:
    print ("{} is in processing".format(col))
    if -1111 not in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]): 
        max_interval = 5 
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval,special_attribute=[],minBinPcnt=0)
        trainData_kf_Bin[col+'_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff,special_attribute=[]))
        monotone = BadRateMonotone(trainData_kf_Bin, col+'_Bin', 'y')
        while(not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute = [],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = []))
            if max_interval == 2:
                break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y')
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [])) 
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    else:
        max_interval = 7 #正常取值最多分5箱
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval = max_interval, special_attribute = [-1111, -999],
                                      minBinPcnt = 0)
        trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1111, -999]))
        monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2'])
        while (not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute=[-1111, -999],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))
            if max_interval == 3:
                break
#             if max_interval == 3 and -1111 in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 3 and -1111 not in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 4 and -1111 in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2']) 
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))  
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    continous_merged_dict[col] = cutOff
    
# print("- Time consumed: %s seconds" % int(time.time() - start_time))

# continous_merged_dict
# model_score_2311：非搜索
# # [12.962097708761268,
# #   15.22112016264258,
# #   17.280072769844406]
# 搜索
# {'model_score_2311': [15.777008619403436, 18.228734839062696]}

# model_score_yl：非搜索
# {'model_score_yl': [13.513768497314754, 14.978775576930174, 17.9695729833906]}

# model_score_2401：非搜索
# {'model_score_2401': [6.19076711035361, 8.138647666450124, 13.48526442400384]}
# 搜索
# {'model_score_2401': [8.87302928213804]}

model_score_2401 is in processing


In [248]:
# （11）统计2023.11-2024.9
# train_1[(train_1.归属渠道 == '搜索渠道') & (train_1.model_score_2311 > 21.435)].下单日期.max()
train_11 = df_model_data[conditions_1 & (df_model_data.is_model == '1')]
train_22 = df_model_data[conditions_1 & (df_model_data.is_model == '2')]
train_33 = df_model_data[conditions_1 & (df_model_data.is_model == '3')]

train_11['model_score_2311_Bin'] = train_11.apply(lambda x: '1.[0,12.9620]' if x.归属渠道 == '芝麻租物' and x.model_score_2311 >= 0  and x.model_score_2311 <= 12.9620
                                                else '2.(12.9620,15.2211]' if x.归属渠道 == '芝麻租物' and x.model_score_2311 > 12.9620  and x.model_score_2311 <= 15.2211
                                                else '3.(15.2211,18.321]' if x.归属渠道 == '芝麻租物' and x.model_score_2311 > 15.2211  and x.model_score_2311 <= 18.321
                                                else '4.(18.321, inf)' if x.归属渠道 == '芝麻租物' and x.model_score_2311 > 18.321

                                                else '5.[0,18.321]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2311 >= 0  and x.model_score_2311 <= 18.321
                                                else '6.(18.321, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2311 > 18.321

                                                else '7.[0,16.6]' if x.归属渠道 == '搜索渠道' and x.model_score_2311 >= 0  and x.model_score_2311 <= 16.6
                                                else '8.(16.6, inf]' if x.归属渠道 == '搜索渠道' and x.model_score_2311 > 16.6
                                                else 'other'
                                                , axis = 1)

train_22['model_score_yl_Bin'] = train_22.apply(lambda x: '1.[0,13.5137]' if x.归属渠道 == '芝麻租物' and x.model_score_yl >= 0  and x.model_score_yl <= 13.5137
                                                else '2.(13.5137,14.9787]' if x.归属渠道 == '芝麻租物' and x.model_score_yl > 13.5137  and x.model_score_yl <= 14.9787
                                                else '3.(14.9787,17.55]' if x.归属渠道 == '芝麻租物' and x.model_score_yl > 14.9787  and x.model_score_yl <= 17.55
                                                else '4.(17.55,inf)' if x.归属渠道 == '芝麻租物' and x.model_score_yl > 17.55 

                                                else '5.[0,17.55]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl >= 0  and x.model_score_yl <= 17.55
                                                else '6.(17.55, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl > 17.55

                                                else '7.[0,10]' if x.归属渠道 == '搜索渠道' and x.model_score_yl >= 0  and x.model_score_yl <= 10
                                                else '8.(10,15.059]' if x.归属渠道 == '搜索渠道' and x.model_score_yl > 10  and x.model_score_yl <= 15.059
                                                else '9.(15.059, inf]' if x.归属渠道 == '搜索渠道' and x.model_score_yl > 15.059
                                                else 'other'
                                                , axis = 1)

train_33['model_score_2401_Bin'] = train_33.apply(lambda x: '1.[0,6.1907]' if x.归属渠道 == '芝麻租物' and x.model_score_2401 >= 0  and x.model_score_2401 <= 6.1907
                                                else '2.(6.1907,8.1386]' if x.归属渠道 == '芝麻租物' and x.model_score_2401 > 6.1907  and x.model_score_2401 <= 8.1386
                                                else '3.(8.1386,11.04]' if x.归属渠道 == '芝麻租物' and x.model_score_2401 > 8.1386  and x.model_score_2401 <= 11.04
                                                else '4.(11.04, inf]' if x.归属渠道 == '芝麻租物' and x.model_score_2401 > 11.04

                                                else '5.[0,11.04]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl >= 0  and x.model_score_yl <= 11.04
                                                else '6.(11.04 , inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl > 11.04 

                                                else '7.[0,9.35]' if x.归属渠道 == '搜索渠道' and x.model_score_2401 >= 0  and x.model_score_2401 <= 9.35
                                                else '8.(9.35, inf]' if x.归属渠道 == '搜索渠道' and x.model_score_2401 > 9.35
                                                else 'other'
                                                , axis = 1)

pd.pivot_table(train_11,index='model_score_2311_Bin',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(train_22,index='model_score_yl_Bin',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(train_33,index='model_score_2401_Bin',columns='y_fraud',values='order_number',aggfunc='count')

pd.pivot_table(train_11,index='model_score_2311_Bin',columns='y_credit',values='order_number',aggfunc='count')
pd.pivot_table(train_22,index='model_score_yl_Bin',columns='y_credit',values='order_number',aggfunc='count')
pd.pivot_table(train_33,index='model_score_2401_Bin',columns='y_credit',values='order_number',aggfunc='count')

y_fraud,0.0,0.5,1.0
model_score_2311_Bin,,,
"1.[0,12.9620]",281,28,20
"2.(12.9620,15.2211]",590,90,55
"3.(15.2211,18.321]",774,138,105
"4.(18.321, inf)",414,95,64
"5.[0,18.321]",225,47,46
"6.(18.321, inf)",79,19,24
"7.[0,16.6]",330,82,48
"8.(16.6, inf]",211,57,32


y_fraud,0.0,0.5,1.0
model_score_yl_Bin,,,
"1.[0,13.5137]",1853.0,233.0,193.0
"2.(13.5137,14.9787]",453.0,78.0,53.0
"3.(14.9787,17.55]",751.0,143.0,115.0
"4.(17.55,inf)",721.0,153.0,118.0
"5.[0,17.55]",393.0,70.0,68.0
"6.(17.55, inf)",113.0,27.0,41.0
"7.[0,10]",104.0,10.0,10.0
"8.(10,15.059]",548.0,109.0,72.0
"9.(15.059, inf]",427.0,118.0,81.0


y_fraud,0.0,0.5,1.0
model_score_2401_Bin,,,
"1.[0,6.1907]",348,47,19
"2.(6.1907,8.1386]",312,55,38
"3.(8.1386,11.04]",506,107,73
"4.(11.04, inf]",345,81,53
"5.[0,11.04]",37,5,3
"6.(11.04 , inf)",146,44,40
"7.[0,9.35]",332,61,65
"8.(9.35, inf]",143,46,29


In [249]:
# （12）统计2024.10-2025.1.7
conditions_4 = (df_model_data.search_time >= '2024-10')
df_model_data['y_fraud_FPD'] = df_model_data.apply(lambda x: 0 if x.FPD == 0 
                                               else 0.5 if x.FPD > 0 and x.FPD < 7
                                               else 1 if x.FPD >= 7
                                               else 2, axis = 1)

train_111 = df_model_data[conditions_4 & (df_model_data.is_model == '1')]
train_222 = df_model_data[conditions_4 & (df_model_data.is_model == '2')]
train_333 = df_model_data[conditions_4 & (df_model_data.is_model == '3')]

train_111['model_score_2311_Bin'] = train_111.apply(lambda x: '1.[0,12.9620]' if x.归属渠道 == '芝麻租物' and x.model_score_2311 >= 0  and x.model_score_2311 <= 12.9620
                                                else '2.(12.9620,15.2211]' if x.归属渠道 == '芝麻租物' and x.model_score_2311 > 12.9620  and x.model_score_2311 <= 15.2211
                                                else '3.(15.2211,18.321]' if x.归属渠道 == '芝麻租物' and x.model_score_2311 > 15.2211  and x.model_score_2311 <= 18.321
                                                else '4.(18.321, inf)' if x.归属渠道 == '芝麻租物' and x.model_score_2311 > 18.321

                                                else '5.[0,18.321]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2311 >= 0  and x.model_score_2311 <= 18.321
                                                else '6.(18.321, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2311 > 18.321

                                                else '7.[0,16.6]' if x.归属渠道 == '搜索渠道' and x.model_score_2311 >= 0  and x.model_score_2311 <= 16.6
                                                else '8.(16.6, inf]' if x.归属渠道 == '搜索渠道' and x.model_score_2311 > 16.6
                                                else 'other'
                                                , axis = 1)

train_222['model_score_yl_Bin'] = train_222.apply(lambda x: '1.[0,13.5137]' if x.归属渠道 == '芝麻租物' and x.model_score_yl >= 0  and x.model_score_yl <= 13.5137
                                                else '2.(13.5137,14.9787]' if x.归属渠道 == '芝麻租物' and x.model_score_yl > 13.5137  and x.model_score_yl <= 14.9787
                                                else '3.(14.9787,17.55]' if x.归属渠道 == '芝麻租物' and x.model_score_yl > 14.9787  and x.model_score_yl <= 17.55
                                                else '4.(17.55,inf)' if x.归属渠道 == '芝麻租物' and x.model_score_yl > 17.55 

                                                else '5.[0,17.55]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl >= 0  and x.model_score_yl <= 17.55
                                                else '6.(17.55, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl > 17.55

                                                else '7.[0,10]' if x.归属渠道 == '搜索渠道' and x.model_score_yl >= 0  and x.model_score_yl <= 10
                                                else '8.(10,15.059]' if x.归属渠道 == '搜索渠道' and x.model_score_yl > 10  and x.model_score_yl <= 15.059
                                                else '9.(15.059, inf]' if x.归属渠道 == '搜索渠道' and x.model_score_yl > 15.059
                                                else 'other'
                                                , axis = 1)

train_333['model_score_2401_Bin'] = train_333.apply(lambda x: '1.[0,6.1907]' if x.归属渠道 == '芝麻租物' and x.model_score_2401 >= 0  and x.model_score_2401 <= 6.1907
                                                else '2.(6.1907,8.1386]' if x.归属渠道 == '芝麻租物' and x.model_score_2401 > 6.1907  and x.model_score_2401 <= 8.1386
                                                else '3.(8.1386,11.04]' if x.归属渠道 == '芝麻租物' and x.model_score_2401 > 8.1386  and x.model_score_2401 <= 11.04
                                                else '4.(11.04, inf]' if x.归属渠道 == '芝麻租物' and x.model_score_2401 > 11.04

                                                else '5.[0,11.04]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl >= 0  and x.model_score_yl <= 11.04
                                                else '6.(11.04 , inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl > 11.04 

                                                else '7.[0,9.35]' if x.归属渠道 == '搜索渠道' and x.model_score_2401 >= 0  and x.model_score_2401 <= 9.35
                                                else '8.(9.35, inf]' if x.归属渠道 == '搜索渠道' and x.model_score_2401 > 9.35
                                                else 'other'
                                                , axis = 1)

pd.pivot_table(train_111[train_111.search_time == '2024-10'],index='model_score_2311_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(train_111[train_111.search_time == '2024-11'],index='model_score_2311_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(train_111[train_111.search_time == '2024-12'],index='model_score_2311_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(train_111[train_111.search_time == '2025-01'],index='model_score_2311_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')

pd.pivot_table(train_222[train_222.search_time == '2024-10'],index='model_score_yl_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(train_222[train_222.search_time == '2024-11'],index='model_score_yl_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(train_222[train_222.search_time == '2024-12'],index='model_score_yl_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(train_222[train_222.search_time == '2025-01'],index='model_score_yl_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')

pd.pivot_table(train_333[train_333.search_time == '2024-10'],index='model_score_2401_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(train_333[train_333.search_time == '2024-11'],index='model_score_2401_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(train_333[train_333.search_time == '2024-12'],index='model_score_2401_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(train_333[train_333.search_time == '2025-01'],index='model_score_2401_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')

y_fraud_FPD,0.0,0.5,1.0
model_score_2311_Bin,,,
"1.[0,12.9620]",9.0,NaN,NaN
"2.(12.9620,15.2211]",29.0,2.0,NaN
"3.(15.2211,18.321]",53.0,2.0,1.0
"4.(18.321, inf)",32.0,NaN,2.0
"5.[0,18.321]",27.0,2.0,1.0
"6.(18.321, inf)",12.0,NaN,1.0
"7.[0,16.6]",61.0,1.0,NaN
"8.(16.6, inf]",47.0,3.0,2.0


y_fraud_FPD,0.0,0.5,1.0
model_score_2311_Bin,,,
"1.[0,12.9620]",10.0,1.0,NaN
"2.(12.9620,15.2211]",32.0,NaN,NaN
"3.(15.2211,18.321]",74.0,3.0,NaN
"4.(18.321, inf)",70.0,6.0,2.0
"5.[0,18.321]",46.0,NaN,1.0
"6.(18.321, inf)",15.0,2.0,2.0
"7.[0,16.6]",96.0,6.0,NaN
"8.(16.6, inf]",78.0,5.0,2.0


y_fraud_FPD,0.0,0.5
model_score_2311_Bin,,
"1.[0,12.9620]",6.0,NaN
"2.(12.9620,15.2211]",30.0,1.0
"3.(15.2211,18.321]",119.0,NaN
"4.(18.321, inf)",113.0,1.0
"5.[0,18.321]",36.0,1.0
"6.(18.321, inf)",20.0,1.0
"7.[0,16.6]",131.0,NaN
"8.(16.6, inf]",121.0,2.0


y_fraud_FPD,0.0
model_score_2311_Bin,
"1.[0,12.9620]",1
"2.(12.9620,15.2211]",5
"3.(15.2211,18.321]",20
"4.(18.321, inf)",29
"5.[0,18.321]",8
"6.(18.321, inf)",4
"7.[0,16.6]",23
"8.(16.6, inf]",17


y_fraud_FPD,0.0,0.5,1.0
model_score_yl_Bin,,,
"1.[0,13.5137]",172.0,5.0,2.0
"2.(13.5137,14.9787]",52.0,2.0,2.0
"3.(14.9787,17.55]",75.0,4.0,2.0
"4.(17.55,inf)",111.0,10.0,8.0
"5.[0,17.55]",70.0,2.0,2.0
"6.(17.55, inf)",41.0,1.0,2.0
"7.[0,10]",28.0,NaN,1.0
"8.(10,15.059]",203.0,7.0,3.0
"9.(15.059, inf]",176.0,6.0,7.0


...

y_fraud_FPD,0.0
model_score_yl_Bin,
"1.[0,13.5137]",30
"2.(13.5137,14.9787]",36
"3.(14.9787,17.55]",24
"4.(17.55,inf)",51
"5.[0,17.55]",12
"6.(17.55, inf)",11
"7.[0,10]",7
"8.(10,15.059]",57
"9.(15.059, inf]",76


y_fraud_FPD,0.0,0.5,1.0
model_score_2401_Bin,,,
"1.[0,6.1907]",29.0,1.0,1.0
"2.(6.1907,8.1386]",23.0,2.0,1.0
"3.(8.1386,11.04]",52.0,NaN,1.0
"4.(11.04, inf]",26.0,NaN,2.0
"5.[0,11.04]",6.0,NaN,1.0
"6.(11.04 , inf)",40.0,2.0,1.0
"7.[0,9.35]",70.0,7.0,2.0
"8.(9.35, inf]",52.0,5.0,1.0


y_fraud_FPD,0.0,0.5,1.0
model_score_2401_Bin,,,
"1.[0,6.1907]",28.0,1.0,NaN
"2.(6.1907,8.1386]",37.0,NaN,NaN
"3.(8.1386,11.04]",69.0,4.0,1.0
"4.(11.04, inf]",63.0,3.0,2.0
"5.[0,11.04]",10.0,NaN,NaN
"6.(11.04 , inf)",58.0,4.0,2.0
"7.[0,9.35]",105.0,3.0,7.0
"8.(9.35, inf]",102.0,6.0,7.0


y_fraud_FPD,0.0,0.5
model_score_2401_Bin,,
"1.[0,6.1907]",47.0,NaN
"2.(6.1907,8.1386]",63.0,NaN
"3.(8.1386,11.04]",96.0,2.0
"4.(11.04, inf]",96.0,1.0
"5.[0,11.04]",10.0,NaN
"6.(11.04 , inf)",52.0,NaN
"7.[0,9.35]",152.0,1.0
"8.(9.35, inf]",114.0,2.0


y_fraud_FPD,0.0
model_score_2401_Bin,
"1.[0,6.1907]",7
"2.(6.1907,8.1386]",10
"3.(8.1386,11.04]",15
"4.(11.04, inf]",16
"5.[0,11.04]",1
"6.(11.04 , inf)",7
"7.[0,9.35]",38
"8.(9.35, inf]",36


In [264]:
# （12）数据检查
# 部分订单没有模型分，豁免
# df_model_data[(df_model_data.y_fraud_FPD.isin([0,1,0.5])) & df_model_data.is_model.isin(['1', '2', '3']) & (df_model_data.search_time >= '2024-10')].search_time.value_counts()
# 没有调自有模型的出库订单
df_model_data[(df_model_data.y_fraud_FPD.isin([0,1,0.5])) & df_model_data.is_model.isnull() & (df_model_data.search_time >= '2024-10')].search_time.value_counts()
df_model_data[(df_model_data.y_fraud_FPD.isin([0,1,0.5])) & df_model_data.is_model.isnull() & (df_model_data.search_time >= '2024-10')].FPD.max()
# df_model_data[(df_model_data.y_fraud_FPD.isin([0,1,0.5])) & df_model_data.is_model.isnull()  & (df_model_data.search_time >= '2024-10')][['order_number', '下单日期', '归属渠道', 'y_fraud_FPD', 'model_score_2311','model_score_yl','model_score_2401']]
pd.pivot_table(df_model_data[(df_model_data.y_fraud_FPD.isin([0,1,0.5])) & df_model_data.is_model.isnull() & (df_model_data.search_time >= '2024-10')],index='search_time',columns='y_fraud_FPD',values='order_number',aggfunc='count')

search_time
2024-12    28
2025-01    27
2024-11    17
Name: count, dtype: int64

0.0

y_fraud_FPD,0.0
search_time,
2024-11,17
2024-12,28
2025-01,27


In [271]:
# （13）目标订单的还款方式
sql = '''
SELECT  distinct tod.order_id, tod.sku_attributes
from db_digua_business.t_order_details tod
;
'''

df_zlfangan = query(sql)
df_zlfangan.shape

df_zlfangan.loc[:,"租赁方案"]=np.where(df_zlfangan["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_zlfangan[:2]

tmp1 = train_111[(train_111.search_time >= '2024-10') & (train_111.model_score_2311_Bin == '6.(18.321, inf)')][['order_id', 'order_number','search_time']]
tmp2 = train_222[(train_222.search_time >= '2024-10') & (train_222.model_score_yl_Bin == '6.(17.55, inf)')][['order_id', 'order_number','search_time']]
tmp3 = train_333[(train_333.search_time >= '2024-10') & (train_333.model_score_2401_Bin == '6.(11.04 , inf)')][['order_id', 'order_number','search_time']]

tmp = pd.concat([tmp1, tmp2, tmp3])
tmp = tmp.merge(df_zlfangan[['order_id', '租赁方案']], on = 'order_id', how = 'left')
# tmp['租赁方案'].value_counts()
pd.pivot_table(tmp,index='search_time',columns='租赁方案',values='order_number',aggfunc='count')

,order_id,sku_attributes,租赁方案
0,8742,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",租完即送
1,8743,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",租完即送


租赁方案,租完即送,租完归还
search_time,,
2024-10,26,74
2024-11,26,132
2024-12,21,118
2025-01,4,18


# 13_联合运营

## 13_1.20250109 风险测算

<!-- 对综合风险等级为0的进件进行分析测算 -->

In [335]:
# （1）数据导入
# 分析日期：20250109
# 鑫宗帮忙更新数据
path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/4_策略分析/'
my_risk_level = pd.read_excel(path + '风险等级_更新20250109.xlsx')
my_risk_level['search_time'] = my_risk_level['下单日期'].dt.strftime('%Y-%m')
my_risk_level.shape
my_risk_level[:2]

path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'
df_maoli = pd.read_excel(path + '实际毛利因子_20250109.xlsx')
df_maoli.drop(columns = ['Unnamed: 0'], inplace = True)
df_maoli = df_maoli.rename(columns = {'订单号': 'order_number'})
df_maoli.shape
df_maoli[:2]
# (237077, 13)
# (250705, 13)

(250705, 13)

,order_id,trace_id,下单日期,order_number,拒绝理由,归属渠道,综合风险等级,细化综合风险等级,共租行为等级,履约历史等级,蚂蚁数控风险等级,蚁盾分,search_time
0,8747,NaN,2022-06-13,A202206131732031,NaN,未知渠道,NaN,NaN,NaN,NaN,NaN,NaN,2022-06
1,8765,NaN,2022-06-25,A202206250925411,NaN,未知渠道,NaN,NaN,NaN,NaN,NaN,NaN,2022-06


(29452, 19)

,order_number,租赁方案,总租金,滞纳金,预估残值2,已收租金,已付残值/已付买断尾款,违约金,总应收,总已收2,采购成本,实际毛利,实际毛利2,还款状态,是否买断,产品名称,订单状态,订单完成时间,当前时间
0,A202207050851373,租完归还,4295.40,0.0,1931.25,7847.20,329.72,0.0,6226.65,8176.92,5150.0,3026.92,3026.92,已买断,1,99新 iPhone 12 Pro 国行 官方质检【租物特惠】内存：128G 颜色：金色,已完成,2024-04-17 01:37:33,2025-01-09
1,A202207151707153,租完归还,4799.16,NaN,2037.45,4799.16,2598.91,0.0,6836.61,7398.07,4794.0,2604.07,2604.07,已买断,1,全新vivo X80 Pro 高通骁龙8蔡司智能5G【租物特惠】内存：12GB+256G 颜...,已完成,2023-06-30 11:54:29,2025-01-09


In [336]:
# （2）进件分布
pd.pivot_table(my_risk_level, index='search_time',columns='综合风险等级',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[my_risk_level.归属渠道	== '芝麻租物'], index='search_time',columns='综合风险等级',values='order_number',aggfunc='count')

综合风险等级,-2,0,1,2,3,DEFAULT
search_time,,,,,,
2023-05,22.0,186.0,397.0,542.0,248.0,NaN
2023-06,27.0,687.0,1458.0,1808.0,872.0,NaN
2023-07,NaN,1057.0,2081.0,2545.0,974.0,NaN
2023-08,1.0,1097.0,2434.0,3172.0,935.0,NaN
2023-09,NaN,1084.0,2230.0,2585.0,629.0,1.0
2023-10,NaN,1496.0,2896.0,3407.0,890.0,NaN
2023-11,NaN,2413.0,4305.0,4594.0,1008.0,NaN
2023-12,NaN,1658.0,3096.0,3073.0,208.0,1.0
2024-01,NaN,1596.0,2926.0,2812.0,152.0,NaN


综合风险等级,-2,0,1,2,3,DEFAULT
search_time,,,,,,
2023-05,6.0,74.0,138.0,188.0,51.0,NaN
2023-06,9.0,355.0,656.0,708.0,231.0,NaN
2023-07,NaN,685.0,1278.0,1603.0,430.0,NaN
2023-08,1.0,650.0,1472.0,1965.0,431.0,NaN
2023-09,NaN,647.0,1275.0,1497.0,301.0,NaN
2023-10,NaN,832.0,1612.0,1903.0,483.0,NaN
2023-11,NaN,1377.0,2372.0,2556.0,517.0,NaN
2023-12,NaN,1014.0,1753.0,1740.0,113.0,1.0
2024-01,NaN,895.0,1539.0,1491.0,81.0,NaN


In [343]:
# （3）关联租后表现
var_1 = ['order_id', 'order_number', '下单日期', 'search_time', '归属渠道', '拒绝理由', '综合风险等级']
var_2 = ['order_number', '历史最大逾期天数', 'FPD', 'SPD', 'TPD', '第二期逾期天数', '第三期逾期天数', '第四期逾期天数']
# var_2 = ['order_number', '历史最大逾期天数', 'FPD', 'SPD', 'TPD', '第二期逾期天数', '第三期逾期天数', '第四期逾期天数', 'search_time']
# df_my = my_risk_level[var_1].merge(df_analy_data[var_2], on = 'order_number', how = 'outer')
df_my = my_risk_level[var_1].merge(df_analy_data[var_2], on = 'order_number', how = 'left')
# df_my = df_my.merge(df_maoli, on = 'order_number', how = 'outer')
df_my = df_my.merge(df_maoli, on = 'order_number', how = 'left')

df_my['综合风险等级'] = np.where(df_my['综合风险等级'] == '-2', '-999', my_risk_level['综合风险等级'])
df_my['综合风险等级'] = np.where((df_my['历史最大逾期天数'] >= 0) & (df_my['综合风险等级'].isna()), '-999', my_risk_level['综合风险等级'])

# 237077
df_my.shape
df_my[:2]
# (237077, 14)
# (237258, 14)
# (237241, 32)
# (237268, 33)
# 181 = 237258 - 237077
# 177
# df_my[(df_my.order_id.isnull()) & (df_my.search_time_y >= '2023-05')].shape
# df_my[(df_my.order_id.isnull()) & (df_my.search_time_y >= '2023-05')].search_time_y.value_counts()

(250705, 32)

,order_id,order_number,下单日期,search_time,归属渠道,拒绝理由,综合风险等级,历史最大逾期天数,FPD,SPD,...,总已收2,采购成本,实际毛利,实际毛利2,还款状态,是否买断,产品名称,订单状态,订单完成时间,当前时间
0,8747,A202206131732031,2022-06-13,2022-06,未知渠道,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,8765,A202206250925411,2022-06-25,2022-06,未知渠道,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [344]:
# （4）标签定义
df_my['y_credit'] = df_my.apply(lambda x: 0 if x.历史最大逾期天数 == 0 
                                else 0.5 if x.历史最大逾期天数 >0 and x.历史最大逾期天数 < 31
                                else 1 if x.历史最大逾期天数 >= 31
                                else 2, axis = 1)

pd.pivot_table(df_my, index='search_time',columns='y_credit',values='order_number',aggfunc='count')
pd.pivot_table(df_my[df_my.归属渠道	== '芝麻租物'], index='search_time',columns='y_credit',values='order_number',aggfunc='count')

y_credit,0.0,0.5,1.0,2.0
search_time,,,,
2022-06,NaN,NaN,NaN,8.0
2022-07,2.0,10.0,10.0,318.0
2022-08,3.0,23.0,30.0,1419.0
2022-09,4.0,22.0,18.0,1630.0
2022-10,4.0,21.0,16.0,1157.0
2022-11,8.0,22.0,13.0,735.0
2022-12,14.0,28.0,24.0,1073.0
2023-01,41.0,88.0,91.0,2718.0
2023-02,103.0,108.0,121.0,3701.0


y_credit,0.0,0.5,1.0,2.0
search_time,,,,
2023-02,21.0,5.0,4.0,129.0
2023-03,25.0,13.0,15.0,353.0
2023-04,96.0,70.0,74.0,1823.0
2023-05,50.0,48.0,47.0,1111.0
2023-06,84.0,81.0,98.0,1749.0
2023-07,173.0,178.0,199.0,3666.0
2023-08,184.0,159.0,175.0,4170.0
2023-09,242.0,170.0,174.0,3963.0
2023-10,255.0,265.0,282.0,4337.0


In [368]:
# （5）综合风险等级分析
# （5-1）综合风险等级为0的进件、风控通过情况
df_my[(df_my.综合风险等级 == '0') & (df_my.归属渠道	== '芝麻租物')].shape
df_my[(df_my.综合风险等级 == '0') & (df_my.归属渠道	== '芝麻租物') & (df_my.拒绝理由.notna())].shape
df_my[(df_my.综合风险等级 == '0') & (df_my.归属渠道	== '芝麻租物') & (df_my.拒绝理由.isna())].shape
pd.pivot_table(df_my[(df_my.综合风险等级 == '0') & (df_my.归属渠道	== '芝麻租物') & ((df_my.拒绝理由.isna()) | (df_my.历史最大逾期天数 >= 0))], index='search_time',columns='综合风险等级',values='order_number',aggfunc='count')
pd.pivot_table(df_my[(df_my.综合风险等级 == '0') & (df_my.归属渠道	== '芝麻租物') & (df_my.历史最大逾期天数 >= 0)], index='search_time',columns='综合风险等级',values='order_number',aggfunc='count')

# （5-2）租后表现
# 2023.5-2024.3
pd.pivot_table(df_my[(df_my.search_time >= '2023-05') & (df_my.search_time <= '2024-03')], index='综合风险等级',columns='y_credit',values='order_number',aggfunc='count')
pd.pivot_table(df_my[(df_my.search_time >= '2023-05') & (df_my.search_time <= '2024-03') & (df_my.归属渠道	== '芝麻租物')], index='综合风险等级',columns='y_credit',values='order_number',aggfunc='count')

# df_my[(df_my.search_time >= '2023-05') & (df_my.search_time <= '2024-03') & (df_my.综合风险等级.isna()) & (df_my.y_credit.isin([0,0.5,1]))].shape
# df_my[(df_my.search_time >= '2023-05') & (df_my.search_time <= '2024-03') & (df_my.综合风险等级.isna()) & (df_my.y_credit.isin([0,0.5,1]))][:2]
# df_my[(df_my.search_time >= '2023-05') & (df_my.search_time <= '2024-03') & (df_my.综合风险等级.isna()) & (df_my.y_credit.isin([0,0.5,1])) & (df_my.归属渠道 == '芝麻租物')].shape
# df_my[(df_my.search_time >= '2023-05') & (df_my.search_time <= '2024-03') & (df_my.综合风险等级.isna()) & (df_my.y_credit.isin([0,0.5,1])) & (df_my.归属渠道 == '芝麻租物')].search_time.value_counts()

# df_my[(df_my.综合风险等级 == '0') & (df_my.归属渠道	== '芝麻租物') & (df_my.历史最大逾期天数 >= 0) & (df_my.拒绝理由.notnull())].shape
# df_my[(df_my.综合风险等级 == '0') & (df_my.归属渠道	== '芝麻租物') & (df_my.历史最大逾期天数 >= 0)].拒绝理由.value_counts()
# pd.pivot_table(df_my[(df_my.search_time >= '2023-05') & (df_my.search_time <= '2024-03') & (df_my.归属渠道	== '芝麻租物') & (df_my.拒绝理由 == '命中云商分低于515强拒(芝麻综合风险等级0豁免)')], index='综合风险等级',columns='y_credit',values='order_number',aggfunc='count')


(26446, 33)

(7493, 33)

(18953, 33)

综合风险等级,0
search_time,
2023-05,44
2023-06,230
2023-07,499
2023-08,483
2023-09,465
2023-10,586
2023-11,1020
2023-12,764
2024-01,644


综合风险等级,0
search_time,
2023-05,10
2023-06,60
2023-07,133
2023-08,109
2023-09,114
2023-10,194
2023-11,320
2023-12,214
2024-01,179


y_credit,0.0,0.5,1.0,2.0
综合风险等级,,,,
-2,NaN,3.0,1.0,46.0
-999,234.0,180.0,208.0,NaN
0,1186.0,779.0,708.0,10860.0
1,1204.0,917.0,1037.0,22097.0
2,862.0,754.0,1094.0,24776.0
3,57.0,63.0,102.0,5763.0
DEFAULT,NaN,NaN,NaN,2.0


y_credit,0.0,0.5,1.0,2.0
综合风险等级,,,,
-2,NaN,3.0,NaN,13.0
-999,116.0,80.0,76.0,NaN
0,823.0,496.0,398.0,6338.0
1,840.0,607.0,600.0,12330.0
2,588.0,510.0,667.0,13846.0
3,42.0,36.0,57.0,2546.0
DEFAULT,NaN,NaN,NaN,1.0


In [387]:
# （6）目标客群毛利测算
df_my_maoli = df_my[(df_my.综合风险等级 == '0') & (df_my.归属渠道	== '芝麻租物') & (df_my.历史最大逾期天数 >= 0)]
df_my_maoli.shape
# df_my_maoli[:2]
# pd.pivot_table(df_my_maoli, index='search_time',columns='实际毛利2',values='order_number',aggfunc='count')
# df_my_maoli.groupby(['search_time']).实际毛利2.sum()
df_my_maoli.to_excel('tmp.xlsx', index = False)

# df_maoli_1 = df_maoli.merge(df_my[['order_number', 'search_time']], on = 'order_number', how = 'left')
# df_maoli_1.to_excel('tmp_1.xlsx', index = False)

(6943, 33)

In [389]:
# （7）R0的截面数据
# 20250113
# （7-1）取租后截面数据
sql = '''-- 租后应收监控  
SELECT  tprm.order_id,  tprm.overdue_type, tprm.unrecovered_cost
from db_digua_business.t_postlease_receivables_monitoring   tprm 
;
'''

dfzh3 = query(sql)
dfzh3.shape

# （7-2）碎屏险
# `price`： '价格'
# special_price`： '优惠价'
sql = ''' 
SELECT  distinct order_id,price
from db_digua_business.t_service_order
'''

df_s_o = query(sql)
df_s_o = df_s_o.fillna(0)
df_s_o.shape
df_s_o.drop_duplicates(subset=["order_id"],inplace=True)
df_s_o.shape

# （7-3）表合并
dfzh3.shape
dfzh3 = dfzh3.merge(df_s_o, on = 'order_id', how = 'left')
dfzh3['price'] = dfzh3['price'].fillna(0)
dfzh3.shape

dfzh3["未回收成本"] = dfzh3.unrecovered_cost - dfzh3.price

dfzh3 = dfzh3.merge(df_my_maoli, on = 'order_id', how = 'right')
dfzh3 = dfzh3.rename(columns = {'search_time': '下单月份', '采购成本': '采购金额', 'overdue_type': '逾期类型'})
dfzh3.shape

df_zt = dfzh3[dfzh3["未回收成本"]>0]    #.groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")

df_zt['M0'] = np.where(df_zt.逾期类型=='M0',df_zt["未回收成本"],0)
df_zt['M1'] = np.where(df_zt.逾期类型=='M1',df_zt["未回收成本"],0)
df_zt['M2'] = np.where(df_zt.逾期类型=='M2',df_zt["未回收成本"],0)
df_zt['M3'] = np.where(df_zt.逾期类型=='M3',df_zt["未回收成本"],0)
df_zt['M4'] = np.where(df_zt.逾期类型=='M4',df_zt["未回收成本"],0)
df_zt['M5'] = np.where(df_zt.逾期类型=='M5',df_zt["未回收成本"],0)
df_zt['M6'] = np.where(df_zt.逾期类型=='M6',df_zt["未回收成本"],0)
dfyq = df_zt.groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=dfzh3[dfzh3["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"] = dfyq.apply(lambda x:x.sum(),axis=1) 
dfyq1=pd.merge(dfyq,dfzh3.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']]

(32798, 3)

(212755, 2)

(211803, 2)

(32798, 3)

(32798, 4)

(6943, 37)

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2023-05,0.00,0.00,0.00,0.00,3974.12,3974.12,67745.00,5.87%,5.87%,5.87%,5.87%
2023-06,0.00,0.00,0.00,151.73,84567.38,84719.11,399969.00,21.18%,21.18%,21.18%,21.14%
2023-07,0.00,344.62,0.00,38363.15,98097.01,136804.78,921260.00,14.85%,14.85%,14.81%,10.65%
2023-08,0.00,1196.20,887.18,18805.69,57998.65,78887.72,760227.00,10.38%,10.38%,10.10%,7.63%
2023-09,2299.12,2184.33,3598.19,45773.66,63338.48,117193.78,829264.00,14.13%,13.86%,13.16%,7.64%
2023-10,617.13,3569.83,24402.24,30233.74,194932.31,253755.25,1502267.00,16.89%,16.85%,14.99%,12.98%
2023-11,3006.15,46026.99,27134.98,56444.26,232933.01,365545.39,2505916.00,14.59%,14.47%,11.55%,9.30%
2023-12,32370.52,17382.91,4375.48,34007.64,113147.10,201283.65,1649858.75,12.20%,10.24%,8.92%,6.86%
2024-01,9595.27,3760.30,2831.82,28176.60,61672.05,106036.04,1314678.00,8.07%,7.34%,6.83%,4.69%


## 13_2.20250117 线下回溯

In [78]:
# （1）样本准备
# 2025.1.17
# 2023.10及之后的样本，本次选取 2023.10-2024.10 的进件，trace_id、逾期标签（M1+、M3+、FPD31+）、品类（全新、二手、非手机）
# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]
# （7-1-1）历史逾期
# 2025.1.17 运行 -- 回溯提供样本
df_lhyy_data = df_analy_data.copy()
df_lhyy_data.shape
df_lhyy_data['label_fpd'] = df_lhyy_data.apply(lambda x: 0 if x.FPD == 0 
                                               else 0.5 if x.FPD >=1 and x.FPD <= 30
                                               else 1 if x.FPD >=31
                                               else 2, axis = 1)
df_lhyy_data['label_overdue_his'] = df_lhyy_data.apply(lambda x: 'M0' if x.历史最大逾期天数 == 0 
                                               else 'M1' if x.历史最大逾期天数 >=1 and x.历史最大逾期天数 <= 30
                                               else 'M2' if x.历史最大逾期天数 >=31 and x.历史最大逾期天数 <= 60
                                               else 'M3+' if x.历史最大逾期天数 >= 61
                                               else 'other', axis = 1)

# （7-1-2）当前逾期
# M0：1-30
# M1:31-60
# M2：61-90
# M3:91-180
# M6：>180
# sql = '''-- 租后应收监控  
# SELECT  order_number,overdue_type
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# '''
# df_zhys = query(sql)
# # tmp.search_time.value_counts()
# df_lhyy_data = df_lhyy_data.merge(df_zhys, on = 'order_number', how = 'left')
df_lhyy_data['label_overdue_now'] = df_lhyy_data.apply(lambda x: 'M1' if x.overdue_type == 'M0'
                                                       else 'M2' if x.overdue_type == 'M1'
                                                       else 'M3+' if x.overdue_type == 'M2' or x.overdue_type == 'M3' or x.overdue_type == 'M6'
                                                       else 'M0', axis = 1)
# （7-1-3）取 trace_id
sql1='''-- trace关联表 
SELECT trace_id, order_number
from db_credit.order_association
;
'''

trace_df =query(sql1)
trace_df.shape

# （7-1-4）关联 out_order_no
sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT trace_id,out_order_no
from db_credit.risk_alipay_auth_query
;
'''
zm_fk_df = query(sql1)
zm_fk_df.shape

# （7-1-5）表关联
trace_df = trace_df.merge(zm_fk_df, on = 'trace_id', how = 'left')
trace_df.shape


df_lhyy_data = df_lhyy_data.merge(trace_df[['order_number', 'trace_id', 'out_order_no']], on = 'order_number', how = 'left')
df_lhyy_data = df_lhyy_data[['order_number', 'trace_id', 'out_order_no', '下单日期', 'search_time', '是否二手', 'label_fpd', 'label_overdue_his', 'label_overdue_now']]
df_lhyy_data.shape
df_lhyy_data = df_lhyy_data[(df_lhyy_data.search_time >= '2023-10') & ((df_lhyy_data.search_time <= '2024-10'))]
df_lhyy_data.shape
# 261单没有调三方数据
df_lhyy_data = df_lhyy_data[~(df_lhyy_data.trace_id.isnull())]
df_lhyy_data.shape
df_lhyy_data.drop_duplicates(subset = ['order_number']).shape
df_lhyy_data.drop_duplicates(subset = ['trace_id','下单日期']).shape

# 432 单没有 out_order_no
df_lhyy_data = df_lhyy_data[~(df_lhyy_data.out_order_no.isnull())]
df_lhyy_data.shape
df_lhyy_data.drop_duplicates(subset = ['order_number']).shape
df_lhyy_data.drop_duplicates(subset = ['out_order_no','下单日期']).shape
df_lhyy_data = df_lhyy_data.drop_duplicates(subset = ['out_order_no','下单日期'])
df_lhyy_data = df_lhyy_data[['out_order_no', '下单日期', '是否二手', 'label_fpd', 'label_overdue_his', 'label_overdue_now']]
df_lhyy_data[:3]

# df_lhyy_data[df_lhyy_data.out_order_no.isnull()].shape
# df_lhyy_data[df_lhyy_data.out_order_no.isnull()][:3]
# sql1='''-- 芝麻租物风控专业版
# -- 95184	95184	95184
# SELECT trace_id,out_order_no
# from db_credit.risk_alipay_auth_query
# where trace_id = '548dc3edeaff6c0d13e55dac9cbd2aee'
# ;
# '''
# query(sql1)

# sql1='''-- trace关联表 
# SELECT trace_id, order_number
# from db_credit.order_association
# where trace_id = '548dc3edeaff6c0d13e55dac9cbd2aee'
# ;
# '''
# query(sql1)

# df_lhyy_data[df_lhyy_data.trace_id	== '548dc3edeaff6c0d13e55dac9cbd2aee']

# df_lhyy_data[df_lhyy_data.trace_id.isnull()].search_time.value_counts()
# pd.pivot_table(df_lhyy_data[df_lhyy_data.trace_id.isnull()],index='search_time',columns='label_overdue_his',values='order_number',aggfunc='count')
# df_lhyy_data[(df_lhyy_data.trace_id.isnull())&(df_lhyy_data.search_time == '2024-08')]
# df_lhyy_data[df_lhyy_data.trace_id.isnull()].shape
# pd.pivot_table(df_lhyy_data,index='search_time',columns='label_fpd',values='order_number',aggfunc='count')
# pd.pivot_table(df_lhyy_data,index='search_time',columns='label_overdue_his',values='order_number',aggfunc='count')

# tmp = df_lhyy_data.groupby(['out_order_no','下单日期']).agg({'order_number':'count'}).reset_index()
# tmp[tmp.order_number > 1]
# df_lhyy_data[df_lhyy_data.out_order_no == 'A20240927154825297']
# df_lhyy_data[df_lhyy_data.out_order_no == '3ba6180b2f1b46d29004e03f0260c437']

# df_lhyy_data.to_excel('lhyy_sample_data_发送.xlsx', index = False)

(31679, 16)

(1629082, 2)

(198597, 2)

(1629082, 3)

(31679, 9)

...

(18370, 9)

(17943, 9)

(17941, 9)

(17939, 9)

,out_order_no,下单日期,是否二手,label_fpd,label_overdue_his,label_overdue_now
5553,1a3e4a0afd1d499aab322dbf0f50f984,2023-10-02,二手,0.0,M2,M3+
5554,f839c89ddcf24786be3e64347a2dd2f3,2023-10-02,全新,0.0,M1,M0
5555,d7d94b80f78f4005b48722f756aa4799,2023-10-02,全新,0.0,M1,M0


## 13_3.20250126 回溯分析

In [142]:
# （1）更新租后表现
# 2025.1.26 运行
# label_fpd：首逾标签
# label_overdue_his：历史最大逾期标签
# label_overdue_now：当前逾期标签

df_lhyy_data = df_analy_data.copy()
df_lhyy_data['label_fpd'] = df_lhyy_data.apply(lambda x: 0 if x.FPD == 0 
                                               else 0.5 if x.FPD >=1 and x.FPD <=6
                                               else 1 if x.FPD >=7
                                               else 2, axis = 1)
df_lhyy_data['label_overdue_his'] = df_lhyy_data.apply(lambda x: 0 if x.历史最大逾期天数 == 0 
                                               else 0.5 if x.历史最大逾期天数 >=1 and x.历史最大逾期天数 <= 30
                                               else 1 if x.历史最大逾期天数 >=31 
                                               else 'other', axis = 1)
df_lhyy_data['label_overdue_now'] = df_lhyy_data.apply(lambda x: 0.5 if x.overdue_type == 'M0'
                                                       else 1 if x.overdue_type == 'M1' or x.overdue_type == 'M2' or x.overdue_type == 'M3' or x.overdue_type == 'M6'
                                                       else 0, axis = 1)
df_lhyy_data.shape
df_lhyy_data[df_lhyy_data.order_id.isin([508404,1678492,1692916,12449])]

(31679, 19)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,首期应还日期,overdue_type,label_fpd,label_overdue_his,label_overdue_now
28,A202208052103231,12449,2022-08-05,2022-08,搜索渠道,全新,未带锁,19,0.0,11.0,0.0,0.0,11.0,9.0,2022-08-05,M6,0.0,0.5,1.0
3719,A2023073114301076,508404,2023-07-31,2023-07,芝麻租物,全新,未带锁,6,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-03,,0.0,0.5,0.0
24302,A20241103155713186,1678492,2024-11-03,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,51,51.0,0.0,0.0,51.0,20.0,0.0,2024-11-06,M1,1.0,1.0,1.0
24566,A20241107103123201,1692916,2024-11-07,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,8,8.0,0.0,0.0,8.0,0.0,0.0,2024-11-10,,1.0,0.5,0.0


In [143]:
# （2）导入回溯结果数据
# 返回结果 outer_order_id 去重
path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/5_联合运营/2_数据返回/'
df_lhyy_in = pd.read_excel(path + 'lhyy_sample_data_diguazuji_return.xlsx')
df_lhyy_in = df_lhyy_in.rename(columns = {'outer_order_id': 'out_order_no'})
df_lhyy_in.shape
df_lhyy_in.drop_duplicates(subset = ['out_order_no']).shape
df_lhyy_in[:2]

(17902, 2)

(17902, 2)

,out_order_no,zm_lvl
0,2dcd7d90ce314b44ad8949bf5319bb94,R4
1,5d2e67d765fc47e0885db72ae6054407,R1


In [144]:
# （3）表关联
# （3-1）取 trace_id
sql1='''-- trace关联表 
SELECT trace_id, order_number
from db_credit.order_association
;
'''

trace_df =query(sql1)
trace_df.shape

# （3-2）关联 out_order_no
sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT trace_id,out_order_no
from db_credit.risk_alipay_auth_query
;
'''
zm_fk_df = query(sql1)
zm_fk_df.shape

# （3-3）表关联
trace_df = trace_df.merge(zm_fk_df, on = 'trace_id', how = 'left')
trace_df.shape

# 发散，有些没有租后表现
df_lhyy_in = df_lhyy_in.merge(trace_df, on = 'out_order_no', how = 'inner')
df_lhyy_in.shape
df_lhyy_in.drop_duplicates(subset = ['out_order_no']).shape
# df_lhyy_in[df_lhyy_in.out_order_no == 'A2024082909142423']

df_lhyy_analy = df_lhyy_in.merge(df_lhyy_data, on = 'order_number', how = 'inner')
df_lhyy_analy.shape
df_lhyy_analy.drop_duplicates(subset = ['order_number']).shape
df_lhyy_analy.drop_duplicates(subset = ['out_order_no','下单日期']).shape
df_lhyy_analy.drop_duplicates(subset = ['out_order_no']).shape

# 订单维度分析
df_lhyy_analy = df_lhyy_analy.drop_duplicates(subset = ['order_number'])
df_lhyy_analy.shape
df_lhyy_analy[:3]
# df_lhyy_analy[df_lhyy_analy.label_overdue_now.isnull()].shape

(1629082, 2)

(198632, 2)

(1629082, 3)

(24751, 4)

(17902, 4)

...

(17941, 22)

(17939, 22)

(17902, 22)

(17941, 22)

,out_order_no,zm_lvl,trace_id,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,首期应还日期,overdue_type,label_fpd,label_overdue_his,label_overdue_now
0,2dcd7d90ce314b44ad8949bf5319bb94,R4,7f82e463fd5f6afb2c7f59d7d1d87d70,A2023102515163441,724228,2023-10-25,2023-10,芝麻租物,全新,未带锁,212,0.0,0.0,0.0,0.0,0.0,0.0,2023-10-28,M6,0.0,1.0,1.0
1,5d2e67d765fc47e0885db72ae6054407,R1,b78378ba2be052f6a3624020d6f48ef0,A20240705135538202,1217103,2024-07-05,2024-07,搜索渠道,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0,2024-07-08,,0.0,0.0,0.0
2,78638d53cb3c4dd9b08dda303384b3ca,R3,d97578ec84e781650eed4a90e52d75fb,A20240606164601110,1109003,2024-06-06,2024-06,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0,2024-06-09,,0.0,0.0,0.0


In [159]:
# （4）统计
# df_lhyy_analy['zm_lvl'] = np.where(df_lhyy_analy.zm_lvl.isin(['R5','R6']), 'R5&R6', df_lhyy_analy.zm_lvl)
param_search_time = '2023-10'
conditions_1 = (df_lhyy_analy.归属渠道 == '芝麻租物')
pd.pivot_table(df_lhyy_analy[conditions_1 & (df_lhyy_analy.search_time == param_search_time)],index='zm_lvl',columns='label_fpd',values='order_number',aggfunc='count')
pd.pivot_table(df_lhyy_analy[conditions_1 & (df_lhyy_analy.search_time == param_search_time)],index='zm_lvl',columns='label_overdue_his',values='order_number',aggfunc='count')
pd.pivot_table(df_lhyy_analy[conditions_1 & (df_lhyy_analy.search_time == param_search_time)],index='zm_lvl',columns='label_overdue_now',values='order_number',aggfunc='count')

label_fpd,0.0,0.5,1.0
zm_lvl,,,
R0,162,4,2
R1,152,8,1
R2,108,4,4
R3,104,10,5
R4,103,11,9
R5&R6,59,7,13


label_overdue_his,0.0,0.5,1.0
zm_lvl,,,
R0,78,55,35
R1,63,49,49
R2,37,41,38
R3,35,34,50
R4,24,44,55
R5&R6,5,26,48


label_overdue_now,0.0,0.5,1.0
zm_lvl,,,
R0,122,5,41
R1,102,5,54
R2,63,7,46
R3,62,6,51
R4,57,7,59
R5&R6,29,2,48


In [192]:
# （5）R0的截面数据
# 20250126
# （5-1）取租后截面数据
sql = '''-- 租后应收监控  
SELECT  tprm.order_id, tprm.purchase_amount, tprm.unrecovered_cost
from db_digua_business.t_postlease_receivables_monitoring   tprm 
;
'''

dfzh3 = query(sql)
dfzh3.shape

# （5-2）碎屏险
# `price`： '价格'
# special_price`： '优惠价'
sql = ''' 
SELECT  distinct order_id,price
from db_digua_business.t_service_order
'''

df_s_o = query(sql)
df_s_o = df_s_o.fillna(0)
df_s_o.shape
df_s_o.drop_duplicates(subset=["order_id"],inplace=True)
df_s_o.shape

# （7-3）表合并
dfzh3.shape
dfzh3 = dfzh3.merge(df_s_o, on = 'order_id', how = 'left')
dfzh3['price'] = dfzh3['price'].fillna(0)
dfzh3.shape

dfzh3["未回收成本"] = dfzh3.unrecovered_cost - dfzh3.price

# dfzh3.shape
# dfzh3[:1]

# dfzh3 = dfzh3.merge(df_my_maoli, on = 'order_id', how = 'right')
# 租物渠道且为R0的客群
dfzh3 = df_lhyy_analy[(df_lhyy_analy.归属渠道 == '芝麻租物')&(df_lhyy_analy.zm_lvl == 'R0')].merge(dfzh3, on = 'order_id', how = 'left')
dfzh3 = dfzh3.rename(columns = {'search_time': '下单月份', 'purchase_amount': '采购金额', 'overdue_type': '逾期类型'})
dfzh3.shape
dfzh3 = dfzh3.drop_duplicates(subset = ['order_number'])
dfzh3.shape
dfzh3[dfzh3["未回收成本"]>0].shape
dfzh3[dfzh3["未回收成本"]<0].shape

df_zt = dfzh3[dfzh3["未回收成本"]>0]    #.groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")

df_zt['M0'] = np.where(df_zt.逾期类型=='M0',df_zt["未回收成本"],0)
df_zt['M1'] = np.where(df_zt.逾期类型=='M1',df_zt["未回收成本"],0)
df_zt['M2'] = np.where(df_zt.逾期类型=='M2',df_zt["未回收成本"],0)
df_zt['M3'] = np.where(df_zt.逾期类型=='M3',df_zt["未回收成本"],0)
df_zt['M4'] = np.where(df_zt.逾期类型=='M4',df_zt["未回收成本"],0)
df_zt['M5'] = np.where(df_zt.逾期类型=='M5',df_zt["未回收成本"],0)
df_zt['M6'] = np.where(df_zt.逾期类型=='M6',df_zt["未回收成本"],0)
dfyq = df_zt.groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=dfzh3[dfzh3["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"] = dfyq.apply(lambda x:x.sum(),axis=1) 
dfyq1=pd.merge(dfyq,dfzh3.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']]

tmp = pd.pivot_table(df_zt[df_zt.逾期类型.isin(['M0','M1','M2','M3','M6'])],index='下单月份',columns='逾期类型',values='order_number',aggfunc='count').reset_index()
tmp = tmp.fillna(0)
tmp['合计逾期'] = tmp['M0'] + tmp['M1'] + tmp['M2'] + tmp['M3'] + tmp['M6']
tmp1 = dfzh3.groupby(['下单月份']).agg({'order_id':'count'}).reset_index()
tmp = tmp.merge(tmp1, on = '下单月份', how = 'outer')
tmp = tmp.rename(columns = {'order_id': '出库订单数'})
tmp

(33782, 3)

(217959, 2)

(216936, 2)

(33782, 3)

(33782, 4)

...

(2878, 26)

(2345, 26)

(525, 26)

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2023-10,2756.97,3205.23,8321.54,40415.72,110584.72,165284.18,1273952.25,12.97%,12.76%,11.85%,8.68%
2023-11,2563.76,13873.61,27278.82,25462.67,125850.86,195029.72,1948288.00,10.01%,9.88%,7.77%,6.46%
2023-12,4878.13,8992.85,1887.97,14177.33,57570.63,87506.91,826976.00,10.58%,9.99%,8.68%,6.96%
2024-01,3209.85,0.00,5793.06,11646.67,38294.67,58944.25,573992.00,10.27%,9.71%,8.70%,6.67%
2024-02,0.00,1070.58,0.00,10678.99,10012.26,21761.83,516803.00,4.21%,4.21%,4.00%,1.94%
2024-03,14131.46,7862.37,3728.22,20380.02,59411.48,105513.55,1228000.00,8.59%,7.44%,6.50%,4.84%
2024-04,25635.67,19467.05,23528.15,84829.87,53063.33,206524.07,2092024.00,9.87%,8.65%,6.59%,2.54%
2024-05,20448.82,14017.42,21317.81,69542.91,18813.60,144140.56,1408874.00,10.23%,8.78%,6.27%,1.34%
2024-06,26105.32,37232.51,48374.78,65560.46,14076.38,191349.45,1916826.00,9.98%,8.62%,4.15%,0.73%


,下单月份,M0,M1,M2,M3,M6,合计逾期,出库订单数
0,2023-10,4.0,2.0,5.0,12.0,21.0,44.0,168
1,2023-11,2.0,9.0,12.0,9.0,27.0,59.0,255
2,2023-12,5.0,7.0,1.0,6.0,12.0,31.0,119
3,2024-01,5.0,0.0,2.0,6.0,8.0,21.0,88
4,2024-02,0.0,1.0,0.0,3.0,2.0,6.0,81
5,2024-03,6.0,4.0,2.0,7.0,11.0,30.0,177
6,2024-04,11.0,6.0,6.0,17.0,8.0,48.0,288
7,2024-05,8.0,4.0,5.0,13.0,3.0,33.0,210
8,2024-06,9.0,9.0,11.0,11.0,2.0,42.0,313
9,2024-07,7.0,10.0,6.0,7.0,0.0,30.0,308


In [199]:
# （6）毛利测算
path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'
df_maoli = pd.read_excel(path + '实际毛利因子_20250109.xlsx')
df_maoli.drop(columns = ['Unnamed: 0'], inplace = True)
df_maoli = df_maoli.rename(columns = {'订单号': 'order_number'})
df_maoli.shape

df_maoli = df_maoli.merge(dfzh3[['order_number','下单月份']], on = 'order_number', how = 'right')
dfzh3.shape
df_maoli.shape
df_maoli[:2]
pd.pivot_table(df_maoli,index='下单月份',columns='还款状态',values='实际毛利2',aggfunc='sum').reset_index()
# df_maoli[(df_maoli.还款状态 == '还款中')&(df_maoli.下单月份 == '2023-11')]

(29452, 19)

(2878, 26)

(2878, 20)

,order_number,租赁方案,总租金,滞纳金,预估残值2,已收租金,已付残值/已付买断尾款,违约金,总应收,总已收2,采购成本,实际毛利,实际毛利2,还款状态,是否买断,产品名称,订单状态,订单完成时间,当前时间,下单月份
0,A20240714103649104,租完归还,5039.65,NaN,3240.0,2749.90,0.0,0.0,8279.65,2749.90,7200.0,1439.25,1079.65,还款中,0.0,iPhone15Pro内存：256G 颜色：白色钛金属,租赁中,NaT,2025-01-09,2024-07
1,A2024092410200296,租完归还,5918.19,NaN,3748.5,2153.52,0.0,0.0,9666.69,2153.52,8330.0,1587.79,1336.69,还款中,0.0,iPhone15ProMax内存：256G 颜色：原色钛金属,租赁中,NaT,2025-01-09,2024-09


还款状态,下单月份,已买断,已完成,已逾期,检测中,续租中,还款中
0,2023-10,104066.44,12233.73,-151583.31,0.0,80319.280,NaN
1,2023-11,195888.58,35413.55,-204072.75,NaN,183259.440,3614.720
2,2023-12,45776.27,4677.07,-97687.82,0.0,97407.675,NaN
3,2024-01,34488.70,4130.45,-56946.74,NaN,49379.235,59689.055
4,2024-02,19380.60,NaN,-28224.07,NaN,7765.570,110504.315
5,2024-03,33979.02,NaN,-104914.82,NaN,4129.870,235184.035
6,2024-04,49540.24,NaN,-192758.21,NaN,NaN,414721.270
7,2024-05,38205.60,NaN,-132127.06,0.0,NaN,298257.040
8,2024-06,17076.10,NaN,-190765.63,NaN,NaN,341321.340
9,2024-07,26447.20,NaN,-155956.84,NaN,NaN,310739.910


# 14_三方价格测算

In [405]:
# （1）冰鉴-火眸分
sql1='''-- 风控-冰鉴-火眸分
SELECT trace_id,  rs, time
from db_credit.risk_icekredit_cds
;
'''
bj_hm_df =query(sql1)
bj_hm_df['bj_hm_score'] = bj_hm_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('card_device_score22', {}).get('bj_score', None))  
bj_hm_df['bj_hm_score'] = bj_hm_df['bj_hm_score'].astype(int)
bj_hm_df['search_time'] = bj_hm_df['time'].dt.strftime('%Y-%m')
bj_hm_df.shape
bj_hm_df.drop_duplicates(subset = ['trace_id']).shape
bj_hm_df[bj_hm_df.bj_hm_score.isna()].shape
bj_hm_df[:2]
bj_hm_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(853367, 5)

(853367, 5)

(0, 5)

,trace_id,rs,time,bj_hm_score,search_time
0,19935d138af3f087b71d587c0d41693a,"{""result"": {""card_device_score22"": {""bj_score""...",2022-12-31 15:02:36,625,2022-12
1,29fc01aa38c3b251410f925c7698edc1,"{""result"": {""card_device_score22"": {""bj_score""...",2023-01-04 14:49:44,582,2023-01


,search_time,trace_id
0,2022-12,2
1,2023-01,2126
2,2023-02,4026
3,2023-03,4075
4,2023-04,6223
5,2023-05,4268
6,2023-06,5087
7,2023-07,40121
8,2023-08,39127
9,2023-09,42766


In [404]:
# （2）冰鉴-青云分
sql1='''-- 风控-冰鉴-青云分
SELECT trace_id, rs, time
from db_credit.risk_icekredit_qy
;
'''
bj_qy_df =query(sql1)
bj_qy_df['bj_qy_score'] = bj_qy_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('qing_yun22', {}).get('bj_score', None))  
bj_qy_df['bj_qy_score'] = bj_qy_df['bj_qy_score'].astype(int)
bj_qy_df['search_time'] = bj_qy_df['time'].dt.strftime('%Y-%m')
bj_qy_df.shape
bj_qy_df.drop_duplicates(subset = ['trace_id']).shape
bj_qy_df[bj_qy_df.bj_qy_score.isna()].shape
bj_qy_df[:2]
bj_qy_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()
# bj_qy_df.search_time.value_counts()

(229990, 5)

(229990, 5)

(0, 5)

,trace_id,rs,time,bj_qy_score,search_time
0,19935d138af3f087b71d587c0d41693a,"{""result"": {""qing_yun22"": {""bj_score"": 463}}, ...",2022-12-31 15:02:37,463,2022-12
1,29fc01aa38c3b251410f925c7698edc1,"{""result"": {""qing_yun22"": {""bj_score"": 594}}, ...",2023-01-04 14:49:44,594,2023-01


,search_time,trace_id
0,2022-12,2
1,2023-01,2202
2,2023-02,4026
3,2023-03,4065
4,2023-04,6164
5,2023-05,4265
6,2023-06,4861
7,2023-07,6644
8,2023-08,7851
9,2023-09,7723


In [407]:
# （3）百融-云商分
sql1='''-- 风控-百融贷前-云商分
-- 101331	101331	101331
SELECT trace_id, query_rs, query_time
from db_credit.risk_bairong_scoreysstd
;
'''
br_ys_df =query(sql1)
br_ys_df['br_ys_score'] = br_ys_df['query_rs'].apply(lambda x: json.loads(x).get('Score', {}).get('scoreysstd', None))
br_ys_df['br_ys_score'] = br_ys_df['br_ys_score'].astype(int)
br_ys_df['search_time'] = br_ys_df['query_time'].dt.strftime('%Y-%m')

br_ys_df.shape
br_ys_df.drop_duplicates(subset = ['trace_id']).shape
br_ys_df[br_ys_df.br_ys_score.isna()].shape
br_ys_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(202915, 5)

(202915, 5)

(0, 5)

,search_time,trace_id
0,2023-04,312
1,2023-05,4685
2,2023-06,5045
3,2023-07,6708
4,2023-08,7853
5,2023-09,7489
6,2023-10,8667
7,2023-11,11640
8,2023-12,7668
9,2024-01,7268


In [ ]:
# （4）百融-大表
# db_credit.risk_bairong_scoreprime：为单独拆出来的小表，忽略
# db_credit.risk_bairong：为百融的大表，含信息验证接口和贷前查询接口，先查信息验证，再查贷前查询；若没有拆出来的产品，都存这张表（如云商分单独存表）
# 以下在同一个接口：
# 反欺诈规则-法院被执行人高级版
# 反欺诈规则-法院被执行人限高版
# 反欺诈规则-特殊名单验证

# 反欺诈规则-借贷风险勘测
# 反欺诈规则-借贷意向验证-线上现金分期
# 反欺诈规则-申请意向-3C租赁

# 融安分A-主流-标准款
# 以下在同一个接口：三要素验证、在网时长、在网状态

# sql1='''-- 风控-百融 
# SELECT trace_id, query_rs, query_time
# from db_credit.risk_bairong_scoreprime
# ;
# '''

# sql1='''-- 风控-百融 
# SELECT trace_id, query_rs, query_time 
#       , verify_rs, verify_time
# from db_credit.risk_bairong
# ;
# '''

# br_raf_df =query(sql1)
# br_raf_df['ran_score'] = br_raf_df['query_rs'].apply(lambda x: json.loads(x).get('RiskStrategy', {}).get('Score', {}).get('scoreprime', None))  
# br_raf_df['ran_score'] = pd.to_numeric(br_raf_df['ran_score'].replace('"null"', np.nan), errors='coerce')

br_raf_df['ran_score'] = br_raf_df['ran_score'].astype(int)
br_raf_df['search_time'] = br_raf_df['query_time'].dt.strftime('%Y-%m')
br_raf_df['search_time_1'] = br_raf_df['verify_time'].dt.strftime('%Y-%m')

br_raf_df.shape
br_raf_df.drop_duplicates(subset = ['trace_id']).shape
br_raf_df[:2]
# br_raf_df[br_raf_df.ran_score.isna()].shape
br_raf_df[br_raf_df.search_time.notnull()].groupby(['search_time']).agg({'trace_id':'count'}).reset_index()
br_raf_df[br_raf_df.search_time_1.notnull()].groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

# br_raf_df[br_raf_df.ran_score.isna()].shape
# br_raf_df.ran_score.min()
# br_raf_df.ran_score.max()

# br_raf_df[(br_raf_df.ran_score >= 560) & (br_raf_df.ran_score <= 964)].shape
# br_raf_df.shape

(301397, 8)

(301397, 8)

,trace_id,query_rs,query_time,verify_rs,verify_time,ran_score,search_time,search_time_1
0,19935d138af3f087b71d587c0d41693a,"{""Flag"": {""score"": ""1"", ""applyloan_d"": ""0"", ""a...",2022-12-31 15:02:34,"{""Flag"": {""telperiod"": ""1"", ""telstatus"": ""1"", ...",2022-12-31 15:02:34,689,2022-12,2022-12
1,29fc01aa38c3b251410f925c7698edc1,"{""Flag"": {""score"": ""1"", ""applyloan_d"": ""0"", ""a...",2023-01-04 14:49:40,"{""Flag"": {""telperiod"": ""1"", ""telstatus"": ""1"", ...",2023-01-04 14:49:41,769,2023-01,2023-01


,search_time,trace_id
0,2022-12,2
1,2023-01,2534
2,2023-02,4957
3,2023-03,8055
4,2023-04,18821
5,2023-05,17126
6,2023-06,17543
7,2023-07,15151
8,2023-08,11633
9,2023-09,13497


,search_time,trace_id
0,2022-12,2
1,2023-01,2171
2,2023-02,4090
3,2023-03,4286
4,2023-04,6875
5,2023-05,4533
6,2023-06,4937
7,2023-07,6683
8,2023-08,7629
9,2023-09,7493


In [424]:
# （5）百融贷前-申请意向-3C租赁
# sql1='''-- 风控-申请意向-3C租赁
# SELECT trace_id, query_rs, query_time
# from db_credit.risk_bairong_applyloanc
# ;
# '''
# br_3C_df =query(sql1)
# # br_3C_df['br_ys_score'] = br_3C_df['query_rs'].apply(lambda x: json.loads(x).get('Score', {}).get('scoreysstd', None))
# # br_3C_df['br_ys_score'] = br_3C_df['br_ys_score'].astype(int)
# br_3C_df['search_time'] = br_3C_df['query_time'].dt.strftime('%Y-%m')

# br_3C_df.shape
# br_3C_df.drop_duplicates(subset = ['trace_id']).shape
# # br_3C_df[br_3C_df.br_ys_score.isna()].shape
# br_3C_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(565, 4)

(565, 4)

,search_time,trace_id
0,2024-09,132
1,2024-10,103
2,2024-11,116
3,2024-12,148
4,2025-01,66


In [426]:
# （6）百融贷前-核批
sql1='''-- 百融贷前-核批
SELECT trace_id, query_rs, query_time
from db_credit.risk_bairong_evaluate
;
'''
br_hp_df =query(sql1)
# br_3C_df['br_ys_score'] = br_3C_df['query_rs'].apply(lambda x: json.loads(x).get('Score', {}).get('scoreysstd', None))
# br_3C_df['br_ys_score'] = br_3C_df['br_ys_score'].astype(int)
br_hp_df['search_time'] = br_hp_df['query_time'].dt.strftime('%Y-%m')

br_hp_df.shape
br_hp_df.drop_duplicates(subset = ['trace_id']).shape
# br_3C_df[br_3C_df.br_ys_score.isna()].shape
br_hp_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(61736, 4)

(61736, 4)

,search_time,trace_id
0,2023-05,442
1,2023-06,5022
2,2023-07,6699
3,2023-08,7749
4,2023-09,7489
5,2023-10,8667
6,2023-11,11640
7,2023-12,7668
8,2024-01,6359
9,2024-08,1


In [427]:
# （7）百融贷前-法院文书
sql1='''-- 百融贷前-法院文书
SELECT trace_id, query_rs, query_time
from db_credit.risk_bairong_executionjud
;
'''
br_executionjud_df =query(sql1)
# br_3C_df['br_ys_score'] = br_3C_df['query_rs'].apply(lambda x: json.loads(x).get('Score', {}).get('scoreysstd', None))
# br_3C_df['br_ys_score'] = br_3C_df['br_ys_score'].astype(int)
br_executionjud_df['search_time'] = br_executionjud_df['query_time'].dt.strftime('%Y-%m')

br_executionjud_df.shape
br_executionjud_df.drop_duplicates(subset = ['trace_id']).shape
# br_3C_df[br_3C_df.br_ys_score.isna()].shape
br_executionjud_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(49519, 4)

(49519, 4)

,search_time,trace_id
0,2024-01,119
1,2024-02,493
2,2024-03,1802
3,2024-04,3681
4,2024-05,3874
5,2024-06,5328
6,2024-07,5782
7,2024-08,5029
8,2024-09,5564
9,2024-10,4096


In [ ]:
# （8）极信-fico分
sql1='''-- 风控-极信-分数sc32007   fico  
SELECT trace_id, rs_decode, time
from db_credit.risk_jixin_score_sc32007
;
'''
jx_fico_df =query(sql1)
jx_fico_df['jx_fico_score'] = jx_fico_df['rs_decode'].apply(lambda x: json.loads(x).get('data', {}).get('resultData', {}).get('score', None))  
jx_fico_df['jx_fico_score'] = pd.to_numeric(jx_fico_df['jx_fico_score'].replace('"null"', np.nan), errors='coerce')

jx_fico_df['jx_fico_score'] = jx_fico_df['jx_fico_score'].astype(int)
jx_fico_df['search_time'] = jx_fico_df['time'].dt.strftime('%Y-%m')

jx_fico_df.shape
jx_fico_df.drop_duplicates(subset = ['trace_id']).shape
jx_fico_df[jx_fico_df.jx_fico_score.isna()].shape
jx_fico_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(58574, 5)

(58574, 5)

(0, 5)

,search_time,trace_id
0,2024-02,429
1,2024-03,1784
2,2024-04,3671
3,2024-05,3864
4,2024-06,5301
5,2024-07,5764
6,2024-08,6259
7,2024-09,7572
8,2024-10,5641
9,2024-11,6359


In [428]:
# （9）极信-TD212
sql1='''
SELECT trace_id, rs_decode, time
from db_credit.risk_jixin_score_jx1000212
;
'''
jx_td_df = query(sql1)
# jx_fico_df['jx_fico_score'] = jx_fico_df['rs_decode'].apply(lambda x: json.loads(x).get('data', {}).get('resultData', {}).get('score', None))  
# jx_fico_df['jx_fico_score'] = pd.to_numeric(jx_fico_df['jx_fico_score'].replace('"null"', np.nan), errors='coerce')

# jx_fico_df['jx_fico_score'] = jx_fico_df['jx_fico_score'].astype(int)
jx_td_df['search_time'] = jx_td_df['time'].dt.strftime('%Y-%m')

jx_td_df.shape
jx_td_df.drop_duplicates(subset = ['trace_id']).shape
# jx_td_df[jx_fico_df.jx_fico_score.isna()].shape
jx_td_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(20831, 4)

(20831, 4)

,search_time,trace_id
0,2024-09,2992
1,2024-10,4097
2,2024-11,4601
3,2024-12,6380
4,2025-01,2761


In [429]:
# （10）极信-银联/交易
sql1='''
SELECT trace_id, rs_decode, time
from db_credit.risk_jixin
;
'''
jx_yl_df = query(sql1)
jx_yl_df['search_time'] = jx_yl_df['time'].dt.strftime('%Y-%m')

jx_yl_df.shape
jx_yl_df.drop_duplicates(subset = ['trace_id']).shape
jx_yl_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(153716, 4)

(153716, 4)

,search_time,trace_id
0,2023-11,8530
1,2023-12,7562
2,2024-01,7191
3,2024-02,2567
4,2024-03,5480
5,2024-04,8488
6,2024-05,7993
7,2024-06,9701
8,2024-07,11437
9,2024-08,12443


In [ ]:
# （11）芝麻租物-非免押--预授权

In [13]:
# （12）芝麻租物-风控专业版
sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT *
from db_credit.risk_alipay_auth_query
;
'''

zm_fk_df = query(sql1)
zm_fk_df['search_time'] = zm_fk_df['time'].dt.strftime('%Y-%m')
zm_fk_df.shape
zm_fk_df.drop_duplicates(subset = ['trace_id']).shape
zm_fk_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(193745, 10)

(193745, 10)

,search_time,trace_id
0,2023-05,1293
1,2023-06,4582
2,2023-07,6350
3,2023-08,7364
4,2023-09,7420
5,2023-10,8577
6,2023-11,11554
7,2023-12,7632
8,2024-01,7201
9,2024-02,2603


In [208]:
# （13）芝麻租物-风控进阶版
# 细化综合风险等级 rentOnlineDetailLevelOverall
sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT *
from db_credit.risk_alipay_auth_query
;
'''

zm_fk_df = query(sql1)
zm_fk_df['search_time'] = zm_fk_df['time'].dt.strftime('%Y-%m')
zm_fk_df.shape
# zm_fk_df[:1]
zm_fk_df.drop_duplicates(subset = ['trace_id']).shape
# zm_fk_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()
pd.pivot_table(zm_fk_df,index='search_time',columns='rentOnlineDetailLevelOverall',values='out_order_no',aggfunc='count')

(198384, 10)

(198384, 10)

rentOnlineDetailLevelOverall,-2,0,1,2,3,4,5,6,DEFAULT
search_time,,,,,,,,,
2024-09,NaN,128.0,199.0,190.0,197.0,167.0,3.0,3.0,NaN
2024-10,2.0,115.0,195.0,199.0,188.0,181.0,2.0,2.0,NaN
2024-11,2.0,720.0,1080.0,1080.0,958.0,792.0,10.0,17.0,2.0
2024-12,NaN,909.0,1456.0,1385.0,1327.0,1950.0,18.0,17.0,NaN
2025-01,1.0,1942.0,3009.0,2752.0,2562.0,3999.0,57.0,56.0,5.0


In [6]:
# （14）蚂蚁链-租赁风险分
sql1='''-- 风控-蚂蚁链租赁风险分
-- 122905	122905	122905
SELECT *
from db_credit.risk_antchain
;
'''

my_zl_df=query(sql1)
my_zl_df['search_time'] = my_zl_df['time'].dt.strftime('%Y-%m')

my_zl_df.shape
my_zl_df.drop_duplicates(subset = ['trace_id']).shape
my_zl_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(228481, 7)

(228481, 7)

,search_time,trace_id
0,2022-12,2
1,2023-01,2170
2,2023-02,4089
3,2023-03,4183
4,2023-04,6482
5,2023-05,4526
6,2023-06,5021
7,2023-07,6829
8,2023-08,8095
9,2023-09,7967


In [7]:
# （15）蚂蚁链-履约保护
sql1='''
-- 122905	122905	122905
SELECT *
from db_credit.risk_antchain_risk_general
;
'''

my_zl_general_df=query(sql1)
my_zl_general_df['search_time'] = my_zl_general_df['time'].dt.strftime('%Y-%m')

my_zl_general_df.shape
my_zl_general_df.drop_duplicates(subset = ['trace_id']).shape
my_zl_general_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(25734, 7)

(25734, 7)

,search_time,trace_id
0,2024-08,2140
1,2024-09,5564
2,2024-10,4097
3,2024-11,4601
4,2024-12,6390
5,2025-01,2942


In [11]:
# （16）芝麻数控-交互安全风控
# （3）芝麻数控-交互安全风控
# 目前使用到 risk_t0, risk_t1、risk_t3、risk_t7 没有使用
# zm_jh_t0_level 蚂蚁数控风险等级 
sql1='''-- 芝麻数控-交互安全风控
select *
from db_credit.risk_alipay_interactive_prod
;
'''

zm_jh_df=query(sql1)
zm_jh_df[:1]
zm_jh_df['search_time'] = zm_jh_df['time_t0'].dt.strftime('%Y-%m')

zm_jh_df.shape
zm_jh_df.drop_duplicates(subset = ['order_id']).shape
zm_jh_df.groupby(['search_time']).agg({'order_id':'count'}).reset_index()

,id,order_id,auth_time,user_id,params,risk_t0,time_t0,risk_t1,time_t1,risk_t3,time_t3,risk_t7,time_t7
0,4,57850,1692085507,2088912958775018,"{""biz_id"":""14acbc35e0b5f41d16c45a85fbc82dbf"",""...","{""code"":""riskLvl_11"",""level"":989,""content"":""ri...",2023-08-16 15:38:40,"{""code"":""riskLvl_4"",""level"":996,""content"":""ris...",2023-08-17 14:31:55,"{""code"":""riskLvl_2"",""level"":998,""content"":""ris...",2023-08-17 13:43:27,"{""code"":""riskLvl_4"",""level"":996,""content"":""ris...",2023-08-21 00:00:02


(58974, 14)

(58974, 14)

,search_time,order_id
0,2023-08,2061
1,2023-09,895
2,2023-10,1054
3,2023-11,2821
4,2023-12,1864
5,2024-01,2233
6,2024-02,1085
7,2024-03,1592
8,2024-04,2945
9,2024-05,3049


In [4]:
# （17）微聚未来-全景雷达
sql1='''-- 风控-新颜-全景雷达
SELECT trace_id, time
from db_credit.risk_xinyan_qjld
;
'''

xy_qj_df =query(sql1)
xy_qj_df.shape

sql1='''-- 风控-微聚未来-全景雷达
SELECT trace_id, time 
from db_credit.risk_weisuper_qjld
;
'''

xy_risk_weisuper_df =query(sql1)
xy_risk_weisuper_df.shape


df_qjld = pd.concat([xy_qj_df, xy_risk_weisuper_df])
df_qjld['search_time'] = df_qjld['time'].dt.strftime('%Y-%m')

df_qjld.shape
df_qjld.drop_duplicates(subset = ['trace_id']).shape
df_qjld.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(44380, 2)

(63801, 2)

(108181, 3)

(108142, 3)

,search_time,trace_id
0,2023-11,1552
1,2023-12,1194
2,2024-01,745
3,2024-02,492
4,2024-03,1542
5,2024-04,3668
6,2024-05,3858
7,2024-06,5300
8,2024-07,6047
9,2024-08,9027


In [ ]:
# （18）E签宝-身份核验查询

In [ ]:
# （19）E签宝-区块链合同

In [14]:
# （20）极光-风险分22
sql1='''-- 风控-极光-风险分22
SELECT * 
from db_credit.risk_jiguang_score22
;
'''

jg_fx_df =query(sql1)
jg_fx_df['search_time'] = jg_fx_df['time'].dt.strftime('%Y-%m')

jg_fx_df.shape
jg_fx_df.drop_duplicates(subset = ['trace_id']).shape
jg_fx_df.groupby(['search_time']).agg({'trace_id':'count'}).reset_index()

(157971, 7)

(157971, 7)

,search_time,trace_id
0,2023-10,1468
1,2023-11,11339
2,2023-12,7513
3,2024-01,7128
4,2024-02,2537
5,2024-03,5494
6,2024-04,8432
7,2024-05,7897
8,2024-06,9640
9,2024-07,11346


# 15_优优客群分析

# 16_灯火渠道的搜索客群

In [84]:
# （1）导入数据
# 鑫仲帮忙取灯火渠道的搜索客群
path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'
df_dhss_customer = pd.read_excel(path + '搜索灯火_20250117.xlsx')
df_dhss_customer.shape
df_dhss_customer = df_dhss_customer.rename(columns = {'订单号': 'order_number'})
df_dhss_customer[:1]

(251, 7)

,下单日期,order_number,商品型号,总租金,买断金额,租赁方案,来源渠道
0,2024-04-05,A2024040513503995,iPhone15ProMax 全新国行 顺丰包邮,12736.7,12996.7,租完即送,付费流量-灯火内测


In [85]:
# （2）关联租后表现
# 历史逾期
df_analy_data['y_fraud'] = df_analy_data.apply(lambda x: 1 if x.第二期逾期天数 >= 7 
                                       else 0.5 if x.第二期逾期天数 > 0 and x.第二期逾期天数 < 7
                                       else 0, axis = 1)
df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.历史最大逾期天数 >= 31 
                                       else 0.5 if x.历史最大逾期天数 > 0 and x.历史最大逾期天数 < 31
                                       else 0, axis = 1)


df_dhss_customer = df_dhss_customer.merge(df_analy_data[['order_number','search_time','y_fraud','y_credit','overdue_type']], on = 'order_number', how = 'left')
# 当前逾期
# sql = '''-- 租后应收监控  
# SELECT  order_number,overdue_type
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# '''
# df_zhys = query(sql)

# df_dhss_customer = df_dhss_customer.merge(df_zhys, on = 'order_number', how = 'left')
df_dhss_customer.shape
df_dhss_customer[:1]

(251, 11)

,下单日期,order_number,商品型号,总租金,买断金额,租赁方案,来源渠道,search_time,y_fraud,y_credit,overdue_type
0,2024-04-05,A2024040513503995,iPhone15ProMax 全新国行 顺丰包邮,12736.7,12996.7,租完即送,付费流量-灯火内测,2024-04,1.0,1.0,M6


In [86]:
# （3）租后表现对比分析
pd.pivot_table(df_dhss_customer,index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(df_dhss_customer,index='search_time',columns='y_credit',values='order_number',aggfunc='count')
pd.pivot_table(df_dhss_customer,index='search_time',columns='overdue_type',values='order_number',aggfunc='count')

pd.pivot_table(df_analy_data[df_analy_data.归属渠道 == '搜索渠道'],index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_data[df_analy_data.归属渠道 == '搜索渠道'],index='search_time',columns='y_credit',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_data[df_analy_data.归属渠道 == '搜索渠道'],index='search_time',columns='overdue_type',values='order_number',aggfunc='count')

y_fraud,0.0,0.5,1.0
search_time,,,
2024-04,18.0,1.0,1.0
2024-05,1.0,NaN,NaN
2024-07,26.0,NaN,2.0
2024-08,40.0,NaN,3.0
2024-09,61.0,3.0,2.0
2024-10,16.0,1.0,1.0
2024-11,12.0,1.0,1.0
2024-12,17.0,2.0,1.0


y_credit,0.0,0.5,1.0
search_time,,,
2024-04,9.0,3.0,8.0
2024-05,1.0,NaN,NaN
2024-07,19.0,3.0,6.0
2024-08,32.0,5.0,6.0
2024-09,48.0,13.0,5.0
2024-10,12.0,6.0,NaN
2024-11,12.0,2.0,NaN
2024-12,17.0,3.0,NaN


overdue_type,,M0,M1,M2,M3,M6
search_time,,,,,,
2024-04,12.0,NaN,NaN,1.0,3.0,4.0
2024-05,1.0,NaN,NaN,NaN,NaN,NaN
2024-07,22.0,1.0,1.0,3.0,1.0,NaN
2024-08,36.0,1.0,3.0,NaN,3.0,NaN
2024-09,59.0,2.0,2.0,2.0,1.0,NaN
2024-10,15.0,3.0,NaN,NaN,NaN,NaN
2024-11,14.0,NaN,NaN,NaN,NaN,NaN
2024-12,18.0,2.0,NaN,NaN,NaN,NaN


y_fraud,0.0,0.5,1.0
search_time,,,
2022-07,6.0,NaN,1.0
2022-08,14.0,5.0,3.0
2022-09,3.0,1.0,4.0
2022-10,5.0,NaN,1.0
2022-11,5.0,1.0,4.0
2022-12,30.0,1.0,6.0
2023-01,98.0,24.0,12.0
2023-02,148.0,25.0,27.0
2023-03,125.0,5.0,20.0


y_credit,0.0,0.5,1.0
search_time,,,
2022-07,NaN,4.0,3.0
2022-08,1.0,10.0,11.0
2022-09,1.0,2.0,5.0
2022-10,1.0,2.0,3.0
2022-11,2.0,2.0,6.0
2022-12,6.0,16.0,15.0
2023-01,25.0,55.0,54.0
2023-02,52.0,64.0,84.0
2023-03,41.0,39.0,70.0


overdue_type,,M0,M1,M2,M3,M6
search_time,,,,,,
2022-07,3.0,NaN,NaN,NaN,NaN,4.0
2022-08,9.0,NaN,NaN,NaN,NaN,13.0
2022-09,2.0,NaN,NaN,NaN,NaN,6.0
2022-10,5.0,NaN,NaN,NaN,NaN,1.0
2022-11,4.0,NaN,NaN,NaN,NaN,6.0
2022-12,24.0,NaN,NaN,NaN,NaN,13.0
2023-01,75.0,NaN,NaN,1.0,NaN,58.0
2023-02,123.0,NaN,NaN,NaN,NaN,77.0
2023-03,78.0,NaN,NaN,NaN,NaN,72.0


# 17_拒量回捞分析

<!-- 2025-01-17 拒量放量，从综合规则--自有模型强拒 回捞 -->

## 17_1.20250117

In [ ]:
# （1）取数-1
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status,om.order_method
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tprm.purchase_amount, tprm.all_rental, tprm.order_number order_number2
,om.buy_service_product,tso.status as service_status 
, tprm.contract_price, tor.update_time, tomt.reason
from db_digua_business.t_order  om 
LEFT JOIN db_digua_business.t_postlease_receivables_monitoring tprm on om.id=tprm.order_id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
;
'''
df = query(sql1)
df.shape

In [195]:
# （2）取数-2
sql2 = '''
SELECT 
id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status  
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.rejected') end,'"','') as rejected  
, mobile
FROM `db_credit`.`risk`
'''
df_risk = query(sql2)
df_risk.shape

(244673, 5)

In [196]:
# （3）数据处理
# （3-1）信息处理
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'] = np.where(df['拒绝理由']=='', df['total_describes'].str.split('原因：').str[-1].str.split('"').str[0], df['rejected'])

# （3-2）判断进件
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")

# （3-3）日期处理
df['下单日期'] = df['create_time'].dt.date
df["下单日期"] = pd.to_datetime(df["下单日期"],errors="coerce")

df.shape

(2236404, 38)

In [197]:
# （4）订单预排序去重 剔除内部订单
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去除刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除订单状态空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
# note 20241209 删除此条件
# df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

In [198]:
# （5）剔除商家数据--只保留自营租机业务数据
df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)

df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="线下小店"].index,inplace=True)

df.drop(df[df['merchant_name'].str.contains(pat='探路者',regex=False)==True].index,inplace=True) 
df.drop(df[df['merchant_name'] == "乙辉数码"].index, inplace=True)

In [199]:
# （6）定义审核状态
# 处理原因
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))|(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)

df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)

In [200]:
# （7）添加小蚂蚁租机被拒理由
df2 = df.copy()
df2.drop(df2[df2['merchant_name']=="小蚂蚁租机"].index,inplace=True)
# 匹配小蚂蚁数据
df_xmy = df[df['merchant_name'] == '小蚂蚁租机']
# 机审强拒数据重命名
df_risk.rename(columns={'time': 'time_risk', 'status': 'status_risk', 'rejected': 'rejected_risk'}, inplace=True)
df_risk['拒绝理由_risk'] = df_risk["rejected_risk"].str.replace("[","").str.replace("]","").str.replace('"','')
# 对小蚂蚁数据和机审强拒数据进行拼接
df_risk_all = pd.merge(df_xmy, df_risk, left_on='id_card_num', right_on='id_card', how='inner')
# 计算出库前强拒的订单数
df_risk_all2 = df_risk_all[(df_risk_all['time_risk'] < df_risk_all['update_time'])]
# 进行排序并取到最近的一个订单
df_risk_all2 = df_risk_all2.sort_values(['order_id', 'time_risk'], ascending=[True, False]).groupby('order_id').head(1)
df_risk_all2 = df_risk_all2[df_risk_all2['status_risk'] == '1']
# 拼接小蚂蚁数据获取全新的拒绝理由
df_risk_new = df.merge(df_risk_all2[['order_number', '拒绝理由_risk']], on='order_number', how='left')
df_risk_new['拒绝理由'] = np.where((df_risk_new['拒绝理由'].isna()) & df_risk_new['拒绝理由_risk'].notna(), df_risk_new['拒绝理由_risk'], df_risk_new['拒绝理由'])

In [201]:
# （8）数据筛选
#进件的数据
df_jj = df_risk_new[df_risk_new['进件']=='进件']
# df_reject = df_jj[(df_jj['机审强拒']==1)|(df_jj['出库前风控强拒']==1)][['create_time', 'order_id', 'order_number', '拒绝理由', 'total_describes', 'result']]
df_reject = df_jj[['create_time', 'order_id', 'order_number', '拒绝理由', 'total_describes', 'result','trace_id']]
df_reject_risk = risk.model_reject(df_reject, 'order_number')
df_reject_risk_merge = df_reject[['order_number','order_id','trace_id','拒绝理由']].merge(df_reject_risk, on='order_number', how='left')


# df_reject_before：前置命中的数据
df_reject_before = df[((df['status2']=='订单取消')&(df['拒绝理由'].notna()))|((df['status2']=='订单取消')&(df['total_describes'].notna()))]
df_reject_before['拒绝理由'] = np.where(df_reject_before['拒绝理由'].isna(), np.where(df_reject_before['total_describes'].str.contains('拒绝理由：'),df_reject_before['total_describes'].str.split('拒绝理由：').str[-1].str.split('"').str[0], df_reject_before['拒绝理由'])  
                                        ,df_reject_before['拒绝理由'])
df_reject_before = df_reject_before[['create_time', 'order_id', 'order_number', 'status2', '拒绝理由', 'total_describes', 'result','trace_id']]

df_reject_before_risk = risk.pre_reject(df_reject_before, 'order_number')
df_reject_before_risk_merge = df_reject_before[['order_number','order_id','拒绝理由','trace_id']].merge(df_reject_before_risk, on='order_number', how='right')


# df_rule_data：前置被拒绝的和进件的样本数据
df_rule_data = df_reject_risk_merge.merge(df_reject_before_risk_merge, on = ['order_number','order_id','trace_id'], how = 'outer')
df_rule_data = df_rule_data.rename(columns = {'拒绝理由_x':'机审拒绝','拒绝理由_y':'前置拒绝'})

df_reject_risk_merge.shape
df_reject_before_risk_merge.shape
df_rule_data.shape
df_rule_data[:1]

(260815, 31)

(554481, 16)

(815296, 44)

,order_number,order_id,trace_id,机审拒绝,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人_x,命中刑事案件_x,命中借贷纠纷_x,命中融安分低于680_x,命中高风险名单_x,命中风险勘测,蚁盾分>=80,命中30天多头>9并且90天多头>30,命中30天多头>13并且90天多头>39,履约历史等级1且为搜索渠道,云商分低于496,评分等级低于D,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,命中拒绝_通过加强包240801强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒,命中抖音_240829策略强拒,240927策略强拒,前置拒绝,年龄超过49岁或低于18岁,命中法院失信/限高被执行人_y,命中刑事案件_y,命中借贷纠纷_y,命中融安分低于680_y,命中高风险名单_y,身份证命中黑名单,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏
0,A202206131732031,8747,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
# （9）取非 仅自有模型强拒的  拒量进件
df_rule_data_1 = df_rule_data[df_rule_data.前置拒绝.isnull()]
df_rule_data_1.shape
df_rule_data_1 = df_rule_data[df_rule_data.机审拒绝.notnull()]
df_rule_data_1.shape
df_rule_data_1 = df_rule_data_1[~(df_rule_data_1['机审拒绝'] == '[命中自有模型强拒]')]
df_rule_data_1.shape
df_rule_data_1 = df_rule_data_1[~(df_rule_data_1['机审拒绝'] == '命中自有模型强拒')]
df_rule_data_1.shape
df_rule_data_1 = df_rule_data_1[~(df_rule_data_1['机审拒绝'] == '[')]
df_rule_data_1 = df_rule_data_1[['order_number' ,'order_id', '机审拒绝']]
df_rule_data_1.shape
df_rule_data_1[:2]

# df_rule_data = df_rule_data[df_rule_data['机审拒绝'].isin(['[命中自有模型强拒]','命中自有模型强拒'])][['order_number' ,'order_id']]
# df_rule_data.shape
# df_rule_data[:2]
# df_rule_data_1.to_excel('tmp_1.xlsx', index = False)

(260815, 44)

(248289, 44)

(228252, 44)

(227169, 44)

(126145, 3)

,order_number,order_id,机审拒绝
7279,A202301090005022,88929,"[命中30天多头>9并且90天多头>30,判定强拒, 天狼分<320命中强拒, 命中布尔一个..."
7280,A2023010900415317,88944,"[评分等级低于D,命中拒绝]"


In [26]:
# （10）自有模型取值
# （10-1）trace关联
sql1='''-- trace关联表 
SELECT order_trace_id, risk_trace_id, trace_id, order_id
from db_credit.order_association
;
'''
trace_df = query(sql1)
trace_df.shape
trace_df[:2]

# （10-2）区别用哪个模型
# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
sql1='''-- risk关联表 
SELECT trace_id, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.is_model') end,'"','') as is_model
from db_credit.risk
;
'''
risk_df = query(sql1)
risk_df.shape
risk_df[:2]

# （10-3）自有模型 2311
sql1='''-- 风控-自有模型-ascore2311 
SELECT trace_id, score
from db_credit.risk_private_model
;
'''
mx_2311_df =query(sql1)
mx_2311_df['model_score_2311'] = pd.to_numeric(mx_2311_df['score'], errors='coerce')
mx_2311_df.shape

# （10-4）自有模型 2401
sql1='''-- 风控-自有模型-2401
SELECT trace_id,  score
from db_credit.risk_private_model_2401
;
'''
mx_2401_df =query(sql1)
mx_2401_df['model_score_2401'] = pd.to_numeric(mx_2401_df['score'], errors='coerce')
mx_2401_df.shape

# （10-5）自有模型-银联
sql1='''-- 风控-自有模型-yl
SELECT trace_id,  score
from db_credit.risk_private_model_yl
;
'''
mx_yl_df =query(sql1)
mx_yl_df['model_score_yl'] = pd.to_numeric(mx_yl_df['score'], errors='coerce')
mx_yl_df.shape

# （10-6）表关联-x
# 模型之间表关联
df_model = mx_2311_df[['trace_id', 'model_score_2311']].merge(mx_2401_df[['trace_id', 'model_score_2401']], on = 'trace_id', how='outer')
df_model.shape

df_model = df_model.merge(mx_yl_df[['trace_id', 'model_score_yl']], on = 'trace_id', how='outer')
df_model.shape

# df_model[df_model.trace_id.isnull()].shape
df_model_data = trace_df.merge(df_model, left_on='order_trace_id', right_on='trace_id', how='outer')
# df_model_data['trace_id_x'] = np.where(df_model_data['trace_id_y'].isna(), df_model_data['trace_id_x'], df_model_data['trace_id_y'])
df_model_data.drop('trace_id_y', axis=1, inplace=True)
df_model_data.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
df_model_data.shape
# df_model_data[:2]

# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
df_model_data = df_model_data.merge(risk_df, left_on='order_trace_id', right_on='trace_id', how='left')
df_model_data.drop('trace_id_y', axis=1, inplace=True)
df_model_data.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
df_model_data.shape
df_model_data[:1]

(1612951, 4)

,order_trace_id,risk_trace_id,trace_id,order_id
0,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,88929
1,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,88931


(242522, 2)

,trace_id,is_model
0,91535f1aafe335dbddfa46a1317dd939,None
1,769ce80d7b55eb8f4e8d64cb3a9e9cef,None


(172587, 3)

...

(172587, 3)

(172587, 4)

(1623654, 7)

(1623654, 8)

,order_trace_id,risk_trace_id,trace_id,order_id,model_score_2311,model_score_2401,model_score_yl,is_model
0,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,88929.0,NaN,NaN,NaN,None


In [77]:
# （11）宽表
# 取基本信息
sql = '''
SELECT  om.order_number, om.create_time
from db_digua_business.t_order om
# where date_format(om.create_time, '%Y-%m-%d') >= '2024-10-01'
;
'''

df_zhys = query(sql)

df_zhys.shape
df_zhys['下单日期'] = df_zhys['create_time'].dt.date
df_zhys["下单日期"] = pd.to_datetime(df_zhys["下单日期"],errors="coerce")
df_zhys['search_time'] = pd.to_datetime(df_zhys["下单日期"]).dt.strftime('%Y-%m')


# 宽表
df_reject_model = df_rule_data_1.merge(df_model_data, on = 'order_id', how = 'left')
df_reject_model = df_reject_model.merge(df_zhys[['order_number','search_time','下单日期']], on = 'order_number', how = 'left')
df_reject_model[df_reject_model.search_time.isnull()].shape
df_reject_model.shape

df_reject_model = df_reject_model[(df_reject_model.下单日期 >= '2024-10-01') & (df_reject_model.下单日期 <= '2025-01-19')]
df_reject_model.shape
df_reject_model[:1]
# 20741 - 20699  inner
del df_zhys

(1980512, 2)

(0, 12)

(124890, 12)

(40756, 12)

,order_number,order_id,机审拒绝,order_trace_id,risk_trace_id,trace_id,model_score_2311,model_score_2401,model_score_yl,is_model,search_time,下单日期
84013,A202410010003455,1545863,"[命中被执行人或曝光台, 命中策略240703_14强拒, 命中自有模型强拒]",f84699819dd7dd11a9441e5fd9b16380,f84699819dd7dd11a9441e5fd9b16380,768ce83947b3f2d966e1ad5b0c3f6150,21.949046,17.459331,30.632389,2,2024-10,2024-10-01


In [ ]:
# （12）数据测算
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401

conditons_1 = (df_reject_model.is_model == '1') & (df_reject_model.model_score_2311 <= 15.2211)
conditons_2 = (df_reject_model.is_model == '2') & (df_reject_model.model_score_yl <= 14.9787)
conditons_3 = (df_reject_model.is_model == '3') & (df_reject_model.model_score_2401 <= 8.1386)


pd.pivot_table(df_reject_model[conditons_1],index='下单日期',columns='is_model',values='order_number',aggfunc='count')
pd.pivot_table(df_reject_model[conditons_2],index='下单日期',columns='is_model',values='order_number',aggfunc='count')
pd.pivot_table(df_reject_model[conditons_3],index='下单日期',columns='is_model',values='order_number',aggfunc='count')

# df_reject_model[df_reject_model.is_model == '1'].model_score_2311.min()
# df_reject_model[df_reject_model.is_model == '2'].model_score_yl.min()
# df_reject_model[df_reject_model.is_model == '3'].model_score_2401.min()

# df_reject_model.to_excel('tmp.xlsx', index = False)
# pd.pivot_table(df_reject_model[conditons_1],index='下单日期',columns='is_model',values='order_number',aggfunc='count')

## 17_2.20250124

In [205]:
# （1）回捞策略中20241205的1的也放开
# df_rule_data：前置被拒绝的和进件的样本数据
df_rule_data.shape
df_rule_data[:1]

df_rule_data[df_rule_data.机审拒绝.isnull()].shape
df_rule_data[(df_rule_data.前置拒绝.isnull()) & (df_rule_data.机审拒绝.notnull())].shape
df_rule_data[(df_rule_data.前置拒绝.isnull()) & (df_rule_data.机审拒绝.isnull()) & (df_rule_data.机审拒绝.isnull())].shape


(815296, 44)

,order_number,order_id,trace_id,机审拒绝,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人_x,命中刑事案件_x,命中借贷纠纷_x,命中融安分低于680_x,命中高风险名单_x,命中风险勘测,蚁盾分>=80,命中30天多头>9并且90天多头>30,命中30天多头>13并且90天多头>39,履约历史等级1且为搜索渠道,云商分低于496,评分等级低于D,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,命中拒绝_通过加强包240801强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒,命中抖音_240829策略强拒,240927策略强拒,前置拒绝,年龄超过49岁或低于18岁,命中法院失信/限高被执行人_y,命中刑事案件_y,命中借贷纠纷_y,命中融安分低于680_y,命中高风险名单_y,身份证命中黑名单,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏
0,A202206131732031,8747,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(567007, 44)

## 17_3.2025.2

In [139]:
# （1）清单导入
path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'
# rr_data = pd.read_excel(path + '拒量数据_20250210.xlsx')
# rr_data = pd.read_excel(path + '拒量数据_20250217.xlsx')
rr_data = pd.read_excel(path + '拒量数据_20250220.xlsx')
rr_data['search_time'] = rr_data['下单日期'].dt.strftime('%Y-%m')
rr_data['策略命中等级_new'] = rr_data.apply(lambda x: '1.等级1' if x.策略命中等级 == '策略241205命中(1)'
                                      else '2.等级2' if x.策略命中等级 == '策略241205命中(2)'
                                      else '3.等级3' if x.策略命中等级 == '策略241205命中(3)'
                                      else '4.等级4/5' if x.策略命中等级 == '策略241205命中(4)' or x.策略命中等级 == '策略241205命中(5)' or x.策略命中等级 == '策略241205命中(6)'
                                      else '5.策略241212' if x.策略命中等级 == '策略241212'
                                      else '6.策略250120' if x.策略命中等级 == '命中自有模型回捞策略'
                                      else 'other', axis = 1)
rr_data.shape
rr_data.drop_duplicates(subset = ['order_number']).shape
rr_data[:2]

(2795, 6)

(2795, 6)

,下单日期,order_number,策略命中等级,是否出库,search_time,策略命中等级_new
0,2024-12-10,A2024121011004511,策略241205命中(2),0,2024-12,2.等级2
1,2024-12-10,A20241210112120198,策略241205命中(2),0,2024-12,2.等级2


In [140]:
# （2）关联历史逾期
var = ['order_number','overdue_type','FPD','历史最大逾期天数']
rr_data = rr_data.merge(df_analy_data[var], on = 'order_number', how = 'left')
rr_data[:1]
rr_data.shape
# # rr_data[rr_data.历史最大逾期天数 > 0].shape
# # rr_data[rr_data.历史最大逾期天数 > 30].shape

rr_data['tmp1'] = np.where(rr_data.overdue_type.isin(['M0','M1','M2','M3','M6']), 1, 0)
rr_data['tmp2'] = np.where(rr_data.FPD >= 7, 1, 0)
rr_data['tmp3'] = np.where(rr_data.历史最大逾期天数 >= 1, 1, 0)
rr_data['tmp4'] = np.where(rr_data.历史最大逾期天数 > 30, 1, 0)

rr_data['tmp5'] = np.where(rr_data.是否出库 == 1, 1, 0)

rr_data.groupby(['search_time','策略命中等级_new']).agg({'order_number':'count','tmp5':'sum','tmp1':'sum','tmp2':'sum','tmp3':'sum','tmp4':'sum'}).reset_index()
# A2024122615104994

,下单日期,order_number,策略命中等级,是否出库,search_time,策略命中等级_new,overdue_type,FPD,历史最大逾期天数
0,2024-12-10,A2024121011004511,策略241205命中(2),0,2024-12,2.等级2,NaN,NaN,NaN


(2795, 9)

,search_time,策略命中等级_new,order_number,tmp5,tmp1,tmp2,tmp3,tmp4
0,2024-12,2.等级2,515,13,2,2,4,0
1,2024-12,3.等级3,220,2,0,0,0,0
2,2024-12,4.等级4/5,39,12,0,0,1,0
3,2024-12,5.策略241212,12,1,0,0,0,0
4,2025-01,1.等级1,42,2,0,0,1,0
5,2025-01,2.等级2,529,13,0,0,0,0
6,2025-01,3.等级3,197,6,0,0,0,0
7,2025-01,4.等级4/5,35,8,0,0,0,0
8,2025-01,5.策略241212,8,0,0,0,0,0
9,2025-01,6.策略250120,26,2,0,0,0,0


In [72]:
rr_data[rr_data.FPD >= 7].groupby(['search_time','策略命中等级_new']).agg({'order_number':'count'}).reset_index()
# A20250211173300202 首期扣款时间大于首期应付日期，不算逾期
rr_data[rr_data.历史最大逾期天数 >= 1].groupby(['search_time','策略命中等级_new']).agg({'order_number':'count'}).reset_index()
rr_data[rr_data.overdue_type.isin(['M0','M1','M2','M3','M6'])].groupby(['search_time','策略命中等级_new','overdue_type']).agg({'order_number':'count'}).reset_index()

,search_time,策略命中等级_new,order_number
0,2024-12,2.等级2,2


,search_time,策略命中等级_new,order_number
0,2024-12,2.等级2,4
1,2024-12,4.等级4/5,1


,search_time,策略命中等级_new,overdue_type,order_number
0,2024-12,2.等级2,M0,2


# 18_绩效逾期指标计算

## 18_1.20250123

In [270]:
# （1）数据整理
# 第2期逾期超过15天的逾期率（金额维度）
# 第4期逾期超过15天的逾期率（金额维度）


# 当前期数
# 逾期天数
# purchase_amount
# paid_money_mob2：第2期实付金额
# paid_money_mob4：第4期实付金额
# 当前应付租金
# 实付日期new
# 应付日期


df_jx_1 = df_xz_concat[["order_number","当前期数","purchase_amount"]].pivot(index="order_number", columns="当前期数", values="purchase_amount").reset_index()
df_jx_1 = df_jx_1.rename(columns = {1:'purchase_amount'})
df_jx_1 = df_jx_1[['order_number', 'purchase_amount']]

df_jx_2 = df_xz_concat[["order_number","当前期数","paid_money_mob2"]].pivot(index="order_number", columns="当前期数", values="paid_money_mob2").reset_index()
df_jx_2 = df_jx_2.rename(columns = {2:'paid_money_mob2'})
df_jx_2 = df_jx_2[['order_number', 'paid_money_mob2']]

df_jx_3 = df_xz_concat[["order_number","当前期数","paid_money_mob4"]].pivot(index="order_number", columns="当前期数", values="paid_money_mob4").reset_index()
df_jx_3 = df_jx_3.rename(columns = {4:'paid_money_mob4'})
df_jx_3 = df_jx_3[['order_number', 'paid_money_mob4']]

df_jx_4 = df_xz_concat[["order_number","当前期数","当前应付租金"]].pivot(index="order_number", columns="当前期数", values="当前应付租金").reset_index()
df_jx_4 = df_jx_4.rename(columns = {2:'第2期应付租金', 4:'第4期应付租金'})
df_jx_4 = df_jx_4[['order_number', '第2期应付租金', '第4期应付租金']]

df_jx_5 = df_xz_concat[["order_number","当前期数","应付日期"]].pivot(index="order_number", columns="当前期数", values="应付日期").reset_index()
df_jx_5 = df_jx_5.rename(columns = {2:'第2期应付日期', 4:'第4期应付日期'})
df_jx_5 = df_jx_5[['order_number', '第2期应付日期', '第4期应付日期']]

df_jx_6 = df_xz_concat[["order_number","当前期数","实付日期new"]].pivot(index="order_number", columns="当前期数", values="实付日期new").reset_index()
df_jx_6 = df_jx_6.rename(columns = {2:'第2期实付日期new', 4:'第4期实付日期new'})
df_jx_6 = df_jx_6[['order_number', '第2期实付日期new', '第4期实付日期new']]

df_jx_1.shape
df_jx_2.shape
df_jx_3.shape
df_jx_4.shape
df_jx_5.shape
df_jx_6.shape
df_analy_data.shape

df_jx_data = df_analy_data.merge(df_jx_1, on = 'order_number', how = 'inner')
df_jx_data = df_jx_data.merge(df_jx_2, on = 'order_number', how = 'inner')
df_jx_data = df_jx_data.merge(df_jx_3, on = 'order_number', how = 'inner')
df_jx_data = df_jx_data.merge(df_jx_4, on = 'order_number', how = 'inner')
df_jx_data = df_jx_data.merge(df_jx_5, on = 'order_number', how = 'inner')
df_jx_data = df_jx_data.merge(df_jx_6, on = 'order_number', how = 'inner')
df_jx_data.shape
# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]
df_jx_data.drop(columns=['归属渠道','是否二手','是否带锁','历史最大逾期天数','FPD','SPD','TPD','第三期逾期天数','overdue_type'],inplace = True)
# A2024010216293470
# A202401030606401

# 未判断该期是否到期
# df_jx_data['第2期实付应付相差天数'] = np.where(df_jx_data['第2期实付日期new'].notnull(), (df_jx_data['第2期实付日期new'] - df_jx_data['第2期应付日期']).dt.days, 999)
# df_jx_data['第4期实付应付相差天数'] = np.where(df_jx_data['第4期实付日期new'].notnull(), (df_jx_data['第4期实付日期new'] - df_jx_data['第4期应付日期']).dt.days, 999)

df_jx_data[df_jx_data.order_id.isin([508404,1678492,1692916,12449])]

(31687, 2)

(31687, 2)

(31687, 2)

(31687, 3)

(31687, 3)

(31687, 3)

(31689, 16)

(31689, 25)

,order_number,order_id,下单日期,search_time,第二期逾期天数,第四期逾期天数,首期应还日期,purchase_amount,paid_money_mob2,paid_money_mob4,第2期应付租金,第4期应付租金,第2期应付日期,第4期应付日期,第2期实付日期new,第4期实付日期new
28,A202208052103231,12449,2022-08-05,2022-08,0.0,9.0,2022-08-05,5945.0,376.68,376.68,376.68,376.68,2022-09-05,2022-11-05,2022-09-05,2022-11-14
3719,A2023073114301076,508404,2023-07-31,2023-07,0.0,0.0,2023-08-03,6650.0,479.73,479.73,479.73,479.73,2023-09-03,2023-11-03,2023-08-30,2023-10-30
24302,A20241103155713186,1678492,2024-11-03,2024-11,52.0,0.0,2024-11-06,8170.0,0.00,0.00,720.60,720.60,2024-12-06,2025-02-06,NaT,NaT
24566,A20241107103123201,1692916,2024-11-07,2024-11,8.0,0.0,2024-11-10,6670.0,507.34,0.00,507.34,507.34,2024-12-10,2025-02-10,2024-12-18,NaT


In [271]:
# （2）统计
conditions_1 = (df_jx_data.第二期逾期天数 > 15)
conditions_2 = (df_jx_data.第四期逾期天数 > 15)

df_jx_data.shape
df_jx_data = df_jx_data.drop_duplicates(subset = ['order_number'])
df_jx_data.shape
# df_jx_data[:1]
# # 按下单日期
# df_jx_data.groupby(by=['search_time']).agg({'purchase_amount': 'sum'})
# df_jx_data.groupby(by=['search_time']).agg({'第2期应付租金': 'sum'})
# df_jx_data.groupby(by=['search_time']).agg({'第4期应付租金': 'sum'})
# df_jx_data[conditions_1].groupby(by=['search_time']).agg({'第2期应付租金': 'sum'})
# df_jx_data[conditions_2].groupby(by=['search_time']).agg({'第4期应付租金': 'sum'})

# 按首期应还日期
df_jx_data['search_time_new'] = df_jx_data['首期应还日期'].dt.strftime('%Y-%m')
tmp_1 = df_jx_data.groupby(by=['search_time_new']).agg({'purchase_amount': 'sum'}).reset_index()
tmp_2 = df_jx_data.groupby(by=['search_time_new']).agg({'第2期应付租金': 'sum'}).reset_index()
tmp_3 = df_jx_data.groupby(by=['search_time_new']).agg({'第4期应付租金': 'sum'}).reset_index()
tmp_4 = df_jx_data[conditions_1].groupby(by=['search_time_new']).agg({'第2期应付租金': 'sum'}).reset_index()
tmp_4 = tmp_4.rename(columns = {'第2期应付租金': '第2期DPD>15的应付租金'})
tmp_5 = df_jx_data[conditions_2].groupby(by=['search_time_new']).agg({'第4期应付租金': 'sum'}).reset_index()
tmp_5 = tmp_5.rename(columns = {'第4期应付租金': '第4期DPD>15的应付租金'})

tmp = tmp_1.merge(tmp_2, on ='search_time_new', how = 'left')
tmp = tmp.merge(tmp_3, on ='search_time_new', how = 'left')
tmp = tmp.merge(tmp_4, on ='search_time_new', how = 'left')
tmp = tmp.merge(tmp_5, on ='search_time_new', how = 'left')
tmp

(31689, 16)

(31687, 16)

,search_time_new,purchase_amount,第2期应付租金,第4期应付租金,第2期DPD>15的应付租金,第4期DPD>15的应付租金
0,2022-07,125405.00,11390.18,11390.18,1212.61,4901.46
1,2022-08,358451.00,27280.03,27280.03,4262.44,8338.21
2,2022-09,277982.75,19537.22,19537.22,1967.61,3710.61
3,2022-10,249543.09,15908.60,15908.60,1286.10,1680.30
4,2022-11,344258.81,21414.90,21414.90,2730.90,3076.80
5,2022-12,566276.59,38587.21,38587.21,4440.50,8003.60
6,2023-01,1533459.00,118207.40,118207.40,8011.20,20208.30
7,2023-02,1920586.82,138856.26,138856.26,12821.33,24815.47
8,2023-03,2076961.00,150046.90,150046.90,13894.61,27113.53
9,2023-04,4209742.50,301021.00,301021.00,14531.23,48936.75


In [ ]:
# （3）催回流水
# 备注：目前系统没有部分还款的功能，客户逾期后，还款到公司对公账户，若部分还款（应付租金部分还款），则要等到客户全部还完租金后才去认领并录入系统，故部分还款在未结清之前无法统计，若是买断/部分买断，及时入账
# df_jx_data[df_jx_data.order_id.isin([870704])]
# sql = '''-- 租后应收监控  
# SELECT  yata.order_number
#        ,yata.amount as '实还金额'
#        ,yata.add_time as '实还日期'
#        ,yata.explain
# from db_rent.ya_agent_task_account yata
# ;
# '''

# df_hkls = query(sql)
# df_hkls["实还日期"] = df_hkls['实还日期'].dt.date
# df_hkls["实还日期"] = pd.to_datetime(df_hkls["实还日期"],errors="coerce")
# # df_hkls['sort'] = df_hkls.apply(lambda x: x.explain[-5:-2], axis = 1)
# df_hkls['is_2'] = np.where(df_hkls['explain'].str.contains('第2期租金'),1,0)
# df_hkls['is_4'] = np.where(df_hkls['explain'].str.contains('第4期租金'),1,0)
# df_hkls.shape
# # df_hkls['实还金额'].sum()
# # df_hkls.tail()

# df_hkls_2 = df_hkls[df_hkls.is_2 == 1]
# df_hkls_4 = df_hkls[df_hkls.is_4 == 1]
# df_hkls_2.drop_duplicates(subset = ['order_number']).shape
# df_hkls_2.shape
# df_hkls_4.drop_duplicates(subset = ['order_number']).shape
# df_hkls_4.shape

# df_hkls_2 =df_hkls_2.merge(df_jx_data[['order_number', '第2期应付日期']], on = 'order_number', how = 'inner')
# df_hkls_2['第2期实付应付相差天数'] = np.where(df_hkls_2['实还日期'].notnull(), (df_hkls_2['实还日期'] - df_hkls_2['第2期应付日期']).dt.days, 999)
# df_hkls_2.shape

# df_hkls_4 =df_hkls_4.merge(df_jx_data[['order_number', '第4期应付日期']], on = 'order_number', how = 'inner')
# df_hkls_4['第4期实付应付相差天数'] = np.where(df_hkls_4['实还日期'].notnull(), (df_hkls_4['实还日期'] - df_hkls_4['第4期应付日期']).dt.days, 999)
# df_hkls_4.shape

# df_hkls_2_1 = df_hkls_2[df_hkls_2.第2期实付应付相差天数 <= 15].groupby(by=['order_number']).agg({'实还金额': 'sum'}).reset_index()
# df_hkls_4_1 = df_hkls_4[df_hkls_4.第4期实付应付相差天数 <= 15].groupby(by=['order_number']).agg({'实还金额': 'sum'}).reset_index()

# df_hkls_2_1.shape
# df_hkls_4_1.shape

(23418, 6)

(978, 6)

(978, 6)

(1806, 6)

(1808, 6)

(978, 8)

(1808, 8)

(729, 2)

(1200, 2)

In [177]:
# （4）表关联
# # df_jx_data = df_jx_data.merge(df_hkls_2_1, on = 'order_number', how = 'left')
# # df_jx_data = df_jx_data.merge(df_hkls_4_1, on = 'order_number', how = 'left')
# # # df_jx_data = df_jx_data.rename(columns = {'实还金额_x': '', '实还金额_y'})
# df_jx_data.shape
# # df_jx_data[:1]

# df_jx_data[(df_jx_data.第二期逾期天数 > 15) & (df_jx_data.实还金额_x > 0)].shape
# df_jx_data[(df_jx_data.第四期逾期天数 > 15) & (df_jx_data.实还金额_y > 0)].shape
# df_jx_data[(df_jx_data.第四期逾期天数 > 15) & (df_jx_data.实还金额_y > 0)]
# df_hkls_4[df_hkls_4.order_number == 'A2024092315064269']
# df_analy_data[df_analy_data.order_number == 'A2024092315064269']


(31594, 20)

(0, 20)

(1, 20)

,order_number,order_id,下单日期,search_time,第二期逾期天数,第四期逾期天数,首期应还日期,purchase_amount,paid_money_mob2,paid_money_mob4,第2期应付租金,第4期应付租金,第2期应付日期,第4期应付日期,第2期实付日期new,第4期实付日期new,第2期实付应付相差天数,第4期实付应付相差天数,实还金额_x,实还金额_y
21944,A2024092315064269,1513430,2024-09-23,2024-09,0.0,28.0,2024-09-26,7675.0,457.98,457.98,457.98,457.98,2024-10-26,2024-12-26,2024-10-24,NaT,-2.0,999.0,NaN,449.58


,order_number,实还金额,实还日期,explain,is_2,is_4,第4期应付日期,第4期实付应付相差天数
1742,A2024092315064269,449.58,2025-01-09,A2024092315064269第4期租金,0,1,2024-12-26,14


,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,首期应还日期,overdue_type
21946,A2024092315064269,1513430,2024-09-23,2024-09,芝麻租物,全新,未带锁,58,0.0,58.0,0.0,0.0,58.0,28.0,2024-09-26,


In [ ]:
# （5）数据检查
# df_jx_data[(df_jx_data.search_time == '2023-07')].shape
# df_jx_data[(df_jx_data.search_time == '2023-07')&(df_jx_data.第四期逾期天数 > 15)].shape
# df_jx_data[(df_jx_data.search_time == '2023-07')].purchase_amount.sum()
# df_jx_data[(df_jx_data.search_time == '2023-07')].第4期应付租金.sum()
# df_jx_data[(df_jx_data.search_time == '2023-07')&(df_jx_data.第四期逾期天数 > 15)].第4期应付租金.sum()
# df_jx_data[(df_jx_data.search_time == '2023-07')&(df_jx_data.第四期逾期天数 > 15)][:5]

# sql_xz = ''' 
# -- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
# -- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
# SELECT ymos.*
# from db_rent.ya_merchant_order_stages ymos 
# inner join db_digua_business.t_postlease_receivables_monitoring tprm 
# on ymos.order_id = tprm.order_id 
# where tprm.order_number = 'A2024092315064269'
# ;
# '''

# query(sql_xz)

# 当前期数
# 逾期天数
# purchase_amount
# paid_money_mob2：第2期实付金额
# paid_money_mob4：第4期实付金额
# 当前应付租金
# 实付日期new
# 应付日期

# df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (df_xz_concat.实付日期.notnull())].shape
# df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (df_xz_concat.实付日期.notnull())].drop_duplicates(subset = ['order_number']).shape
# df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (df_xz_concat.实付日期.notnull())][['order_number','当前期数','逾期天数','purchase_amount','paid_money_mob2','paid_money_mob4','当前应付租金','应付日期','实付日期new','实付日期']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092315064269'][['order_number','当前期数','逾期天数','purchase_amount','paid_money_mob2','paid_money_mob4','当前应付租金','应付日期','实付日期new','实付日期']]

(256, 160)

(87, 160)

# 19_ 黑名单回传

## 19_1.2025.2

In [212]:
# （1）疑似黑名单
# 鑫仲导数
path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'
df_black_1 = pd.read_excel(path + '黑名单明细_20250220.xlsx')
df_black_1['添加日期'] = df_black_1['添加日期'].astype('str')
df_black_1.shape
df_black_1[:1]

(86086, 4)

,添加日期,id_card,mobile,命中项
0,2023-01-05,520181199409263014,15286053847,命中法院限高/被执行人强拒


In [ ]:
# （2）逾期黑名单
temp_1 = df_analy_data[df_analy_data.历史最大逾期天数 > 180]
temp_1 = temp_1[['order_number']]
temp_1['添加日期'] = '2025-02-20'
temp_1['命中项'] = '历史最大逾期天数大于180'
temp_1.shape

# 关联属性信息
sql = '''-- 租后应收监控  
SELECT  om.id as order_id, om.order_number, om.create_time, om.merchant_name
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
, tmu.true_name, tmu.id_card_num, tmu.mobile 
, tod.sku_attributes
, top.total_describes
, pa.name as activity_name        -- 活动名称
from db_digua_business.t_order om
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on om.id = tod.order_id
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
;
'''

df_info = query(sql)
df_info.shape
# df_info[:2]

# 关联
temp_2 = temp_1.merge(df_info[['order_number','id_card_num','mobile']], on = 'order_number', how = 'left')
temp_2 = temp_2.rename(columns = {'id_card_num': 'id_card'})
# temp_2[temp_2.order_number.isin(['A20240618153424174','A2024062316085438'])]
temp_2 = temp_2.drop(columns = ['order_number'])
temp_2.shape

df_black_list = pd.concat([df_black_1, temp_2])
df_black_list['添加日期'] = df_black_list['添加日期'].astype('str')
df_black_list.shape
df_black_list = df_black_list.drop_duplicates(subset = ['id_card','mobile','命中项','添加日期'])
df_black_list.shape
df_black_list[:2]
df_black_list.命中项.value_counts()

df_black_list = df_black_list[df_black_list.命中项.isin(['命中法院限高/被执行人强拒','命中案件纠纷判定强拒','历史最大逾期天数大于180'])]
df_black_list.shape

(3085, 3)

(3085, 4)

(89171, 4)

(89163, 4)

,添加日期,id_card,mobile,命中项
0,2023-01-05,520181199409263014,15286053847,命中法院限高/被执行人强拒
1,2023-01-09,320121199504210929,13305197371,命中案件纠纷判定强拒


命中项
命中法院限高/被执行人强拒           44978
命中特殊名单|地区|策略验证-高风险强拒    15918
命中案件纠纷判定强拒               9454
命中强拒加入临时黑名单              8324
客服手动添加                   4355
历史最大逾期天数大于180            3077
命中行业黑名单强拒                2913
命中疑似欺诈强拒                   94
其他                         50
Name: count, dtype: int64

In [225]:
# （3）加密
import hashlib
 
def sha256_encryption(var):
    sha256_hash = hashlib.sha256()  # 创建一个md5对象
    sha256_hash.update(var.encode('utf-8'))  # 使用utf-8编码数据
    return sha256_hash.hexdigest()  # 返回加密后的十六进制字符串

df_black_list['id_card'] = df_black_list['id_card'].astype('str')
df_black_list['mobile'] = df_black_list['mobile'].astype('str')
df_black_list['idcard_sha256'] = df_black_list.apply(lambda x: sha256_encryption(x.id_card), axis = 1)
df_black_list['mobile_sha256'] = df_black_list.apply(lambda x: sha256_encryption(x.mobile), axis = 1)
df_black_list.shape
df_black_list.tail()

path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'
df_black_list.to_excel(path + 'blacklist_20250220.xlsx', index = False)

(57509, 6)

,添加日期,id_card,mobile,命中项,idcard_sha256,mobile_sha256
3080,2025-02-20,341002200011190611,15955598608,历史最大逾期天数大于180,96fba209eb650a5c128f6a92fa61ed5db78467eb2f7886...,31292d712bda6f1566707ef7f7f80267e51b441c38cfa3...
3081,2025-02-20,511621199605060011,17628230506,历史最大逾期天数大于180,7c4fd509be97d198cd8af9d1ec9063f73f303d4bfbe5fa...,7064b622e106130d59bbf749e74d088e4eb2b8506beced...
3082,2025-02-20,412726198204066762,18803869132,历史最大逾期天数大于180,09dc222ed7503a07cb0c57f2f3a1853416a247d888a8f6...,d08900216b60d855815e1073a590ed76199989a6ace1f3...
3083,2025-02-20,32072319781203242X,17826536555,历史最大逾期天数大于180,bc288af031a5cd9756047efa2fb00753b7307c60f561db...,61d933a0b5393949ec0518e1f85186f1aefaaca9b9e5c9...
3084,2025-02-20,341004199507010618,15683844338,历史最大逾期天数大于180,91afab83f5577c213c76ebcc782736b857446b733c3217...,278a6bb65b12440387bbc155ce54e84d9c5aeae87b5913...


In [ ]:
# （4）数据检查
# df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 > '2024-07-01')][['order_number']][:3]
# A20240702112528105
# 14232719930414061X
# df_info[df_info.order_number.isin(['A20240702112528105'])]
# A20240702112528105	2024-07-02	bd6c5d76798672c06a27f8ef82b85b931ebe430a53ddfdf88e43f448532baf25
# 14232719930414061X	15735410456  bd6c5d76798672c06a27f8ef82b85b931ebe430a53ddfdf88e43f448532baf25

,order_number
15714,A20240702024619113
15732,A20240702112528105
15815,A20240703145421290


,order_id,order_number,create_time,merchant_name,订单状态,true_name,id_card_num,mobile,sku_attributes,total_describes,activity_name
1158750,1202295,A20240702112528105,2024-07-02 11:25:28,维客自营账户,租赁中,王天养,14232719930414061X,15735410456,"[{""type"":""text"",""value"":""租完归还/可买断"",""key"":""租赁方案...","[""煤矿上班6年，月收入8k，芝麻分624，+v"", ""修改收货信息：将原收货信息 王先生 ...",抖音直播628首月降价引流款


# 20.号卡分析

## 20_1.2025.2

In [67]:
# （1）导数
path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'
df_hk_1 = pd.read_excel(path + '联通号卡在租_20250221.xlsx')
df_hk_1 = df_hk_1.rename(columns = {'订单号': 'order_number'})
df_hk_1.shape
df_hk_1[:2]

(164, 4)

,下单时间,order_number,下单手机号,号卡手机号
0,2024-09-29 19:03:33,A2024092919033313,15207243378,13268376979
1,2024-11-05 21:07:33,A2024110521073310,18598279743,13126473385


In [68]:
# （2）关联历史逾期
var = ['order_number','search_time','overdue_type','FPD','历史最大逾期天数']
df_hk_2 = df_hk_1.merge(df_analy_data[var], on = 'order_number', how = 'left')
df_hk_2.shape
df_hk_2[:2]

(164, 8)

,下单时间,order_number,下单手机号,号卡手机号,search_time,overdue_type,FPD,历史最大逾期天数
0,2024-09-29 19:03:33,A2024092919033313,15207243378,13268376979,2024-09,,0.0,0.0
1,2024-11-05 21:07:33,A2024110521073310,18598279743,13126473385,2024-11,,0.0,0.0


In [69]:
# （3）统计
df_hk_2['tmp1'] = np.where(df_hk_2.overdue_type.isin(['M0','M1','M2','M3','M6']), 1, 0)
df_hk_2['tmp2'] = np.where(df_hk_2.FPD >= 7, 1, 0)
df_hk_2['tmp3'] = np.where(df_hk_2.历史最大逾期天数 >= 1, 1, 0)
df_hk_2['tmp4'] = np.where(df_hk_2.历史最大逾期天数 > 30, 1, 0)

df_hk_2.groupby(['search_time']).agg({'order_number':'count','tmp1':'sum','tmp2':'sum','tmp3':'sum','tmp4':'sum'}).reset_index()

df_hk_2[df_hk_2.历史最大逾期天数 >= 1]
# A2024120521403595 2
# A20250111153226165 2

,search_time,order_number,tmp1,tmp2,tmp3,tmp4
0,2024-09,1,0,0,0,0
1,2024-11,12,0,0,0,0
2,2024-12,16,0,0,1,0
3,2025-01,84,0,0,1,0
4,2025-02,49,0,0,0,0


,下单时间,order_number,下单手机号,号卡手机号,search_time,overdue_type,FPD,历史最大逾期天数,tmp1,tmp2,tmp3,tmp4
17,2024-12-05 21:40:36,A2024120521403595,19958224897,16620844951,2024-12,,2.0,2.0,0,0,1,0
65,2025-01-11 15:32:26,A20250111153226165,15889970976,13113969222,2025-01,,2.0,2.0,0,0,1,0
